In [2]:
import pandas as pd
import numpy as np
import requests as r
import json
import re

In [3]:
#setting konfigurasi
with open("env.json", "r", encoding="utf-8") as file:
    config = json.load(file)  # Membaca dan parsing JSON

In [4]:
#Get Daftar Data Dinamis BPS Bali
def getVarByDomain(domain='5100'):
    global config
    var_pages=[]
    var = r.get(f"https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/key/{config['token']}/")
    pages=var.json()['data'][0]['pages']
    current_page = 1
    while current_page <= pages:
        url = f"https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/page/{current_page}/key/{config['token']}/"
        var_page=r.get(url)
        var_pages.append(pd.json_normalize(var_page.json()['data'][1]))
        current_page=current_page+1
    var_pages_df = pd.concat(var_pages)
    return var_pages_df

In [5]:
#Get Daftar Periode
def getPriodeByVar(domain='5100',varId='106'):
    global config
    current_page = 1
    url = f"https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/"
    dt_json = r.get(url)
    pages = dt_json.json()['data'][0]['pages']
    dt_all = []
    while current_page <= pages:
        url = f"https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/"
        dt = pd.json_normalize(dt_json.json()['data'][1])
        dt_all.append(dt)
        current_page = current_page+1
    dt_all = pd.concat(dt_all)
    dt_all['varId']=varId
    return dt_all

In [7]:
def crossJoin(left,right):
    if left.empty:
        left = right
    else:
        left = pd.merge(left,right,how='cross')
    return left
def getDataByVarId(varId='106',domain='5100'):
    global config
    #Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk
    dt_th = getPriodeByVar(domain=domain,varId=varId)
    dt_all = []
    for i in dt_th.index:
        th_id = dt_th.iloc[i]['th_id']
        url = f"https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain}/var/{varId}/key/{config['token']}/th/{th_id}/"
        dt_json = r.get(url)
        attr = pd.DataFrame()
        if 'vervar' in dt_json.json().keys():
            vervar=pd.json_normalize(dt_json.json()['vervar'])
            vervar=vervar.rename(columns={'val': 'verval_val','label': 'vervar_label'})
            attr = crossJoin(attr,vervar)
        if 'var' in dt_json.json().keys():
            var=pd.json_normalize(dt_json.json()['var'])
            var=var.rename(columns={'val': 'var_val','label': 'var_label'})
            var=var[['var_val']]
            attr = crossJoin(attr,var)
        if 'turvar' in dt_json.json().keys():
            turvar=pd.json_normalize(dt_json.json()['turvar'])
            turvar=turvar.rename(columns={'val': 'turvar_val','label': 'turvar_label'})
            attr = crossJoin(attr,turvar)
        if 'tahun' in dt_json.json().keys():
            tahun=pd.json_normalize(dt_json.json()['tahun'])
            tahun=tahun.rename(columns={'val': 'tahun_val','label': 'tahun_label'})
            attr = crossJoin(attr,tahun)
        if 'turtahun' in dt_json.json().keys():
            turtahun=pd.json_normalize(dt_json.json()['turtahun'])
            turtahun=turtahun.rename(columns={'val': 'turtahun_val','label': 'turtahun_label'})
            attr = crossJoin(attr,turtahun)
        val_columns = [col for col in attr.columns if col.endswith('val')]
        attr['id']=attr[val_columns].astype('str').agg(''.join, axis=1)
        dt_content=pd.json_normalize(dt_json.json()['datacontent']).transpose().reset_index().rename(columns={'index':'content_val',0:'content_label'})
        attr = pd.merge(attr,dt_content,left_on='id',right_on='content_val',how='left')
        attr = attr[attr.content_label.notna()]
        dt_all.append(attr)
    dt_all=pd.concat(dt_all)
    return dt_all

Ambil Data dari  Masing-Masing

339 (Tingkat Partisipasi Angkatan Kerja Provinsi Bali Menurut Kabupaten/Kota dan Jenis Kelamin)

In [81]:
data_399 = getDataByVarId(domain='5100', varId='399')
data_399.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,399,72,Laki-Laki,124,2024,0,Tahun,1399721240,1399721240,86.78
1,1,Kab. Jembrana,399,74,Perempuan,124,2024,0,Tahun,1399741240,1399741240,73.49
2,2,Kab. Tabanan,399,72,Laki-Laki,124,2024,0,Tahun,2399721240,2399721240,83.17
3,2,Kab. Tabanan,399,74,Perempuan,124,2024,0,Tahun,2399741240,2399741240,67.59
4,3,Kab. Badung,399,72,Laki-Laki,124,2024,0,Tahun,3399721240,3399721240,83.16


In [ ]:
# --- cleaning dasar ---
data_399 = data_399[['vervar_label','turvar_label','tahun_label','content_label']].copy()
data_399['vervar_label'] = data_399['vervar_label'].str.replace(r"<.*?>", "", regex=True).str.strip()
data_399['turvar_label'] = data_399['turvar_label'].astype(str).str.strip()     # sudah "Laki-Laki"/"Perempuan"
data_399['tahun_label']  = data_399['tahun_label'].astype(str).str.strip()

# nilai → float (tangani format BPS)
data_399['content_label'] = (data_399['content_label'].astype(str)
                       .str.replace(r'<.*?>','', regex=True)
                       .str.replace(r'[^\d,.\-]', '', regex=True)
                       .str.replace(',', '.', regex=False)
                       .replace({'': np.nan, '-': np.nan})
                       .astype(float))

# rename konsisten
df = data_399.rename(columns={
    'vervar_label':'kabupaten_kota',
    'turvar_label':'jenis_kelamin',
    'tahun_label' :'tahun',
    'content_label':'nilai'
})

# tahun numerik & simpan urutan kemunculan kab/kota
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)
row_order = df['kabupaten_kota'].drop_duplicates().tolist()  # <-- urutan baris tanpa daftar manual

# --- pivot: baris = kab/kota, kolom = (jenis_kelamin, tahun) ---
wide = df.pivot_table(index='kabupaten_kota',
                    columns=['jenis_kelamin','tahun'],
                    values='nilai',
                    aggfunc='first',   # tidak mengubah angka
                    sort=False)

# urutan kolom: per tahun (naik), dalam tahun: Laki-Laki lalu Perempuan
gender_order = [g for g in ['Laki-Laki','Perempuan'] if g in wide.columns.get_level_values(0)]
wide = wide.reindex(gender_order, level=0, axis=1)
wide = wide.sort_index(axis=1, level=[1,0])  # level1=tahun, level0=gender

# urut baris sesuai kemunculan awal
wide = wide.reindex(row_order)

# --- flatten kolom (mis. 'Laki-Laki_2016') ---
wide = wide.reset_index()
wide.columns = [c if not isinstance(c, tuple) else f"{c[0]}_{c[1]}" for c in wide.columns]

# opsional: pembulatan 2 desimal
num_cols = [c for c in wide.columns if c != 'kabupaten_kota']
wide[num_cols] = wide[num_cols].round(2)


In [85]:
wide

,kabupaten_kota_,Laki-Laki_2018,Perempuan_2018,Laki-Laki_2019,Perempuan_2019,Laki-Laki_2020,Perempuan_2020,Laki-Laki_2021,Perempuan_2021,Laki-Laki_2022,Perempuan_2022,Laki-Laki_2023,Perempuan_2023,Laki-Laki_2024,Perempuan_2024
0,Kab. Jembrana,85.21,69.61,78.91,56.15,83.12,68.94,90.38,72.42,85.91,75.46,85.49,74.67,86.78,73.49
1,Kab. Tabanan,83.65,71.11,83.12,67.94,81.26,69.84,81.43,68.72,83.15,70.89,81.28,67.17,83.17,67.59
2,Kab. Badung,79.61,62.38,80.95,63.94,81.25,63.82,77.65,67.21,80.15,65.19,81.42,66.21,83.16,68.07
3,Kab. Gianyar,83.09,74.79,82.37,69.78,78.11,64.36,76.13,63.55,85.10,75.38,82.08,72.53,83.70,69.46
4,Kab. Klungkung,80.49,75.37,80.34,71.95,78.69,72.01,75.38,70.39,85.10,74.76,84.97,74.82,82.89,71.58
5,Kab. Bangli,88.70,82.68,85.79,80.38,86.22,78.11,84.98,79.16,87.24,79.42,86.50,79.56,88.02,76.49
6,Kab. Karangasem,84.17,80.42,83.99,77.28,83.23,78.31,84.65,77.70,88.69,82.15,89.50,82.52,87.41,80.11
7,Kab. Buleleng,82.68,70.67,78.41,60.82,80.87,69.40,80.35,66.00,85.85,65.18,83.82,67.18,84.77,74.78
8,Kota Denpasar,82.94,63.68,83.02,59.88,78.91,62.55,75.33,61.71,82.36,61.94,81.31,64.84,78.80,60.50
9,Provinsi Bali,82.94,70.14,81.82,65.67,80.75,67.86,79.44,67.61,84.06,69.62,83.55,70.63,83.77,70.49


In [86]:
wide.reset_index().to_excel("data_399.xlsx", index=False, engine="openpyxl")

408 ([Metode Baru] Umur Harapan Hidup Saat Lahir (UHH) Menurut Kabupaten/Kota dan Jenis Kelamin di Provinsi Bali Hasil Long Form SP2020)

In [87]:
data_408 = getDataByVarId(domain='5100', varId='408')
data_408.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,408,72,Laki-Laki,124,2024,0,Tahun,1408721240,1408721240,72.40
1,1,Kab. Jembrana,408,74,Perempuan,124,2024,0,Tahun,1408741240,1408741240,77.87
2,2,Kab. Tabanan,408,72,Laki-Laki,124,2024,0,Tahun,2408721240,2408721240,73.43
3,2,Kab. Tabanan,408,74,Perempuan,124,2024,0,Tahun,2408741240,2408741240,77.63
4,3,Kab. Badung,408,72,Laki-Laki,124,2024,0,Tahun,3408721240,3408721240,74.11


In [88]:
# --- cleaning dasar ---
data_408 = data_408[['vervar_label','turvar_label','tahun_label','content_label']].copy()
data_408['vervar_label'] = data_408['vervar_label'].str.replace(r"<.*?>", "", regex=True).str.strip()
data_408['turvar_label'] = data_408['turvar_label'].astype(str).str.strip()     # sudah "Laki-Laki"/"Perempuan"
data_408['tahun_label']  = data_408['tahun_label'].astype(str).str.strip()

# nilai → float (tangani format BPS)
data_408['content_label'] = (data_408['content_label'].astype(str)
                       .str.replace(r'<.*?>','', regex=True)
                       .str.replace(r'[^\d,.\-]', '', regex=True)
                       .str.replace(',', '.', regex=False)
                       .replace({'': np.nan, '-': np.nan})
                       .astype(float))

# rename konsisten
df = data_408.rename(columns={
    'vervar_label':'kabupaten_kota',
    'turvar_label':'jenis_kelamin',
    'tahun_label' :'tahun',
    'content_label':'nilai'
})

# tahun numerik & simpan urutan kemunculan kab/kota
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)
row_order = df['kabupaten_kota'].drop_duplicates().tolist()  # <-- urutan baris tanpa daftar manual

# --- pivot: baris = kab/kota, kolom = (jenis_kelamin, tahun) ---
wide = df.pivot_table(index='kabupaten_kota',
                    columns=['jenis_kelamin','tahun'],
                    values='nilai',
                    aggfunc='first',   # tidak mengubah angka
                    sort=False)

# urutan kolom: per tahun (naik), dalam tahun: Laki-Laki lalu Perempuan
gender_order = [g for g in ['Laki-Laki','Perempuan'] if g in wide.columns.get_level_values(0)]
wide = wide.reindex(gender_order, level=0, axis=1)
wide = wide.sort_index(axis=1, level=[1,0])  # level1=tahun, level0=gender

# urut baris sesuai kemunculan awal
wide = wide.reindex(row_order)

# --- flatten kolom (mis. 'Laki-Laki_2016') ---
wide = wide.reset_index()
wide.columns = [c if not isinstance(c, tuple) else f"{c[0]}_{c[1]}" for c in wide.columns]

# opsional: pembulatan 2 desimal
num_cols = [c for c in wide.columns if c != 'kabupaten_kota']
wide[num_cols] = wide[num_cols].round(2)


In [89]:
wide

,kabupaten_kota_,Laki-Laki_2020,Perempuan_2020,Laki-Laki_2021,Perempuan_2021,Laki-Laki_2022,Perempuan_2022,Laki-Laki_2023,Perempuan_2023,Laki-Laki_2024,Perempuan_2024
0,Kab. Jembrana,71.76,76.68,71.82,76.80,72.02,77.22,72.22,77.60,72.40,77.87
1,Kab. Tabanan,72.13,76.67,72.19,76.80,72.43,76.96,73.17,77.05,73.43,77.63
2,Kab. Badung,73.28,77.02,73.37,77.09,73.64,77.15,73.88,77.24,74.11,78.01
3,Kab. Gianyar,72.15,76.69,72.22,76.81,72.45,76.90,73.23,77.00,73.49,77.65
4,Kab. Klungkung,71.64,76.43,71.73,76.62,71.97,77.12,72.05,77.30,72.26,77.66
5,Kab. Bangli,71.19,75.48,71.24,75.59,71.44,76.01,71.64,76.44,71.81,76.78
6,Kab. Karangasem,71.07,75.23,71.24,75.58,71.42,75.96,71.61,76.37,71.77,76.70
7,Kab. Buleleng,71.73,76.63,71.80,76.77,72.01,77.21,72.08,77.35,72.12,77.43
8,Kota Denpasar,73.01,76.63,73.10,77.73,73.42,78.18,73.81,78.59,74.08,78.74
9,Provinsi Bali,71.83,76.83,71.88,76.93,72.05,77.28,72.25,77.64,72.43,77.91


In [ ]:
wide.reset_index().to_excel("data_408.xlsx", index=False, engine="openpyxl")

410 ([Metode Baru] Indeks Pembangunan Manusia Menurut Kabupaten/Kota dan Jenis Kelamin di Provinsi Bali (Umur Harapan Hidup Hasil Long Form SP2020))

In [90]:
data_410 = getDataByVarId(domain='5100', varId='410')
data_410.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,410,72,Laki-Laki,124,2024,0,Tahun,1410721240,1410721240,77.71
1,1,Kab. Jembrana,410,74,Perempuan,124,2024,0,Tahun,1410741240,1410741240,74.21
2,2,Kab. Tabanan,410,72,Laki-Laki,124,2024,0,Tahun,2410721240,2410721240,80.18
3,2,Kab. Tabanan,410,74,Perempuan,124,2024,0,Tahun,2410741240,2410741240,77.10
4,3,Kab. Badung,410,72,Laki-Laki,124,2024,0,Tahun,3410721240,3410721240,85.97


In [91]:
# --- cleaning dasar ---
data_410 = data_410[['vervar_label','turvar_label','tahun_label','content_label']].copy()
data_410['vervar_label'] = data_410['vervar_label'].str.replace(r"<.*?>", "", regex=True).str.strip()
data_410['turvar_label'] = data_410['turvar_label'].astype(str).str.strip()     # sudah "Laki-Laki"/"Perempuan"
data_410['tahun_label']  = data_410['tahun_label'].astype(str).str.strip()

# nilai → float (tangani format BPS)
data_410['content_label'] = (data_410['content_label'].astype(str)
                       .str.replace(r'<.*?>','', regex=True)
                       .str.replace(r'[^\d,.\-]', '', regex=True)
                       .str.replace(',', '.', regex=False)
                       .replace({'': np.nan, '-': np.nan})
                       .astype(float))

# rename konsisten
df = data_410.rename(columns={
    'vervar_label':'kabupaten_kota',
    'turvar_label':'jenis_kelamin',
    'tahun_label' :'tahun',
    'content_label':'nilai'
})

# tahun numerik & simpan urutan kemunculan kab/kota
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)
row_order = df['kabupaten_kota'].drop_duplicates().tolist()  # <-- urutan baris tanpa daftar manual

# --- pivot: baris = kab/kota, kolom = (jenis_kelamin, tahun) ---
wide = df.pivot_table(index='kabupaten_kota',
                    columns=['jenis_kelamin','tahun'],
                    values='nilai',
                    aggfunc='first',   # tidak mengubah angka
                    sort=False)

# urutan kolom: per tahun (naik), dalam tahun: Laki-Laki lalu Perempuan
gender_order = [g for g in ['Laki-Laki','Perempuan'] if g in wide.columns.get_level_values(0)]
wide = wide.reindex(gender_order, level=0, axis=1)
wide = wide.sort_index(axis=1, level=[1,0])  # level1=tahun, level0=gender

# urut baris sesuai kemunculan awal
wide = wide.reindex(row_order)

# --- flatten kolom (mis. 'Laki-Laki_2016') ---
wide = wide.reset_index()
wide.columns = [c if not isinstance(c, tuple) else f"{c[0]}_{c[1]}" for c in wide.columns]

# opsional: pembulatan 2 desimal
num_cols = [c for c in wide.columns if c != 'kabupaten_kota']
wide[num_cols] = wide[num_cols].round(2)


In [92]:
wide

,kabupaten_kota_,Laki-Laki_2020,Perempuan_2020,Laki-Laki_2021,Perempuan_2021,Laki-Laki_2022,Perempuan_2022,Laki-Laki_2023,Perempuan_2023,Laki-Laki_2024,Perempuan_2024
0,Kab. Jembrana,76.22,71.68,76.23,72.09,76.76,73.07,77.19,73.49,77.71,74.21
1,Kab. Tabanan,78.24,74.99,78.51,75.34,78.77,75.56,79.44,76.14,80.18,77.10
2,Kab. Badung,83.97,80.31,84.20,80.59,84.46,80.82,85.35,81.68,85.97,82.64
3,Kab. Gianyar,80.07,76.15,80.39,76.56,81.05,77.11,81.89,77.92,82.43,78.56
4,Kab. Klungkung,77.71,71.24,77.68,71.33,78.34,72.16,78.69,72.80,79.34,73.77
5,Kab. Bangli,74.98,68.82,74.96,68.86,75.67,69.77,76.07,70.53,76.59,71.36
6,Kab. Karangasem,73.60,65.58,73.63,65.73,73.93,66.75,74.35,67.39,74.87,68.30
7,Kab. Buleleng,77.82,71.77,77.81,71.85,78.58,72.78,78.93,73.17,79.35,73.78
8,Kota Denpasar,85.68,82.90,85.67,83.52,85.99,83.90,86.44,84.18,86.88,84.61
9,Provinsi Bali,79.50,75.16,79.63,75.47,80.10,76.25,80.64,76.91,81.20,77.79


In [ ]:
wide.reset_index().to_excel("data_410.xlsx", index=False, engine="openpyxl")

411 ([Metode Baru] Indeks Pembangunan Gender (IPG) Provinsi Bali Menurut Kabupaten/Kota (Umur Harapan Hidup Hasil Long Form SP2020))

In [93]:
data_411 = getDataByVarId(domain='5100', varId='411')
data_411.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,411,0,Tidak ada,124,2024,0,Tahun,141101240,141101240,95.50
1,2,Kab. Tabanan,411,0,Tidak ada,124,2024,0,Tahun,241101240,241101240,96.16
2,3,Kab. Badung,411,0,Tidak ada,124,2024,0,Tahun,341101240,341101240,96.13
3,4,Kab. Gianyar,411,0,Tidak ada,124,2024,0,Tahun,441101240,441101240,95.31
4,5,Kab. Klungkung,411,0,Tidak ada,124,2024,0,Tahun,541101240,541101240,92.98


In [94]:
# --- ambil & bersihkan untuk urutan asli ---
data_411 = data_411[['vervar_label','tahun_label','content_label']].copy()
data_411['kabupaten_kota'] = (data_411['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_411['kabupaten_kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_411.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['kabupaten_kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='kabupaten_kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [95]:
wide

tahun,2020,2021,2022,2023,2024
kabupaten_kota,,,,,
Kab. Jembrana,94.04,94.57,95.19,95.21,95.50
Kab. Tabanan,95.85,95.96,95.92,95.85,96.16
Kab. Badung,95.64,95.71,95.69,95.70,96.13
Kab. Gianyar,95.10,95.24,95.14,95.15,95.31
Kab. Klungkung,91.67,91.83,92.11,92.51,92.98
Kab. Bangli,91.78,91.86,92.20,92.72,93.17
Kab. Karangasem,89.10,89.27,90.29,90.64,91.22
Kab. Buleleng,92.23,92.34,92.62,92.70,92.98
Kota Denpasar,96.76,97.49,97.57,97.39,97.39


In [ ]:
wide.reset_index().to_excel("data_411.xlsx", index=False, engine="openpyxl")

111 (Pertumbuhan PDRB/Ekonomi Kabupaten/Kota di Provinsi Bali)

In [96]:
data_111 = getDataByVarId(domain='5100', varId='111')
data_111.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,111,0,Tidak ada,124,2024,0,Tahun,111101240,111101240,4.98
1,2,Kab. Tabanan,111,0,Tidak ada,124,2024,0,Tahun,211101240,211101240,4.94
2,3,Kab. Badung,111,0,Tidak ada,124,2024,0,Tahun,311101240,311101240,5.94
3,4,Kab. Gianyar,111,0,Tidak ada,124,2024,0,Tahun,411101240,411101240,5.47
4,5,Kab. Klungkung,111,0,Tidak ada,124,2024,0,Tahun,511101240,511101240,5.08


In [97]:
# --- ambil & bersihkan untuk urutan asli ---
data_111 = data_111[['vervar_label','tahun_label','content_label']].copy()
data_111['kabupaten_kota'] = (data_111['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_111['kabupaten_kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_111.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['kabupaten_kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='kabupaten_kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [98]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kabupaten_kota,,,,,,,,,,
Kab. Jembrana,6.19,5.96,5.28,5.59,5.56,-4.98,-0.65,2.98,3.66,4.98
Kab. Tabanan,6.19,6.14,5.37,5.71,5.58,-6.17,-1.98,2.93,3.56,4.94
Kab. Badung,6.24,6.81,6.08,6.73,5.81,-16.55,-6.74,9.97,11.29,5.94
Kab. Gianyar,6.30,6.31,5.46,6.01,5.62,-8.39,-1.05,4.04,5.06,5.47
Kab. Klungkung,6.11,6.28,5.32,5.48,5.42,-6.38,-0.23,3.12,4.70,5.08
Kab. Bangli,6.16,6.24,5.31,5.48,5.45,-4.10,-0.33,2.80,3.50,4.54
Kab. Karangasem,6.00,5.92,5.06,5.44,5.50,-4.49,-0.56,2.58,3.10,4.33
Kab. Buleleng,6.07,6.02,5.38,5.60,5.51,-5.80,-1.27,3.11,3.64,5.04
Kota Denpasar,6.14,6.51,6.05,6.42,5.82,-9.44,-0.92,5.02,5.69,5.55


In [ ]:
wide.reset_index().to_excel("data_111.xlsx", index=False, engine="openpyxl")

140 (PDRB Triwulanan Provinsi Bali Atas Dasar Harga Berlaku Menurut Lapangan Usaha)

In [99]:
data_140 = getDataByVarId(domain='5100', varId='140')
data_140.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Pertanian, Kehutanan, dan Perikanan",140,0,Tidak ada,125,2025,31,Triwulan I,1140012531,1140012531,10272952.15
1,1,"Pertanian, Kehutanan, dan Perikanan",140,0,Tidak ada,125,2025,32,Triwulan II,1140012532,1140012532,10525899.26
5,2,Pertambangan dan Penggalian,140,0,Tidak ada,125,2025,31,Triwulan I,2140012531,2140012531,633479.15
6,2,Pertambangan dan Penggalian,140,0,Tidak ada,125,2025,32,Triwulan II,2140012532,2140012532,660970.35
10,3,Industri Pengolahan,140,0,Tidak ada,125,2025,31,Triwulan I,3140012531,3140012531,4691809.70


In [100]:
# --- Ambil & bersihkan untuk urutan asli ---
data_140 = data_140[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_140['Kab/Kota'] = (
    data_140['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_140['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_140['Triwulan'] = data_140['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_140.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [101]:
wide

Tahun                                                      2016               \
Triwulan                                             Triwulan I  Triwulan II   
Kab/Kota                                                                       
Pertanian, Kehutanan, dan Perikanan                  6627950.23   6833915.39   
Pertambangan dan Penggalian                           509405.11    527510.18   
Industri Pengolahan                                  3025966.75   3088063.52   
Pengadaan Listrik dan Gas                              97515.19    104965.39   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...     86259.24     87017.82   
Konstruksi                                           4136148.60   4238878.28   
Perdagangan Besar dan Eceran; Reparasi Mobil da...   3873750.21   3959304.89   
Transportasi dan Pergudangan                         4286330.27   4503778.17   
Penyediaan Akomodasi dan Makan Minum                10557389.61  10967988.62   
Informasi dan Komunikasi                             2420603.21   2471772.23   
Jasa Keuangan dan Asuransi                           1912715.38   1953499.99   
Real Estat                                           1927918.99   1948159.51   
Jasa Perusahaan                                       481357.32    490774.29   
Administrasi Pemerintahan, Pertahanan dan Jamin...   2073645.83   2474615.98   
Jasa Pendidikan                                      2310537.02   2354526.44   
Jasa Kesehatan dan Kegiatan Sosial                    971940.00   1006269.00   
Jasa lainnya                                          709183.50    727948.43   
Jumlah                                              46008616.47  47738988.14   

Tahun                                                                         \
Triwulan                                           Triwulan III  Triwulan IV   
Kab/Kota                                                                       
Pertanian, Kehutanan, dan Perikanan                  7151537.81   7530381.82   
Pertambangan dan Penggalian                           528390.65    530524.42   
Industri Pengolahan                                  3103865.10   3193076.81   
Pengadaan Listrik dan Gas                             107683.83    109024.96   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...     91171.77     93662.93   
Konstruksi                                           4357335.28   4486201.33   
Perdagangan Besar dan Eceran; Reparasi Mobil da...   4088876.94   4174210.85   
Transportasi dan Pergudangan                         4987190.20   4788756.59   
Penyediaan Akomodasi dan Makan Minum                11485851.82  11373355.60   
Informasi dan Komunikasi                             2549318.66   2617862.43   
Jasa Keuangan dan Asuransi                           2071547.14   2115676.72   
Real Estat                                           1979534.40   2009846.10   
Jasa Perusahaan                                       511422.85    532595.24   
Administrasi Pemerintahan, Pertahanan dan Jamin...   2482990.50   2535590.12   
Jasa Pendidikan                                      2520557.94   2585077.51   
Jasa Kesehatan dan Kegiatan Sosial                   1040495.39   1070076.42   
Jasa lainnya                                          757242.64    781037.67   
Jumlah                                              49815012.93  50526957.53   

Tahun                                                                    2017  \
Triwulan                                                 Tahunan   Triwulan I   
Kab/Kota                                                                        
Pertanian, Kehutanan, dan Perikanan                 2.814379e+07   7115585.45   
Pertambangan dan Penggalian                         2.095830e+06    530384.20   
Industri Pengolahan                                 1.241097e+07   3224176.97   
Pengadaan Listrik dan Gas                           4.191894e+05    117685.06   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...  3.581118e+05     94

In [103]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_140.xlsx", index=False, engine="openpyxl")

141 (PDRB Triwulanan Provinsi Bali Atas Dasar Harga Konstan 2010 Menurut Lapangan Usaha)

In [104]:
data_141 = getDataByVarId(domain='5100', varId='141')
data_141.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Pertanian, Kehutanan, dan Perikanan",141,0,Tidak ada,125,2025,31,Triwulan I,1141012531,1141012531,5379974.62
1,1,"Pertanian, Kehutanan, dan Perikanan",141,0,Tidak ada,125,2025,32,Triwulan II,1141012532,1141012532,5456617.11
5,2,Pertambangan dan Penggalian,141,0,Tidak ada,125,2025,31,Triwulan I,2141012531,2141012531,356280.90
6,2,Pertambangan dan Penggalian,141,0,Tidak ada,125,2025,32,Triwulan II,2141012532,2141012532,369631.69
10,3,Industri Pengolahan,141,0,Tidak ada,125,2025,31,Triwulan I,3141012531,3141012531,2701507.12


In [105]:
# --- Ambil & bersihkan untuk urutan asli ---
data_141 = data_141[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_141['Kab/Kota'] = (
    data_141['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_141['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_141['Triwulan'] = data_141['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_141.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [106]:
wide

Tahun                                                      2016               \
Triwulan                                             Triwulan I  Triwulan II   
Kab/Kota                                                                       
Pertanian, Kehutanan, dan Perikanan                  4597423.20   4718813.94   
Pertambangan dan Penggalian                           369011.23    378866.97   
Industri Pengolahan                                  2236516.54   2275377.58   
Pengadaan Listrik dan Gas                              74809.93     77571.63   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...     75321.73     75698.55   
Konstruksi                                           3105609.15   3178660.66   
Perdagangan Besar dan Eceran; Reparasi Mobil da...   2987279.72   3036149.03   
Transportasi dan Pergudangan                         2412829.50   2516528.47   
Penyediaan Akomodasi dan Makan Minum                 6457798.32   6672937.16   
Informasi dan Komunikasi                             2256885.04   2299252.29   
Jasa Keuangan dan Asuransi                           1435635.75   1454774.98   
Real Estat                                           1603080.56   1614352.01   
Jasa Perusahaan                                       363164.26    367787.74   
Administrasi Pemerintahan, Pertahanan dan Jamin...   1845565.69   2163950.83   
Jasa Pendidikan                                      1784758.12   1803670.73   
Jasa Kesehatan dan Kegiatan Sosial                    756176.41    777254.41   
Jasa lainnya                                          522449.91    535143.61   
Jumlah                                              32884315.08  33946790.61   

Tahun                                                                         \
Triwulan                                           Triwulan III  Triwulan IV   
Kab/Kota                                                                       
Pertanian, Kehutanan, dan Perikanan                  4866405.46   5113053.99   
Pertambangan dan Penggalian                           376254.72    378135.99   
Industri Pengolahan                                  2275520.01   2317262.07   
Pengadaan Listrik dan Gas                              74263.26     74685.00   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...     76834.25     76789.45   
Konstruksi                                           3256898.25   3345195.25   
Perdagangan Besar dan Eceran; Reparasi Mobil da...   3108625.94   3158876.45   
Transportasi dan Pergudangan                         2654941.23   2598304.45   
Penyediaan Akomodasi dan Makan Minum                 6970296.52   6882816.19   
Informasi dan Komunikasi                             2368460.32   2424165.73   
Jasa Keuangan dan Asuransi                           1525311.11   1536473.80   
Real Estat                                           1633519.97   1644930.86   
Jasa Perusahaan                                       378789.79    392069.93   
Administrasi Pemerintahan, Pertahanan dan Jamin...   2160018.10   2189167.51   
Jasa Pendidikan                                      1922509.97   1951791.14   
Jasa Kesehatan dan Kegiatan Sosial                    800744.30    815838.80   
Jasa lainnya                                          551578.46    564811.29   
Jumlah                                              35000971.65  35464367.89   

Tahun                                                                    2017  \
Triwulan                                                 Tahunan   Triwulan I   
Kab/Kota                                                                        
Pertanian, Kehutanan, dan Perikanan                 1.929570e+07   4792405.73   
Pertambangan dan Penggalian                         1.502269e+06    378382.17   
Industri Pengolahan                                 9.104676e+06   2316553.57   
Pengadaan Listrik dan Gas                           3.013298e+05     75108.60   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...  3.046440e+05     76

In [ ]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_141.xlsx", index=False, engine="openpyxl")

143 (Pertumbuhan PDRB Triwulanan Provinsi Bali (y-o-y) Menurut Lapangan Usaha)

In [107]:
data_143 = getDataByVarId(domain='5100', varId='143')
data_143.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Pertanian, Kehutanan, dan Perikanan",143,0,Tidak ada,125,2025,31,Triwulan I,1143012531,1143012531,5.17
1,1,"Pertanian, Kehutanan, dan Perikanan",143,0,Tidak ada,125,2025,32,Triwulan II,1143012532,1143012532,-0.28
5,2,Pertambangan dan Penggalian,143,0,Tidak ada,125,2025,31,Triwulan I,2143012531,2143012531,0.81
6,2,Pertambangan dan Penggalian,143,0,Tidak ada,125,2025,32,Triwulan II,2143012532,2143012532,1.26
10,3,Industri Pengolahan,143,0,Tidak ada,125,2025,31,Triwulan I,3143012531,3143012531,-0.09


In [108]:
# --- Ambil & bersihkan untuk urutan asli ---
data_143 = data_143[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_143['Kab/Kota'] = (
    data_143['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_143['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_143['Triwulan'] = data_143['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_143.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [109]:
wide

Tahun                                                    2016              \
Triwulan                                           Triwulan I Triwulan II   
Kab/Kota                                                                    
Pertanian, Kehutanan, dan Perikanan                      2.68        0.02   
Pertambangan dan Penggalian                              5.01        6.31   
Industri Pengolahan                                      4.97        2.49   
Pengadaan Listrik dan Gas                               10.58       15.68   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...       9.64        6.82   
Konstruksi                                               7.62        8.07   
Perdagangan Besar dan Eceran; Reparasi Mobil da...       8.57        6.49   
Transportasi dan Pergudangan                             7.01        8.44   
Penyediaan Akomodasi dan Makan Minum                     5.15        6.82   
Informasi dan Komunikasi                                 7.65        8.80   
Jasa Keuangan dan Asuransi                               8.45        7.67   
Real Estat                                               5.92        5.53   
Jasa Perusahaan                                          8.94        6.39   
Administrasi Pemerintahan, Pertahanan dan Jamin...       5.48       15.19   
Jasa Pendidikan                                          9.50        9.37   
Jasa Kesehatan dan Kegiatan Sosial                       8.53        9.50   
Jasa lainnya                                             7.61        7.88   
Jumlah                                                   6.22        6.53   

Tahun                                                                        \
Triwulan                                           Triwulan III Triwulan IV   
Kab/Kota                                                                      
Pertanian, Kehutanan, dan Perikanan                        6.27        5.15   
Pertambangan dan Penggalian                                5.71        0.35   
Industri Pengolahan                                        3.43        2.64   
Pengadaan Listrik dan Gas                                  9.90       -1.63   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...         6.14        3.05   
Konstruksi                                                 6.98        6.43   
Perdagangan Besar dan Eceran; Reparasi Mobil da...         5.22        6.41   
Transportasi dan Pergudangan                               9.65        6.96   
Penyediaan Akomodasi dan Makan Minum                       6.94        7.50   
Informasi dan Komunikasi                                   8.47        9.19   
Jasa Keuangan dan Asuransi                                 7.05        9.08   
Real Estat                                                 5.42        2.63   
Jasa Perusahaan                                            6.27        5.96   
Administrasi Pemerintahan, Pertahanan dan Jamin...         2.31        0.04   
Jasa Pendidikan                                            8.72        8.13   
Jasa Kesehatan dan Kegiatan Sosial                         8.57        8.06   
Jasa lainnya                                               9.09       10.60   
Jumlah                                                     6.55        6.02   

Tahun                                                            2017  \
Triwulan                                           Tahunan Triwulan I   
Kab/Kota                                                                
Pertanian, Kehutanan, dan Perikanan                   3.53       4.24   
Pertambangan dan Penggalian                           4.28       2.54   
Industri Pengolahan                                   3.36       3.58   
Pengadaan Listrik dan Gas                             8.31       0.40   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...    6.34       1.82   
Konstruksi                                            7.26       6.27   
Perdagangan Besar dan Eceran; Reparasi Mobil da...    6.64       

In [ ]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_143.xlsx", index=False, engine="openpyxl")

155 (Pertumbuhan PDRB Triwulanan Provinsi Bali (c to c) Menurut Lapangan Usaha)

In [110]:
data_155 = getDataByVarId(domain='5100', varId='155')
data_155.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Pertanian, Kehutanan, dan Perikanan",155,0,Tidak ada,125,2025,31,Triwulan I,1155012531,1155012531,5.17
1,1,"Pertanian, Kehutanan, dan Perikanan",155,0,Tidak ada,125,2025,32,Triwulan II,1155012532,1155012532,2.36
5,2,Pertambangan dan Penggalian,155,0,Tidak ada,125,2025,31,Triwulan I,2155012531,2155012531,0.81
6,2,Pertambangan dan Penggalian,155,0,Tidak ada,125,2025,32,Triwulan II,2155012532,2155012532,1.04
10,3,Industri Pengolahan,155,0,Tidak ada,125,2025,31,Triwulan I,3155012531,3155012531,-0.09


In [111]:
# --- Ambil & bersihkan untuk urutan asli ---
data_155 = data_155[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_155['Kab/Kota'] = (
    data_140['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_155['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_155['Triwulan'] = data_155['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_155.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [112]:
wide

Tahun                                                    2016              \
Triwulan                                           Triwulan I Triwulan II   
Kab/Kota                                                                    
Pertanian, Kehutanan, dan Perikanan                      2.68        1.31   
Pertambangan dan Penggalian                              5.01        5.66   
Industri Pengolahan                                      4.97        3.71   
Pengadaan Listrik dan Gas                               10.58       13.12   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...       9.64        8.21   
Konstruksi                                               7.62        7.85   
Perdagangan Besar dan Eceran; Reparasi Mobil da...       8.57        7.51   
Transportasi dan Pergudangan                             7.01        7.74   
Penyediaan Akomodasi dan Makan Minum                     5.15        5.99   
Informasi dan Komunikasi                                 7.65        8.23   
Jasa Keuangan dan Asuransi                               8.45        8.06   
Real Estat                                               5.92        5.73   
Jasa Perusahaan                                          8.94        7.64   
Administrasi Pemerintahan, Pertahanan dan Jamin...       5.48       10.51   
Jasa Pendidikan                                          9.50        9.43   
Jasa Kesehatan dan Kegiatan Sosial                       8.53        9.02   
Jasa lainnya                                             7.61        7.75   
Jumlah                                                   6.22        6.38   

Tahun                                                                        \
Triwulan                                           Triwulan III Triwulan IV   
Kab/Kota                                                                      
Pertanian, Kehutanan, dan Perikanan                        2.96        3.53   
Pertambangan dan Penggalian                                5.68        4.28   
Industri Pengolahan                                        3.61        3.36   
Pengadaan Listrik dan Gas                                 12.04        8.31   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...         7.50        6.34   
Konstruksi                                                 7.55        7.26   
Perdagangan Besar dan Eceran; Reparasi Mobil da...         6.72        6.64   
Transportasi dan Pergudangan                               8.40        8.03   
Penyediaan Akomodasi dan Makan Minum                       6.32        6.62   
Informasi dan Komunikasi                                   8.31        8.54   
Jasa Keuangan dan Asuransi                                 7.71        8.06   
Real Estat                                                 5.63        4.85   
Jasa Perusahaan                                            7.17        6.85   
Administrasi Pemerintahan, Pertahanan dan Jamin...         7.49        5.44   
Jasa Pendidikan                                            9.18        8.91   
Jasa Kesehatan dan Kegiatan Sosial                         8.86        8.65   
Jasa lainnya                                               8.20        8.82   
Jumlah                                                     6.44        6.33   

Tahun                                                            2017  \
Triwulan                                           Tahunan Triwulan I   
Kab/Kota                                                                
Pertanian, Kehutanan, dan Perikanan                   3.53       4.24   
Pertambangan dan Penggalian                           4.28       2.54   
Industri Pengolahan                                   3.36       3.58   
Pengadaan Listrik dan Gas                             8.31       0.40   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...    6.34       1.82   
Konstruksi                                            7.26       6.27   
Perdagangan Besar dan Eceran; Reparasi Mobil da...    6.64       

In [ ]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_155.xlsx", index=False, engine="openpyxl")

156 (Pertumbuhan PDRB Triwulanan Provinsi Bali (q to q) Menurut Lapangan Usaha)

In [113]:
data_156 = getDataByVarId(domain='5100', varId='156')
data_156.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Pertanian, Kehutanan, dan Perikanan",156,0,Tidak ada,125,2025,31,Triwulan I,1156012531,1156012531,-4.72
1,1,"Pertanian, Kehutanan, dan Perikanan",156,0,Tidak ada,125,2025,32,Triwulan II,1156012532,1156012532,1.42
5,2,Pertambangan dan Penggalian,156,0,Tidak ada,125,2025,31,Triwulan I,2156012531,2156012531,-4.14
6,2,Pertambangan dan Penggalian,156,0,Tidak ada,125,2025,32,Triwulan II,2156012532,2156012532,3.75
10,3,Industri Pengolahan,156,0,Tidak ada,125,2025,31,Triwulan I,3156012531,3156012531,-2.90


In [115]:
# --- Ambil & bersihkan untuk urutan asli ---
data_156 = data_156[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_156['Kab/Kota'] = (
    data_156['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_156['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_156['Triwulan'] = data_156['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_156.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [116]:
wide

Tahun                                                    2016              \
Triwulan                                           Triwulan I Triwulan II   
Kab/Kota                                                                    
Pertanian, Kehutanan, dan Perikanan                     -5.45        2.64   
Pertambangan dan Penggalian                             -2.08        2.67   
Industri Pengolahan                                     -0.94        1.74   
Pengadaan Listrik dan Gas                               -1.46        3.69   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...       1.08        0.50   
Konstruksi                                              -1.20        2.35   
Perdagangan Besar dan Eceran; Reparasi Mobil da...       0.63        1.64   
Transportasi dan Pergudangan                            -0.67        4.30   
Penyediaan Akomodasi dan Makan Minum                     0.86        3.33   
Informasi dan Komunikasi                                 1.65        1.88   
Jasa Keuangan dan Asuransi                               1.92        1.33   
Real Estat                                               0.02        0.70   
Jasa Perusahaan                                         -1.86        1.27   
Administrasi Pemerintahan, Pertahanan dan Jamin...     -15.66       17.25   
Jasa Pendidikan                                         -1.12        1.06   
Jasa Kesehatan dan Kegiatan Sosial                       0.16        2.79   
Jasa lainnya                                             2.31        2.43   
Jumlah                                                  -1.70        3.23   

Tahun                                                                        \
Triwulan                                           Triwulan III Triwulan IV   
Kab/Kota                                                                      
Pertanian, Kehutanan, dan Perikanan                        3.13        5.07   
Pertambangan dan Penggalian                               -0.69        0.50   
Industri Pengolahan                                        0.01        1.83   
Pengadaan Listrik dan Gas                                 -4.26        0.57   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...         1.50       -0.06   
Konstruksi                                                 2.46        2.71   
Perdagangan Besar dan Eceran; Reparasi Mobil da...         2.39        1.62   
Transportasi dan Pergudangan                               5.50       -2.13   
Penyediaan Akomodasi dan Makan Minum                       4.46       -1.26   
Informasi dan Komunikasi                                   3.01        2.35   
Jasa Keuangan dan Asuransi                                 4.85        0.73   
Real Estat                                                 1.19        0.70   
Jasa Perusahaan                                            2.99        3.51   
Administrasi Pemerintahan, Pertahanan dan Jamin...        -0.18        1.35   
Jasa Pendidikan                                            6.59        1.52   
Jasa Kesehatan dan Kegiatan Sosial                         3.02        1.89   
Jasa lainnya                                               3.07        2.40   
Jumlah                                                     3.11        1.32   

Tahun                                                            2017  \
Triwulan                                           Tahunan Triwulan I   
Kab/Kota                                                                
Pertanian, Kehutanan, dan Perikanan                    NaN      -6.27   
Pertambangan dan Penggalian                            NaN       0.07   
Industri Pengolahan                                    NaN      -0.03   
Pengadaan Listrik dan Gas                              NaN       0.57   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...     NaN      -0.12   
Konstruksi                                             NaN      -1.34   
Perdagangan Besar dan Eceran; Reparasi Mobil da...     NaN       

In [ ]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_156.xlsx", index=False, engine="openpyxl")

161 (Indeks Implisit PDRB Triwulanan Provinsi Bali Series 2010 Menurut Lapangan Usaha)

In [117]:
data_161 = getDataByVarId(domain='5100', varId='161')
data_161.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Pertanian, Kehutanan, dan Perikanan",161,0,Tidak ada,125,2025,31,Triwulan I,1161012531,1161012531,190.95
1,1,"Pertanian, Kehutanan, dan Perikanan",161,0,Tidak ada,125,2025,32,Triwulan II,1161012532,1161012532,192.90
5,2,Pertambangan dan Penggalian,161,0,Tidak ada,125,2025,31,Triwulan I,2161012531,2161012531,177.80
6,2,Pertambangan dan Penggalian,161,0,Tidak ada,125,2025,32,Triwulan II,2161012532,2161012532,178.82
10,3,Industri Pengolahan,161,0,Tidak ada,125,2025,31,Triwulan I,3161012531,3161012531,173.67


In [118]:
# --- Ambil & bersihkan untuk urutan asli ---
data_161 = data_161[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_161['Kab/Kota'] = (
    data_161['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_161['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_161['Triwulan'] = data_161['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_161.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [119]:
wide

Tahun                                                    2016              \
Triwulan                                           Triwulan I Triwulan II   
Kab/Kota                                                                    
Pertanian, Kehutanan, dan Perikanan                    144.17      144.82   
Pertambangan dan Penggalian                            138.05      139.23   
Industri Pengolahan                                    135.30      135.72   
Pengadaan Listrik dan Gas                              130.35      135.31   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...     114.52      114.95   
Konstruksi                                             133.18      133.35   
Perdagangan Besar dan Eceran; Reparasi Mobil da...     129.67      130.41   
Transportasi dan Pergudangan                           177.65      178.97   
Penyediaan Akomodasi dan Makan Minum                   163.48      164.37   
Informasi dan Komunikasi                               107.25      107.50   
Jasa Keuangan dan Asuransi                             133.23      134.28   
Real Estat                                             120.26      120.68   
Jasa Perusahaan                                        132.55      133.44   
Administrasi Pemerintahan, Pertahanan dan Jamin...     112.36      114.36   
Jasa Pendidikan                                        129.46      130.54   
Jasa Kesehatan dan Kegiatan Sosial                     128.53      129.46   
Jasa lainnya                                           135.74      136.03   
Jumlah                                                 139.91      140.63   

Tahun                                                                        \
Triwulan                                           Triwulan III Triwulan IV   
Kab/Kota                                                                      
Pertanian, Kehutanan, dan Perikanan                      146.96      147.28   
Pertambangan dan Penggalian                              140.43      140.30   
Industri Pengolahan                                      136.40      137.80   
Pengadaan Listrik dan Gas                                145.00      145.98   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...       118.66      121.97   
Konstruksi                                               133.79      134.11   
Perdagangan Besar dan Eceran; Reparasi Mobil da...       131.53      132.14   
Transportasi dan Pergudangan                             187.85      184.30   
Penyediaan Akomodasi dan Makan Minum                     164.78      165.24   
Informasi dan Komunikasi                                 107.64      107.99   
Jasa Keuangan dan Asuransi                               135.81      137.70   
Real Estat                                               121.18      122.18   
Jasa Perusahaan                                          135.01      135.84   
Administrasi Pemerintahan, Pertahanan dan Jamin...       114.95      115.82   
Jasa Pendidikan                                          131.11      132.45   
Jasa Kesehatan dan Kegiatan Sosial                       129.94      131.16   
Jasa lainnya                                             137.29      138.28   
Jumlah                                                   142.32      142.47   

Tahun                                                            2017  \
Triwulan                                           Tahunan Triwulan I   
Kab/Kota                                                                
Pertanian, Kehutanan, dan Perikanan                 145.86     148.48   
Pertambangan dan Penggalian                         139.51     140.17   
Industri Pengolahan                                 136.31     139.18   
Pengadaan Listrik dan Gas                           139.11     156.69   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...  117.55     122.58   
Konstruksi                                          133.62     135.13   
Perdagangan Besar dan Eceran; Reparasi Mobil da...  130.96     13

In [ ]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_161.xlsx", index=False, engine="openpyxl")

165 (Distribusi PDRB Triwulanan Provinsi Bali Menurut Lapangan Usaha)

In [120]:
data_165 = getDataByVarId(domain='5100', varId='165')
data_165.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,"Pertanian, Kehutanan, dan Perikanan",165,0,Tidak ada,125,2025,31,Triwulan I,1165012531,1165012531,13.60
1,1,"Pertanian, Kehutanan, dan Perikanan",165,0,Tidak ada,125,2025,32,Triwulan II,1165012532,1165012532,12.93
5,2,Pertambangan dan Penggalian,165,0,Tidak ada,125,2025,31,Triwulan I,2165012531,2165012531,0.84
6,2,Pertambangan dan Penggalian,165,0,Tidak ada,125,2025,32,Triwulan II,2165012532,2165012532,0.81
10,3,Industri Pengolahan,165,0,Tidak ada,125,2025,31,Triwulan I,3165012531,3165012531,6.21


In [121]:
# --- Ambil & bersihkan untuk urutan asli ---
data_165 = data_165[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_165['Kab/Kota'] = (
    data_165['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_165['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_165['Triwulan'] = data_165['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_165.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [122]:
wide

Tahun                                                    2016              \
Triwulan                                           Triwulan I Triwulan II   
Kab/Kota                                                                    
Pertanian, Kehutanan, dan Perikanan                     14.41       14.32   
Pertambangan dan Penggalian                              1.11        1.10   
Industri Pengolahan                                      6.58        6.47   
Pengadaan Listrik dan Gas                                0.21        0.22   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...       0.19        0.18   
Konstruksi                                               8.99        8.88   
Perdagangan Besar dan Eceran; Reparasi Mobil da...       8.42        8.29   
Transportasi dan Pergudangan                             9.32        9.43   
Penyediaan Akomodasi dan Makan Minum                    22.95       22.97   
Informasi dan Komunikasi                                 5.26        5.18   
Jasa Keuangan dan Asuransi                               4.16        4.09   
Real Estat                                               4.19        4.08   
Jasa Perusahaan                                          1.05        1.03   
Administrasi Pemerintahan, Pertahanan dan Jamin...       4.51        5.18   
Jasa Pendidikan                                          5.02        4.93   
Jasa Kesehatan dan Kegiatan Sosial                       2.11        2.11   
Jasa lainnya                                             1.54        1.52   
Jumlah                                                 100.00      100.00   

Tahun                                                                        \
Triwulan                                           Triwulan III Triwulan IV   
Kab/Kota                                                                      
Pertanian, Kehutanan, dan Perikanan                       14.36       14.90   
Pertambangan dan Penggalian                                1.06        1.05   
Industri Pengolahan                                        6.23        6.32   
Pengadaan Listrik dan Gas                                  0.22        0.22   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...         0.18        0.19   
Konstruksi                                                 8.75        8.88   
Perdagangan Besar dan Eceran; Reparasi Mobil da...         8.21        8.26   
Transportasi dan Pergudangan                              10.01        9.48   
Penyediaan Akomodasi dan Makan Minum                      23.06       22.51   
Informasi dan Komunikasi                                   5.12        5.18   
Jasa Keuangan dan Asuransi                                 4.16        4.19   
Real Estat                                                 3.97        3.98   
Jasa Perusahaan                                            1.03        1.05   
Administrasi Pemerintahan, Pertahanan dan Jamin...         4.98        5.02   
Jasa Pendidikan                                            5.06        5.12   
Jasa Kesehatan dan Kegiatan Sosial                         2.09        2.12   
Jasa lainnya                                               1.52        1.55   
Jumlah                                                   100.00      100.00   

Tahun                                                            2017  \
Triwulan                                           Tahunan Triwulan I   
Kab/Kota                                                                
Pertanian, Kehutanan, dan Perikanan                  14.50      14.13   
Pertambangan dan Penggalian                           1.08       1.05   
Industri Pengolahan                                   6.39       6.40   
Pengadaan Listrik dan Gas                             0.22       0.23   
Pengadaan Air, Pengelolaan Sampah, Limbah dan D...    0.18       0.19   
Konstruksi                                            8.87       8.85   
Perdagangan Besar dan Eceran; Reparasi Mobil da...    8.29       

In [ ]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_165.xlsx", index=False, engine="openpyxl")

170 (PDRB Kabupaten/Kota di Provinsi Bali Atas Dasar Harga Berlaku)

In [127]:
data_170 = getDataByVarId(domain='5100', varId='170')
data_170.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,170,0,Tidak ada,124,2024,0,Tahun,117001240,117001240,16845.01
1,2,Kab. Tabanan,170,0,Tidak ada,124,2024,0,Tahun,217001240,217001240,27649.80
2,3,Kab. Badung,170,0,Tidak ada,124,2024,0,Tahun,317001240,317001240,75098.20
3,4,Kab. Gianyar,170,0,Tidak ada,124,2024,0,Tahun,417001240,417001240,33046.60
4,5,Kab. Klungkung,170,0,Tidak ada,124,2024,0,Tahun,517001240,517001240,11015.49


In [128]:
# --- ambil & bersihkan untuk urutan asli ---
data_170 = data_170[['vervar_label','tahun_label','content_label']].copy()
data_170['kabupaten_kota'] = (data_170['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_170['kabupaten_kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_170.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['kabupaten_kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='kabupaten_kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [129]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kabupaten_kota,,,,,,,,,,
Kab. Jembrana,10198.25,11167.67,12116.48,13136.61,14136.70,13437.61,13510.63,14532.26,15613.28,16845.01
Kab. Tabanan,16996.23,18630.25,20376.58,22127.88,23795.93,22257.58,22021.14,23681.30,25531.69,27649.80
Kab. Badung,42429.25,47208.17,52343.65,57791.25,62836.11,49026.03,44882.08,55290.00,68399.54,75098.20
Kab. Gianyar,20140.34,22113.25,24224.22,26460.40,28520.28,25841.72,25788.23,27944.21,30529.42,33046.60
Kab. Klungkung,6426.23,7112.02,7784.62,8459.34,9099.50,8450.67,8534.43,9210.21,10072.22,11015.49
Kab. Bangli,4945.79,5457.23,5976.57,6490.23,6993.64,6716.09,6825.56,7337.99,7908.31,8465.60
Kab. Karangasem,12233.23,13410.89,14598.38,15886.26,17086.88,16399.77,16506.62,17669.49,18974.78,20210.33
Kab. Buleleng,25170.26,27690.11,30318.76,32926.63,35362.32,33302.72,33363.29,35805.28,38351.36,41387.75
Kota Denpasar,38423.92,42384.43,46835.75,51374.78,55456.04,49607.46,49686.63,54633.83,60100.30,65300.21


In [130]:
wide.reset_index().to_excel("data_170.xlsx", index=False, engine="openpyxl")

171 (PDRB Kabupaten/Kota di Provinsi Bali Atas Dasar Harga Konstan 2010)

In [131]:
data_171 = getDataByVarId(domain='5100', varId='171')
data_171.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,171,0,Tidak ada,124,2024,0,Tahun,117101240,117101240,9965.23
1,2,Kab. Tabanan,171,0,Tidak ada,124,2024,0,Tahun,217101240,217101240,16236.00
2,3,Kab. Badung,171,0,Tidak ada,124,2024,0,Tahun,317101240,317101240,37667.34
3,4,Kab. Gianyar,171,0,Tidak ada,124,2024,0,Tahun,417101240,417101240,19896.35
4,5,Kab. Klungkung,171,0,Tidak ada,124,2024,0,Tahun,517101240,517101240,6348.54


In [132]:
# --- ambil & bersihkan untuk urutan asli ---
data_171 = data_171[['vervar_label','tahun_label','content_label']].copy()
data_171['kabupaten_kota'] = (data_171['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_171['kabupaten_kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_171.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['kabupaten_kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='kabupaten_kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [133]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kabupaten_kota,,,,,,,,,,
Kab. Jembrana,7576.31,8027.93,8452.03,8924.38,9420.44,8951.35,8892.80,9157.67,9492.38,9965.23
Kab. Tabanan,12644.52,13420.55,14141.72,14949.32,15783.04,14808.65,14515.58,14940.44,15472.12,16236.00
Kab. Badung,29170.24,31157.37,33052.05,35275.42,37326.47,31147.82,29049.52,31946.05,35554.32,37667.34
Kab. Gianyar,15168.55,16125.28,17005.12,18027.09,19040.90,17442.44,17258.77,17956.69,18865.02,19896.35
Kab. Klungkung,4813.39,5115.61,5387.61,5682.94,5990.94,5608.69,5595.88,5770.48,6041.65,6348.54
Kab. Bangli,3686.10,3916.10,4124.22,4350.14,4587.22,4399.19,4384.73,4507.38,4665.10,4876.68
Kab. Karangasem,8991.75,9524.23,10006.13,10550.25,11130.34,10630.37,10570.60,10843.41,11179.22,11663.31
Kab. Buleleng,18818.62,19950.72,21023.60,22201.45,23425.32,22066.16,21785.73,22463.38,23281.43,24453.82
Kota Denpasar,28422.70,30273.39,32105.35,34166.04,36154.42,32740.22,32439.61,34066.71,36004.30,38001.22


In [134]:
wide.reset_index().to_excel("data_171.xlsx", index=False, engine="openpyxl")

172 (PDRB Perkapita Atas Dasar harga Berlaku Kabupaten/Kota di Provinsi Bali)

In [135]:
data_172 = getDataByVarId(domain='5100', varId='172')
data_172.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,172,0,Tidak ada,124,2024,0,Tahun,117201240,117201240,51730.04
1,2,Kab. Tabanan,172,0,Tidak ada,124,2024,0,Tahun,217201240,217201240,59122.83
2,3,Kab. Badung,172,0,Tidak ada,124,2024,0,Tahun,317201240,317201240,132087.94
3,4,Kab. Gianyar,172,0,Tidak ada,124,2024,0,Tahun,417201240,417201240,62692.86
4,5,Kab. Klungkung,172,0,Tidak ada,124,2024,0,Tahun,517201240,517201240,52627.86


In [136]:
# --- ambil & bersihkan untuk urutan asli ---
data_172 = data_172[['vervar_label','tahun_label','content_label']].copy()
data_172['kabupaten_kota'] = (data_172['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_172['kabupaten_kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_172.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['kabupaten_kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='kabupaten_kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [137]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kabupaten_kota,,,,,,,,,,
Kab. Jembrana,37548.79,40862.31,44070.03,47495.22,50722.07,42516.29,41967.00,45248.71,48270.17,51730.04
Kab. Tabanan,38991.12,42486.32,46201.84,49892.63,53275.51,48248.65,47324.00,50978.06,54773.65,59122.83
Kab. Badung,68833.95,74933.60,81345.41,87972.91,92561.50,89312.31,81715.00,99075.72,121419.38,132087.94
Kab. Gianyar,40679.34,44261.91,48073.46,52076.32,55452.83,50179.76,49642.00,53644.67,58258.58,62692.86
Kab. Klungkung,36575.03,40249.11,43885.69,47453.19,50795.49,40970.58,40617.00,44269.42,48259.84,52627.86
Kab. Bangli,22218.27,24384.40,26551.46,28697.41,30722.11,26035.78,25999.00,28181.53,30255.87,32274.88
Kab. Karangasem,29932.05,32645.79,35362.42,38298.59,40971.60,33432.29,32957.00,35518.56,37953.27,40232.93
Kab. Buleleng,38951.20,42593.62,46387.04,50102.90,53429.02,42241.48,41361.00,44603.34,47413.39,50795.67
Kota Denpasar,43633.80,47235.52,51226.98,55204.95,57897.95,68298.89,68383.00,73731.75,80305.37,86425.33


In [138]:
wide.reset_index().to_excel("data_172.xlsx", index=False, engine="openpyxl")

363 (PDRB Tahunan Provinsi Bali Atas Dasar Harga Berlaku Menurut Lapangan Usaha)

In [139]:
data_363 = getDataByVarId(domain='5100', varId='363')
data_363.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1000,"<b>A Pertanian, Kehutanan, dan Perikanan </b>",363,0,Tidak ada,124,2024,0,Tahun,100036301240,100036301240,40059.24
1,1100,"1 Pertanian, Peternakan, Perburuan dan Jasa P...",363,0,Tidak ada,124,2024,0,Tahun,110036301240,110036301240,27303.95
2,1110,a. Tanaman Pangan,363,0,Tidak ada,124,2024,0,Tahun,111036301240,111036301240,4240.07
3,1120,b. Tanaman Hortikultura,363,0,Tidak ada,124,2024,0,Tahun,112036301240,112036301240,4875.68
4,1130,c. Perkebunan,363,0,Tidak ada,124,2024,0,Tahun,113036301240,113036301240,4001.07


In [140]:
# --- ambil & bersihkan untuk urutan asli ---
data_363 = data_363[['vervar_label','tahun_label','content_label']].copy()
data_363['PDRB Lapangan Usaha (Seri 2010)'] = (data_363['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_363['PDRB Lapangan Usaha (Seri 2010)'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_363.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['PDRB Lapangan Usaha (Seri 2010)','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='PDRB Lapangan Usaha (Seri 2010)', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [141]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
PDRB Lapangan Usaha (Seri 2010),,,,,,,,,,
"A Pertanian, Kehutanan, dan Perikanan",25843.65,28143.79,29782.34,32126.08,33881.27,33832.18,34766.74,36013.89,37709.95,40059.24
"1 Pertanian, Peternakan, Perburuan dan Jasa Pertanian",18494.65,19968.50,20862.15,22042.55,23240.59,23260.30,23547.31,24376.64,25582.23,27303.95
a. Tanaman Pangan,3878.69,4081.73,3854.97,4049.64,3857.15,3744.01,3800.86,4055.12,4230.71,4240.07
b. Tanaman Hortikultura,3797.34,4119.51,4200.79,4329.64,4561.40,4628.99,4665.20,4422.23,4623.44,4875.68
c. Perkebunan,2621.37,2887.93,3015.19,3132.57,3380.99,3441.29,3480.86,3616.22,3734.89,4001.07
...,...,...,...,...,...,...,...,...,...,...
"O Administrasi Pemerintahan, Pertahanan dan Jaminan Sosial Wajib",8738.31,9566.84,10493.06,11554.48,12409.41,13201.73,13689.63,13646.46,13888.80,14933.54
P Jasa Pendidikan,8555.07,9770.70,10815.80,11996.64,12974.87,13186.47,13453.38,13614.50,13669.13,14228.76
Q Jasa Kesehatan dan Kegiatan Sosial,3616.71,4088.78,4561.05,5076.07,5481.44,5780.17,6238.29,6485.86,6805.15,7066.96


In [142]:
wide.reset_index().to_excel("data_363.xlsx", index=False, engine="openpyxl")

365 (PDRB Tahunan Provinsi Bali Atas Dasar Harga Konstan 2010 Menurut Lapangan Usaha)

In [143]:
data_365 = getDataByVarId(domain='5100', varId='365')
data_365.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1000,"<b>A Pertanian, Kehutanan, dan Perikanan </b>",365,0,Tidak ada,124,2024,0,Tahun,100036501240,100036501240,21579.83
1,1100,"1 Pertanian, Peternakan, Perburuan dan Jasa P...",365,0,Tidak ada,124,2024,0,Tahun,110036501240,110036501240,14556.08
2,1110,a. Tanaman Pangan,365,0,Tidak ada,124,2024,0,Tahun,111036501240,111036501240,2419.20
3,1120,b. Tanaman Hortikultura,365,0,Tidak ada,124,2024,0,Tahun,112036501240,112036501240,2409.47
4,1130,c. Perkebunan,365,0,Tidak ada,124,2024,0,Tahun,113036501240,113036501240,2261.61


In [144]:
# --- ambil & bersihkan untuk urutan asli ---
data_365 = data_365[['vervar_label','tahun_label','content_label']].copy()
data_365['PDRB Lapangan Usaha (Seri 2010)'] = (data_365['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_365['PDRB Lapangan Usaha (Seri 2010)'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_365.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['PDRB Lapangan Usaha (Seri 2010)','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='PDRB Lapangan Usaha (Seri 2010)', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [145]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
PDRB Lapangan Usaha (Seri 2010),,,,,,,,,,
"A Pertanian, Kehutanan, dan Perikanan",18637.35,19295.70,19821.71,20760.22,21479.55,21269.19,21338.45,21157.30,21039.32,21579.83
"1 Pertanian, Peternakan, Perburuan dan Jasa Pertanian",13248.85,13620.82,13894.29,14326.30,14849.05,14785.36,14573.79,14400.13,14265.98,14556.08
a. Tanaman Pangan,2885.87,2964.59,2738.90,2777.57,2649.72,2571.76,2596.33,2657.63,2542.88,2419.20
b. Tanaman Hortikultura,2482.99,2489.53,2476.99,2481.42,2551.30,2568.36,2563.74,2345.50,2347.21,2409.47
c. Perkebunan,1883.66,1958.91,2023.17,2099.69,2273.71,2313.71,2275.10,2273.65,2251.19,2261.61
...,...,...,...,...,...,...,...,...,...,...
"O Administrasi Pemerintahan, Pertahanan dan Jaminan Sosial Wajib",7927.62,8358.70,8224.62,8569.58,8955.58,8921.60,8973.81,8632.77,8539.73,9157.52
P Jasa Pendidikan,6852.51,7462.73,7985.13,8574.40,8991.01,8916.71,8987.39,8989.23,8976.16,9091.78
Q Jasa Kesehatan dan Kegiatan Sosial,2899.10,3150.01,3415.75,3709.33,3930.79,4042.26,4269.26,4314.73,4410.79,4506.06


In [146]:
wide.reset_index().to_excel("data_365.xlsx", index=False, engine="openpyxl")

368 (Distribusi PDRB Tahunan Provinsi Bali Atas Dasar Harga Berlaku Menurut Lapangan Usaha)

In [147]:
data_368 = getDataByVarId(domain='5100', varId='368')
data_368

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1000,"<b>A Pertanian, Kehutanan, dan Perikanan </b>",368,0,Tidak ada,124,2024,0,Tahun,100036801240,100036801240,13.42
1,1100,"1 Pertanian, Peternakan, Perburuan dan Jasa P...",368,0,Tidak ada,124,2024,0,Tahun,110036801240,110036801240,9.15
2,1110,a. Tanaman Pangan,368,0,Tidak ada,124,2024,0,Tahun,111036801240,111036801240,1.42
3,1120,b. Tanaman Hortikultura,368,0,Tidak ada,124,2024,0,Tahun,112036801240,112036801240,1.63
4,1130,c. Perkebunan,368,0,Tidak ada,124,2024,0,Tahun,113036801240,113036801240,1.34
...,...,...,...,...,...,...,...,...,...,...,...,...
57,50000,"<b>O Administrasi Pemerintahan, Pertahanan dan...",368,0,Tidak ada,115,2015,0,Tahun,5000036801150,5000036801150,4.95
58,60000,<b>P Jasa Pendidikan </b>,368,0,Tidak ada,115,2015,0,Tahun,6000036801150,6000036801150,4.85
59,70000,<b>Q Jasa Kesehatan dan Kegiatan Sosial </b>,368,0,Tidak ada,115,2015,0,Tahun,7000036801150,7000036801150,2.05
60,80000,"<b>R,S,T,U Jasa lainnya </b>",368,0,Tidak ada,115,2015,0,Tahun,8000036801150,8000036801150,1.49


In [ ]:
# 1) ambil & beresin dari API
df = data_368[['verval_val','vervar_label','tahun_label','content_label']].copy()
df['uraian'] = (df['vervar_label'].astype(str)
                .str.replace(r'<.*?>','', regex=True)
                .str.replace(r'\s+',' ', regex=True)
                .str.strip())

# urutan baris sesuai verval_val
order = (df[['uraian','verval_val']]
         .drop_duplicates()
         .sort_values('verval_val'))['uraian'].tolist()

# tahun -> int
df['tahun'] = pd.to_numeric(df['tahun_label'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# 2) parse persen apa adanya (tanpa ngubah skala)
def parse_percent(s):
    s = re.sub(r'[^\d.,-]', '', str(s))
    if ',' in s and '.' in s:              # "1.234,56" -> 1234.56
        s = s.replace('.', '').replace(',', '.')
    elif ',' in s:                         # "13,42" -> 13.42
        s = s.replace(',', '.')
    return float(s) if s not in ('', '-', None) else np.nan

df['persen'] = df['content_label'].map(parse_percent)

# 3) pivot apa adanya (tanpa hitung ulang)
wide = (df.pivot_table(index='uraian',
                       columns='tahun',
                       values='persen',
                       aggfunc='first',      # pakai nilai API langsung
                       sort=False)
          .reindex(sorted(df['tahun'].unique()), axis=1))

# urut baris sesuai verval_val
wide = wide.reindex([u for u in order if u in wide.index])

# pastikan benar2 float (biar Excel gak jadi "015", "010", dst)
for c in wide.columns:
    wide[c] = pd.to_numeric(wide[c], errors='coerce')


In [149]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
uraian,,,,,,,,,,
"A Pertanian, Kehutanan, dan Perikanan",14.65,14.50,13.98,13.75,13.45,15.09,15.77,14.68,13.74,13.42
"1 Pertanian, Peternakan, Perburuan dan Jasa Pertanian",10.48,10.29,9.79,9.43,9.22,10.37,10.68,9.93,9.32,9.15
a. Tanaman Pangan,2.20,2.10,1.81,1.73,1.53,1.67,1.72,1.65,1.54,1.42
b. Tanaman Hortikultura,2.15,2.12,1.97,1.85,1.81,2.06,2.12,1.80,1.69,1.63
c. Perkebunan,1.49,1.49,1.42,1.34,1.34,1.53,1.58,1.47,1.36,1.34
...,...,...,...,...,...,...,...,...,...,...
"O Administrasi Pemerintahan, Pertahanan dan Jaminan Sosial Wajib",4.95,4.93,4.93,4.95,4.93,5.89,6.21,5.56,5.06,5.00
P Jasa Pendidikan,4.85,5.03,5.08,5.13,5.15,5.88,6.10,5.55,4.98,4.77
Q Jasa Kesehatan dan Kegiatan Sosial,2.05,2.11,2.14,2.17,2.18,2.58,2.83,2.64,2.48,2.37


In [150]:
wide.reset_index().to_excel("data_368.xlsx", index=False, engine="openpyxl")

369 (Pertumbuhan PDRB/Ekonomi Tahunan Provinsi Bali Menurut Lapangan Usaha)

In [125]:
data_369 = getDataByVarId(domain='5100', varId='369')
data_369.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1000,"<b>A Pertanian, Kehutanan, dan Perikanan </b>",369,0,Tidak ada,124,2024,0,Tahun,100036901240,100036901240,2.57
1,1100,"1 Pertanian, Peternakan, Perburuan dan Jasa P...",369,0,Tidak ada,124,2024,0,Tahun,110036901240,110036901240,2.03
2,1110,a. Tanaman Pangan,369,0,Tidak ada,124,2024,0,Tahun,111036901240,111036901240,-4.86
3,1120,b. Tanaman Hortikultura,369,0,Tidak ada,124,2024,0,Tahun,112036901240,112036901240,2.65
4,1130,c. Perkebunan,369,0,Tidak ada,124,2024,0,Tahun,113036901240,113036901240,0.46


In [126]:
# --- ambil & bersihkan untuk urutan asli ---
data_369 = data_369[['vervar_label','tahun_label','content_label']].copy()
data_369['PDRB Lapangan Usaha (Seri 2010)'] = (data_369['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_369['PDRB Lapangan Usaha (Seri 2010)'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_369.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['PDRB Lapangan Usaha (Seri 2010)','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='PDRB Lapangan Usaha (Seri 2010)', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [127]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
PDRB Lapangan Usaha (Seri 2010),,,,,,,,,,
"A Pertanian, Kehutanan, dan Perikanan",2.68,3.53,2.73,4.73,3.46,-0.98,0.33,-0.85,-0.56,2.57
"1 Pertanian, Peternakan, Perburuan dan Jasa Pertanian",3.45,2.81,2.01,3.11,3.65,-0.43,-1.43,-1.19,-0.93,2.03
a. Tanaman Pangan,-2.47,2.73,-7.61,1.41,-4.60,-2.94,0.96,2.36,-4.32,-4.86
b. Tanaman Hortikultura,10.95,0.26,-0.50,0.18,2.82,0.67,-0.18,-8.51,0.07,2.65
c. Perkebunan,11.24,4.00,3.28,3.78,8.29,1.76,-1.67,-0.06,-0.99,0.46
d. Peternakan,1.08,3.31,7.46,4.93,5.95,-0.54,-2.68,-0.11,0.02,4.94
e. Jasa Pertanian dan Perburuan,7.53,8.18,2.01,-0.60,3.15,-1.64,-1.40,-2.66,-0.06,1.07
2 Kehutanan dan Penebangan Kayu,9.00,9.09,6.56,1.79,-6.20,-12.36,0.37,0.53,-1.49,-2.25
3 Perikanan,0.82,5.31,4.45,8.55,3.07,-2.20,4.34,-0.11,0.24,3.70


In [154]:
wide.reset_index().to_excel("data_369.xlsx", index=False, engine="openpyxl")

370 (Indeks Implisit PDRB Tahunan Provinsi Bali Menurut Lapangan Usaha)

In [155]:
data_370 = getDataByVarId(domain='5100', varId='370')
data_370.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1000,"<b>A Pertanian, Kehutanan, dan Perikanan </b>",370,0,Tidak ada,124,2024,0,Tahun,100037001240,100037001240,185.63
1,1100,"1 Pertanian, Peternakan, Perburuan dan Jasa P...",370,0,Tidak ada,124,2024,0,Tahun,110037001240,110037001240,187.58
2,1110,a. Tanaman Pangan,370,0,Tidak ada,124,2024,0,Tahun,111037001240,111037001240,175.27
3,1120,b. Tanaman Hortikultura,370,0,Tidak ada,124,2024,0,Tahun,112037001240,112037001240,202.35
4,1130,c. Perkebunan,370,0,Tidak ada,124,2024,0,Tahun,113037001240,113037001240,176.91


In [156]:
# --- ambil & bersihkan untuk urutan asli ---
data_370 = data_370[['vervar_label','tahun_label','content_label']].copy()
data_370['PDRB Lapangan Usaha (Seri 2010)'] = (data_370['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_370['PDRB Lapangan Usaha (Seri 2010)'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_370.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['PDRB Lapangan Usaha (Seri 2010)','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='PDRB Lapangan Usaha (Seri 2010)', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [157]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
PDRB Lapangan Usaha (Seri 2010),,,,,,,,,,
"A Pertanian, Kehutanan, dan Perikanan",138.67,145.86,150.25,154.75,157.74,159.07,162.93,170.22,179.24,185.63
"1 Pertanian, Peternakan, Perburuan dan Jasa Pertanian",139.59,146.60,150.15,153.86,156.51,157.32,161.57,169.28,179.32,187.58
a. Tanaman Pangan,134.40,137.68,140.75,145.80,145.57,145.58,146.39,152.58,166.38,175.27
b. Tanaman Hortikultura,152.93,165.47,169.59,174.48,178.79,180.23,181.97,188.54,196.98,202.35
c. Perkebunan,139.16,147.43,149.03,149.19,148.70,148.73,153.00,159.05,165.91,176.91
d. Peternakan,136.14,142.16,146.23,178.01,154.36,155.32,161.88,171.95,182.07,189.94
e. Jasa Pertanian dan Perburuan,149.17,161.31,166.79,149.61,174.06,175.61,177.43,184.29,189.58,192.38
2 Kehutanan dan Penebangan Kayu,122.82,127.11,129.15,132.83,138.01,140.23,141.75,146.35,150.68,154.04
3 Perikanan,136.40,144.08,150.52,156.75,160.51,163.07,165.88,172.25,179.08,181.63


In [158]:
wide.reset_index().to_excel("data_370.xlsx", index=False, engine="openpyxl")

373 (Laju Implisit PDRB Tahunan Provinsi Bali Menurut Lapangan Usaha)

In [159]:
data_373 = getDataByVarId(domain='5100', varId='373')
data_373.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1000,"<b>A Pertanian, Kehutanan, dan Perikanan </b>",373,0,Tidak ada,124,2024,0,Tahun,100037301240,100037301240,3.57
1,1100,"1 Pertanian, Peternakan, Perburuan dan Jasa P...",373,0,Tidak ada,124,2024,0,Tahun,110037301240,110037301240,4.60
2,1110,a. Tanaman Pangan,373,0,Tidak ada,124,2024,0,Tahun,111037301240,111037301240,5.34
3,1120,b. Tanaman Hortikultura,373,0,Tidak ada,124,2024,0,Tahun,112037301240,112037301240,2.73
4,1130,c. Perkebunan,373,0,Tidak ada,124,2024,0,Tahun,113037301240,113037301240,6.63


In [160]:
# --- ambil & bersihkan untuk urutan asli ---
data_373 = data_373[['vervar_label','tahun_label','content_label']].copy()
data_373['PDRB Lapangan Usaha (Seri 2010)'] = (data_373['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_373['PDRB Lapangan Usaha (Seri 2010)'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_373.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['PDRB Lapangan Usaha (Seri 2010)','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='PDRB Lapangan Usaha (Seri 2010)', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [161]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
PDRB Lapangan Usaha (Seri 2010),,,,,,,,,,
"A Pertanian, Kehutanan, dan Perikanan",9.85,5.18,3.01,2.99,1.93,1.93,2.43,4.47,5.30,3.57
"1 Pertanian, Peternakan, Perburuan dan Jasa Pertanian",10.17,5.02,2.42,2.47,1.72,1.72,2.70,4.77,5.93,4.60
a. Tanaman Pangan,12.92,2.44,2.23,3.59,-0.16,-0.16,0.56,4.23,9.04,5.34
b. Tanaman Hortikultura,15.03,8.20,2.49,1.59,2.47,2.47,0.96,3.61,4.47,2.73
c. Perkebunan,7.18,5.94,1.09,2.88,-0.33,-0.33,2.87,3.96,4.31,6.63
d. Peternakan,7.11,4.43,2.86,0.11,2.72,2.72,4.22,6.22,5.89,4.32
e. Jasa Pertanian dan Perburuan,13.41,8.14,3.40,0.04,1.54,1.54,1.04,3.87,2.87,1.48
2 Kehutanan dan Penebangan Kayu,6.93,3.50,1.60,2.85,3.90,3.90,1.09,3.24,2.96,2.23
3 Perikanan,9.03,5.63,4.47,4.14,2.39,2.39,1.72,3.84,3.97,1.42


In [162]:
wide.reset_index().to_excel("data_373.xlsx", index=False, engine="openpyxl")

415 (PDRB Triwulanan Menurut Kabupaten/Kota Atas Dasar Harga Berlaku Provinsi Bali)

In [172]:
data_415 = getDataByVarId(domain='5100', varId='415')
data_415.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kabupaten Jembrana,415,0,Tidak ada,125,2025,31,Triwulan I,1415012531,1415012531,4297.22
5,2,Kabupaten Tabanan,415,0,Tidak ada,125,2025,31,Triwulan I,2415012531,2415012531,7019.67
10,3,Kabupaten Badung,415,0,Tidak ada,125,2025,31,Triwulan I,3415012531,3415012531,18754.84
15,4,Kabupaten Gianyar,415,0,Tidak ada,125,2025,31,Triwulan I,4415012531,4415012531,8530.39
20,5,Kabupaten Klungkung,415,0,Tidak ada,125,2025,31,Triwulan I,5415012531,5415012531,2833.17


In [ ]:
# --- Ambil & bersihkan untuk urutan asli ---
data_415 = data_415[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_415['Kab/Kota'] = (
    data_415['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_415['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_415['Triwulan'] = data_415['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_415.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [176]:
wide

Tahun                      2022                                       \
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV   
Kab/Kota                                                               
Kabupaten Jembrana      3407.48     3626.82      3719.32     3778.63   
Kabupaten Tabanan       5469.21     5967.17      6023.72     6221.20   
Kabupaten Badung       11629.49    13480.71     14587.22    15592.58   
Kabupaten Gianyar       6509.27     7014.76      7069.77     7350.40   
Kabupaten Klungkung     2147.25     2320.28      2340.85     2401.82   
Kabupaten Bangli        1693.21     1850.70      1864.11     1929.97   
Kabupaten Karangasem    4085.95     4460.88      4502.78     4619.88   
Kabupaten Buleleng      8268.65     8977.93      9162.22     9396.47   
Kota Denpasar          12476.99    13603.75     13978.59    14574.51   

Tahun                                2023                           \
Triwulan               Tahunan Triwulan I Triwulan II Triwulan III   
Kab/Kota                                                             
Kabupaten Jembrana    14532.26    3661.29     3874.95      3980.32   
Kabupaten Tabanan     23681.30    6019.14     6394.27      6437.08   
Kabupaten Badung      55290.00   15760.77    16896.12     17803.34   
Kabupaten Gianyar     27944.21    7105.20     7577.53      7734.06   
Kabupaten Klungkung    9210.21    2349.21     2505.17      2533.71   
Kabupaten Bangli       7337.99    1859.39     1995.51      1979.89   
Kabupaten Karangasem  17669.49    4421.71     4789.32      4817.38   
Kabupaten Buleleng    35805.28    8922.38     9605.30      9713.45   
Kota Denpasar         54633.83   14008.47    15166.55     15206.13   

Tahun                                            2024              \
Triwulan             Triwulan IV   Tahunan Triwulan I Triwulan II   
Kab/Kota                                                            
Kabupaten Jembrana       4096.71  15613.28    3951.10     4173.20   
Kabupaten Tabanan        6681.21  25531.69    6468.29     6970.19   
Kabupaten Badung        17939.31  68399.54   17412.16    18811.95   
Kabupaten Gianyar        8112.63  30529.42    7808.97     8135.42   
Kabupaten Klungkung      2684.12  10072.22    2603.67     2740.84   
Kabupaten Bangli         2073.51   7908.31    1998.14     2134.92   
Kabupaten Karangasem     4946.36  18974.78    4774.61     5076.65   
Kabupaten Buleleng      10110.24  38351.36    9648.08    10318.87   
Kota Denpasar           15719.15  60100.30   15226.77    16457.94   

Tahun                                                         2025  \
Triwulan             Triwulan III Triwulan IV   Tahunan Triwulan I   
Kab/Kota                                                             
Kabupaten Jembrana        4248.32     4472.39  16845.01    4297.22   
Kabupaten Tabanan         6971.72     7239.60  27649.80    7019.67   
Kabupaten Badung         19319.94    19554.15  75098.20   18754.84   
Kabupaten Gianyar         8250.76     8851.45  33046.60    8530.39   
Kabupaten Klungkung       2775.80     2895.17  11015.49    2833.17   
Kabupaten Bangli          2128.66     2203.88   8465.60    2158.57   
Kabupaten Karangasem      5084.46     5274.60  20210.33    5162.54   
Kabupaten Buleleng       10511.40    10909.41  41387.75   10540.40   
Kota Denpasar            16588.05    17027.45  65300.21   16557.16   

Tahun                                                              
Triwulan             Triwulan II Triwulan III Triwulan IV Tahunan  
Kab/Kota                                                           
Kabupaten Jembrana           NaN          NaN         NaN     NaN  
Kabupaten Tabanan            NaN          NaN         NaN     NaN  
Kabupaten Badung             NaN          NaN         NaN     NaN  
Kabupaten Gianyar            NaN          NaN         NaN     NaN  
Kabupaten Klungkung          NaN          NaN         NaN     NaN  
Kabupaten Bangli             NaN          NaN         NaN     NaN  
Kabupaten Ka

In [177]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_415.xlsx", index=False, engine="openpyxl")

416 (PDRB Triwulanan Menurut Kabupaten/Kota Atas Dasar Harga Konstan Provinsi Bali)

In [179]:
data_416 = getDataByVarId(domain='5100', varId='416')
data_416.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kabupaten Jembrana,416,0,Tidak ada,125,2025,31,Triwulan I,1416012531,1416012531,2494.09
5,2,Kabupaten Tabanan,416,0,Tidak ada,125,2025,31,Triwulan I,2416012531,2416012531,4050.17
10,3,Kabupaten Badung,416,0,Tidak ada,125,2025,31,Triwulan I,3416012531,3416012531,9325.94
15,4,Kabupaten Gianyar,416,0,Tidak ada,125,2025,31,Triwulan I,4416012531,4416012531,5024.54
20,5,Kabupaten Klungkung,416,0,Tidak ada,125,2025,31,Triwulan I,5416012531,5416012531,1601.12


In [180]:
# --- Ambil & bersihkan untuk urutan asli ---
data_416 = data_416[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_416['Kab/Kota'] = (
    data_416['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_416['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_416['Triwulan'] = data_416['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_416.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [181]:
wide

Tahun                      2022                                       \
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV   
Kab/Kota                                                               
Kabupaten Jembrana      2194.99     2299.16      2321.49     2342.04   
Kabupaten Tabanan       3523.96     3790.65      3764.08     3861.75   
Kabupaten Badung        7277.26     7968.96      8156.44     8543.39   
Kabupaten Gianyar       4268.68     4539.55      4501.32     4647.14   
Kabupaten Klungkung     1375.78     1463.12      1452.10     1479.48   
Kabupaten Bangli        1060.86     1144.46      1132.08     1169.99   
Kabupaten Karangasem    2559.66     2758.33      2736.91     2788.51   
Kabupaten Buleleng      5290.68     5665.43      5696.36     5810.92   
Kota Denpasar           7972.15     8546.97      8629.99     8917.60   

Tahun                                2023                           \
Triwulan               Tahunan Triwulan I Triwulan II Triwulan III   
Kab/Kota                                                             
Kabupaten Jembrana     9157.67    2245.06     2361.26      2413.45   
Kabupaten Tabanan     14940.44    3686.13     3885.50      3888.10   
Kabupaten Badung      31946.05    8379.90     8850.17      9086.79   
Kabupaten Gianyar     17956.69    4435.17     4704.15      4757.66   
Kabupaten Klungkung    5770.48    1429.40     1514.09      1518.19   
Kabupaten Bangli       4507.38    1107.97     1180.64      1162.42   
Kabupaten Karangasem  10843.41    2629.68     2828.35      2824.97   
Kabupaten Buleleng    22463.38    5465.72     5846.96      5876.06   
Kota Denpasar         34066.71    8451.62     9106.00      9082.64   

Tahun                                            2024              \
Triwulan             Triwulan IV   Tahunan Triwulan I Triwulan II   
Kab/Kota                                                            
Kabupaten Jembrana       2472.61   9492.38    2367.73     2475.37   
Kabupaten Tabanan        4012.39  15472.12    3849.25     4104.64   
Kabupaten Badung         9237.47  35554.32    8838.46     9449.56   
Kabupaten Gianyar        4968.04  18865.02    4755.63     4923.43   
Kabupaten Klungkung      1579.97   6041.65    1520.59     1586.00   
Kabupaten Bangli         1214.07   4665.10    1160.97     1232.60   
Kabupaten Karangasem     2896.23  11179.22    2781.14     2932.59   
Kabupaten Buleleng       6092.68  23281.43    5768.42     6117.43   
Kota Denpasar            9364.03  36004.30    8990.65     9592.06   

Tahun                                                         2025  \
Triwulan             Triwulan III Triwulan IV   Tahunan Triwulan I   
Kab/Kota                                                             
Kabupaten Jembrana        2504.85     2617.29   9965.23    2494.09   
Kabupaten Tabanan         4074.88     4207.22  16236.00    4050.17   
Kabupaten Badung          9600.06     9779.26  37667.34    9325.94   
Kabupaten Gianyar         4956.65     5260.64  19896.35    5024.54   
Kabupaten Klungkung       1593.76     1648.19   6348.54    1601.12   
Kabupaten Bangli          1220.73     1262.38   4876.68    1220.84   
Kabupaten Karangasem      2923.06     3026.51  11663.31    2933.76   
Kabupaten Buleleng        6184.07     6383.90  24453.82    6093.99   
Kota Denpasar             9599.82     9818.69  38001.22    9488.50   

Tahun                                                              
Triwulan             Triwulan II Triwulan III Triwulan IV Tahunan  
Kab/Kota                                                           
Kabupaten Jembrana           NaN          NaN         NaN     NaN  
Kabupaten Tabanan            NaN          NaN         NaN     NaN  
Kabupaten Badung             NaN          NaN         NaN     NaN  
Kabupaten Gianyar            NaN          NaN         NaN     NaN  
Kabupaten Klungkung          NaN          NaN         NaN     NaN  
Kabupaten Bangli             NaN          NaN         NaN     NaN  
Kabupaten Ka

In [184]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_416.xlsx", index=False, engine="openpyxl")

417 (Distribusi PDRB Triwulanan Menurut Kabupaten/Kota Atas Dasar Harga Berlaku Provinsi Bali)

In [185]:
data_417 = getDataByVarId(domain='5100', varId='417')
data_417.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,417,0,Tidak ada,125,2025,31,Triwulan I,1417012531,1417012531,5.67
5,2,Kab. Tabanan,417,0,Tidak ada,125,2025,31,Triwulan I,2417012531,2417012531,9.25
10,3,Kab. Badung,417,0,Tidak ada,125,2025,31,Triwulan I,3417012531,3417012531,24.72
15,4,Kab. Gianyar,417,0,Tidak ada,125,2025,31,Triwulan I,4417012531,4417012531,11.25
20,5,Kab. Klungkung,417,0,Tidak ada,125,2025,31,Triwulan I,5417012531,5417012531,3.74


In [186]:
# --- Ambil & bersihkan untuk urutan asli ---
data_417 = data_417[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_417['Kab/Kota'] = (
    data_417['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_417['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_417['Triwulan'] = data_417['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_417.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [187]:
wide

Tahun                 2022                                               \
Triwulan        Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan   
Kab/Kota                                                                  
Kab. Jembrana         6.12        5.92         5.88        5.74    5.90   
Kab. Tabanan          9.82        9.73         9.52        9.45    9.62   
Kab. Badung          20.88       21.99        23.06       23.67   22.47   
Kab. Gianyar         11.69       11.44        11.18       11.16   11.35   
Kab. Klungkung        3.86        3.78         3.70        3.65    3.74   
Kab. Bangli           3.04        3.02         2.95        2.93    2.98   
Kab. Karangasem       7.34        7.28         7.12        7.01    7.18   
Kab. Buleleng        14.85       14.65        14.49       14.27   14.55   
Kota Denpasar        22.41       22.19        22.10       22.13   22.20   
Provinsi Bali       100.00      100.00       100.00      100.00  100.00   

Tahun                 2023                                               \
Triwulan        Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan   
Kab/Kota                                                                  
Kab. Jembrana         5.71        5.63         5.67        5.66    5.67   
Kab. Tabanan          9.39        9.29         9.17        9.23    9.27   
Kab. Badung          24.58       24.56        25.36       24.79   24.83   
Kab. Gianyar         11.08       11.01        11.02       11.21   11.08   
Kab. Klungkung        3.66        3.64         3.61        3.71    3.66   
Kab. Bangli           2.90        2.90         2.82        2.87    2.87   
Kab. Karangasem       6.90        6.96         6.86        6.84    6.89   
Kab. Buleleng        13.92       13.96        13.84       13.97   13.92   
Kota Denpasar        21.85       22.04        21.66       21.72   21.82   
Provinsi Bali       100.00      100.00       100.00      100.00  100.00   

Tahun                 2024                                               \
Triwulan        Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan   
Kab/Kota                                                                  
Kab. Jembrana         5.65        5.58         5.60        5.70    5.63   
Kab. Tabanan          9.25        9.32         9.19        9.23    9.25   
Kab. Badung          24.91       25.14        25.46       24.93   25.11   
Kab. Gianyar         11.17       10.87        10.87       11.29   11.05   
Kab. Klungkung        3.73        3.66         3.66        3.69    3.68   
Kab. Bangli           2.86        2.85         2.81        2.81    2.83   
Kab. Karangasem       6.83        6.79         6.70        6.73    6.76   
Kab. Buleleng        13.80       13.79        13.85       13.91   13.84   
Kota Denpasar        21.79       22.00        21.86       21.71   21.84   
Provinsi Bali       100.00      100.00       100.00      100.00  100.00   

Tahun                 2025                                               
Triwulan        Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan  
Kab/Kota                                                                 
Kab. Jembrana         5.67         NaN          NaN         NaN     NaN  
Kab. Tabanan          9.25         NaN          NaN         NaN     NaN  
Kab. Badung          24.72         NaN          NaN         NaN     NaN  
Kab. Gianyar         11.25         NaN          NaN         NaN     NaN  
Kab. Klungkung        3.74         NaN          NaN         NaN     NaN  
Kab. Bangli           2.85         NaN          NaN         NaN     NaN  
Kab. Karangasem       6.81         NaN          NaN         NaN     NaN  
Kab. Buleleng        13.90         NaN          NaN         NaN     NaN  
Kota Denpasar        21.83         NaN          NaN         NaN     NaN  
Provinsi Bali       100.00         NaN          NaN         NaN     NaN

In [188]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_417.xlsx", index=False, engine="openpyxl")

418 (Laju Pertumbuhan (q-to-q) PDRB Triwulanan Menurut Kabupaten/Kota Provinsi Bali)

In [189]:
data_418 = getDataByVarId(domain='5100', varId='418')
data_418.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kabupaten Jembrana,418,0,Tidak ada,125,2025,31,Triwulan I,1418012531,1418012531,-4.71
5,2,Kabupaten Tabanan,418,0,Tidak ada,125,2025,31,Triwulan I,2418012531,2418012531,-3.73
10,3,Kabupaten Badung,418,0,Tidak ada,125,2025,31,Triwulan I,3418012531,3418012531,-4.64
15,4,Kabupaten Gianyar,418,0,Tidak ada,125,2025,31,Triwulan I,4418012531,4418012531,-4.49
20,5,Kabupaten Klungkung,418,0,Tidak ada,125,2025,31,Triwulan I,5418012531,5418012531,-2.86


In [190]:
# --- Ambil & bersihkan untuk urutan asli ---
data_418 = data_418[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_418['Kab/Kota'] = (
    data_418['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_418['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_418['Triwulan'] = data_418['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_418.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [191]:
wide

Tahun                      2023                                               \
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan   
Kab/Kota                                                                       
Kabupaten Jembrana        -4.14        5.18         2.21        2.45     NaN   
Kabupaten Tabanan         -4.55        5.41         0.07        3.20     NaN   
Kabupaten Badung          -1.91        5.61         2.67        1.66     NaN   
Kabupaten Gianyar         -4.56        6.06         1.14        4.42     NaN   
Kabupaten Klungkung       -3.38        5.92         0.27        4.07     NaN   
Kabupaten Bangli          -5.30        6.56        -1.54        4.44     NaN   
Kabupaten Karangasem      -5.70        7.55        -0.12        2.52     NaN   
Kabupaten Buleleng        -5.94        6.98         0.50        3.69     NaN   
Kota Denpasar             -5.23        7.74        -0.26        3.10     NaN   

Tahun                      2024                                               \
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan   
Kab/Kota                                                                       
Kabupaten Jembrana        -4.24        4.55         1.19        4.49     NaN   
Kabupaten Tabanan         -4.07        6.63        -0.73        3.25     NaN   
Kabupaten Badung          -4.32        6.91         1.59        1.87     NaN   
Kabupaten Gianyar         -4.28        3.53         0.67        6.13     NaN   
Kabupaten Klungkung       -3.76        4.30         0.49        3.41     NaN   
Kabupaten Bangli          -4.37        6.17        -0.96        3.41     NaN   
Kabupaten Karangasem      -3.97        5.45        -0.32        3.54     NaN   
Kabupaten Buleleng        -5.32        6.05         1.09        3.23     NaN   
Kota Denpasar             -3.99        6.69         0.08        2.28     NaN   

Tahun                      2025                                               
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan  
Kab/Kota                                                                      
Kabupaten Jembrana        -4.71         NaN          NaN         NaN     NaN  
Kabupaten Tabanan         -3.73         NaN          NaN         NaN     NaN  
Kabupaten Badung          -4.64         NaN          NaN         NaN     NaN  
Kabupaten Gianyar         -4.49         NaN          NaN         NaN     NaN  
Kabupaten Klungkung       -2.86         NaN          NaN         NaN     NaN  
Kabupaten Bangli          -3.29         NaN          NaN         NaN     NaN  
Kabupaten Karangasem      -3.06         NaN          NaN         NaN     NaN  
Kabupaten Buleleng        -4.54         NaN          NaN         NaN     NaN  
Kota Denpasar             -3.36         NaN          NaN         NaN     NaN

In [192]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_418.xlsx", index=False, engine="openpyxl")

419 (Laju Pertumbuhan (y-on-y) PDRB Triwulanan Menurut Kabupaten/Kota Provinsi Bali)

In [217]:
data_419 = getDataByVarId(domain='5100', varId='419')
data_419.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kabupaten Jembrana,419,0,Tidak ada,125,2025,31,Triwulan I,1419012531,1419012531,5.34
5,2,Kabupaten Tabanan,419,0,Tidak ada,125,2025,31,Triwulan I,2419012531,2419012531,5.22
10,3,Kabupaten Badung,419,0,Tidak ada,125,2025,31,Triwulan I,3419012531,3419012531,5.52
15,4,Kabupaten Gianyar,419,0,Tidak ada,125,2025,31,Triwulan I,4419012531,4419012531,5.65
20,5,Kabupaten Klungkung,419,0,Tidak ada,125,2025,31,Triwulan I,5419012531,5419012531,5.30


In [218]:
# --- Ambil & bersihkan untuk urutan asli ---
data_419 = data_419[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_419['Kab/Kota'] = (
    data_419['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_419['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_419['Triwulan'] = data_419['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_419.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [219]:
wide

Tahun                      2023                                               \
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan   
Kab/Kota                                                                       
Kabupaten Jembrana        -4.14        5.18         2.21        2.45    5.63   
Kabupaten Tabanan         -4.55        5.41         0.07        3.20    9.25   
Kabupaten Badung          -1.91        5.61         2.67        1.66   25.11   
Kabupaten Gianyar         -4.56        6.06         1.14        4.42   11.05   
Kabupaten Klungkung       -3.38        5.92         0.27        4.07    3.68   
Kabupaten Bangli          -5.30        6.56        -1.54        4.44    2.83   
Kabupaten Karangasem      -5.70        7.55        -0.12        2.52    6.76   
Kabupaten Buleleng        -5.94        6.98         0.50        3.69   13.84   
Kota Denpasar             -5.23        7.74        -0.26        3.10   21.84   

Tahun                      2024                                               \
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan   
Kab/Kota                                                                       
Kabupaten Jembrana         5.46        4.83         3.79        5.85    4.98   
Kabupaten Tabanan          4.43        5.64         4.80        4.86    4.94   
Kabupaten Badung           5.47        6.77         5.65        5.87    5.94   
Kabupaten Gianyar          7.23        4.66         4.18        5.89    5.47   
Kabupaten Klungkung        6.38        4.75         4.98        4.32    5.08   
Kabupaten Bangli           4.78        4.40         5.02        3.98    4.54   
Kabupaten Karangasem       5.76        3.69         3.47        4.50    4.33   
Kabupaten Buleleng         5.54        4.63         5.24        4.78    5.04   
Kota Denpasar              6.38        5.34         5.69        4.86    5.55   

Tahun                      2025                                               
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan  
Kab/Kota                                                                      
Kabupaten Jembrana         5.34         NaN          NaN         NaN     NaN  
Kabupaten Tabanan          5.22         NaN          NaN         NaN     NaN  
Kabupaten Badung           5.52         NaN          NaN         NaN     NaN  
Kabupaten Gianyar          5.65         NaN          NaN         NaN     NaN  
Kabupaten Klungkung        5.30         NaN          NaN         NaN     NaN  
Kabupaten Bangli           5.16         NaN          NaN         NaN     NaN  
Kabupaten Karangasem       5.49         NaN          NaN         NaN     NaN  
Kabupaten Buleleng         5.64         NaN          NaN         NaN     NaN  
Kota Denpasar              5.54         NaN          NaN         NaN     NaN

In [ ]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {komo}" for yr, komo in wide_flat.columns]
wide_flat.reset_index().to_excel("data_419.xlsx", index=False, engine="openpyxl")

420 (Laju Pertumbuhan (c-to-c) PDRB Triwulanan Menurut Kabupaten/Kota Provinsi Bali)

In [221]:
data_420 = getDataByVarId(domain='5100', varId='420')
data_420.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kabupaten Jembrana,420,0,Tidak ada,125,2025,31,Triwulan I,1420012531,1420012531,5.34
5,2,Kabupaten Tabanan,420,0,Tidak ada,125,2025,31,Triwulan I,2420012531,2420012531,5.22
10,3,Kabupaten Badung,420,0,Tidak ada,125,2025,31,Triwulan I,3420012531,3420012531,5.52
15,4,Kabupaten Gianyar,420,0,Tidak ada,125,2025,31,Triwulan I,4420012531,4420012531,5.65
20,5,Kabupaten Klungkung,420,0,Tidak ada,125,2025,31,Triwulan I,5420012531,5420012531,5.30


In [222]:
# --- Ambil & bersihkan untuk urutan asli ---
data_420 = data_420[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_420['Kab/Kota'] = (
    data_420['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_420['Kab/Kota'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_420['Triwulan'] = data_420['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_420.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Kab/Kota','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Kab/Kota',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [223]:
wide

Tahun                      2023                                               \
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan   
Kab/Kota                                                                       
Kabupaten Jembrana         2.28        2.50         3.00        3.66    3.66   
Kabupaten Tabanan          4.60        3.51         3.44        3.56    3.56   
Kabupaten Badung          15.15       13.01        12.45       11.29   11.29   
Kabupaten Gianyar          3.90        3.76         4.41        5.06    5.06   
Kabupaten Klungkung        3.90        3.68         3.98        4.70    4.70   
Kabupaten Bangli           4.44        3.78         3.40        3.50    3.50   
Kabupaten Karangasem       2.74        2.63         2.83        3.10    3.10   
Kabupaten Buleleng         3.31        3.25         3.22        3.64    3.64   
Kota Denpasar              6.01        6.29         5.93        5.69    5.69   

Tahun                      2024                                               \
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan   
Kab/Kota                                                                       
Kabupaten Jembrana         5.46        5.14         4.67        4.98    4.98   
Kabupaten Tabanan          4.43        5.05         4.97        4.94    4.94   
Kabupaten Badung           5.47        6.14         5.97        5.94    5.94   
Kabupaten Gianyar          7.23        5.91         5.32        5.47    5.47   
Kabupaten Klungkung        6.38        5.54         5.35        5.08    5.08   
Kabupaten Bangli           4.78        4.59         4.73        4.54    4.54   
Kabupaten Karangasem       5.76        4.68         4.27        4.33    4.33   
Kabupaten Buleleng         5.54        5.07         5.13        5.04    5.04   
Kota Denpasar              6.38        5.84         5.79        5.55    5.55   

Tahun                      2025                                               
Triwulan             Triwulan I Triwulan II Triwulan III Triwulan IV Tahunan  
Kab/Kota                                                                      
Kabupaten Jembrana         5.34         NaN          NaN         NaN     NaN  
Kabupaten Tabanan          5.22         NaN          NaN         NaN     NaN  
Kabupaten Badung           5.52         NaN          NaN         NaN     NaN  
Kabupaten Gianyar          5.65         NaN          NaN         NaN     NaN  
Kabupaten Klungkung        5.30         NaN          NaN         NaN     NaN  
Kabupaten Bangli           5.16         NaN          NaN         NaN     NaN  
Kabupaten Karangasem       5.49         NaN          NaN         NaN     NaN  
Kabupaten Buleleng         5.64         NaN          NaN         NaN     NaN  
Kota Denpasar              5.54         NaN          NaN         NaN     NaN

In [ ]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {komo}" for yr, komo in wide_flat.columns]
wide_flat.reset_index().to_excel("data_420.xlsx", index=False, engine="openpyxl")

231 (Produksi Tanaman Pangan Provinsi Bali Menurut Kabupaten/Kota)

In [210]:
data_231 = getDataByVarId(domain='5100', varId='231')
data_231.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,231,142,Padi,115,2015,0,Tahun,12311421150,12311421150,65295
1,1,Kab. Jembrana,231,143,Jagung,115,2015,0,Tahun,12311431150,12311431150,510
2,1,Kab. Jembrana,231,144,Kedelai,115,2015,0,Tahun,12311441150,12311441150,2397
3,1,Kab. Jembrana,231,145,Kacang Tanah,115,2015,0,Tahun,12311451150,12311451150,127
4,1,Kab. Jembrana,231,146,Kacang Hijau,115,2015,0,Tahun,12311461150,12311461150,42


In [ ]:
import pandas as pd
import numpy as np
import re

# ===== 1) Ambil & bersihkan =====
d = data_231[['vervar_label','tahun_label','turvar_label','content_label']].copy()

# Kab/Kota + urutan kemunculan
d['Kab/Kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Kab/Kota'].drop_duplicates().tolist()

# Komoditas dari turvar_label
d['Komoditas'] = (d['turvar_label'].astype(str)
                  .str.replace(r'<.*?>','', regex=True)
                  .str.replace(r'\s+',' ', regex=True)
                  .str.strip()
                  .str.title())

# Tahun -> int
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# Parse angka BPS (titik=ribuan, koma=desimal)
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s:  # ada desimal koma
        s = s.replace('.', '').replace(',', '.')
    else:         # hanya ribuan bertitik
        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ===== 2) Urutan komoditas & tahun =====
komo_order = ['Padi','Jagung','Kedelai','Kacang Tanah','Kacang Hijau','Ubi Kayu','Ubi Jalar']
present_komo = [k for k in komo_order if k in d['Komoditas'].unique()]
years = sorted(d['Tahun'].unique())

# ===== 3) Pivot: kolom = (Tahun, Komoditas) =====
wide = d.pivot_table(index='Kab/Kota',
                     columns=['Tahun','Komoditas'],  # <-- Tahun dulu, baru Komoditas
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# Susun kolom: tiap tahun berisi 7 komoditas berurutan
full_cols = pd.MultiIndex.from_product([years, present_komo], names=['Tahun','Komoditas'])
wide = wide.reindex(columns=full_cols)

# Urut baris sesuai kemunculan Kab/Kota
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order_kab if k in wide.index], ordered=True)
wide = wide.sort_index()

# ===== 4) Ekspor: flatten "YYYY - Komoditas" =====
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {komo}" for yr, komo in wide_flat.columns]
wide_flat.reset_index().to_excel("data_231.xlsx", index=False, engine="openpyxl")


In [215]:
wide

Tahun                2006                                              \
Komoditas            Padi   Jagung  Kedelai Kacang Tanah Kacang Hijau   
Kab/Kota                                                                
Kab. Jembrana     50700.0    164.0   2654.0         22.0         61.0   
Kab. Tabanan     239182.0   2365.0   1017.0        100.0         22.0   
Kab. Badung      117892.0   1194.0   2942.0        522.0          5.0   
Kab. Gianyar     166881.0   1059.0   1249.0        774.0          4.0   
Kab. Klungkung    28994.0  14438.0   1388.0       2234.0          0.0   
Kab. Bangli       35085.0  12253.0    133.0       4898.0         92.0   
Kab. Karangasem   60535.0  20453.0    104.0       7750.0        588.0   
Kab. Buleleng    110609.0  26127.0    231.0       1711.0        488.0   
Kota Denpasar     31013.0     52.0   1126.0         30.0          0.0   
Provinsi Bali    840891.0  78105.0  10845.0      18040.0       1260.0   

Tahun                                    2007                   ...  \
Komoditas        Ubi Kayu Ubi Jalar      Padi   Jagung Kedelai  ...   
Kab/Kota                                                        ...   
Kab. Jembrana       370.0       0.0   48922.0   2281.0   893.0  ...   
Kab. Tabanan       1087.0    2396.0  230433.0   6909.0  1195.0  ...   
Kab. Badung        6858.0    5848.0  119902.0   1749.0  1992.0  ...   
Kab. Gianyar       4958.0    5921.0  173241.0   1875.0  1682.0  ...   
Kab. Klungkung    17538.0   48909.0   33394.0  11636.0  1681.0  ...   
Kab. Bangli       29716.0    3293.0   27717.0  17277.0    95.0  ...   
Kab. Karangasem   86238.0   24886.0   65658.0  14704.0   234.0  ...   
Kab. Buleleng     12293.0     824.0  110573.0  12779.0   212.0  ...   
Kota Denpasar         0.0       0.0   29936.0      0.0   432.0  ...   
Provinsi Bali    159058.0   92078.0  839775.0  69210.0  8417.0  ...   

Tahun                   2014                          2015                   \
Komoditas       Kacang Hijau  Ubi Kayu Ubi Jalar      Padi   Jagung Kedelai   
Kab/Kota                                                                      
Kab. Jembrana          132.0     375.0       0.0   65295.0    510.0  2397.0   
Kab. Tabanan             0.0     870.0     610.0  194134.0   5035.0   679.0   
Kab. Badung              1.0    4828.0   12963.0  105951.0    149.0  1509.0   
Kab. Gianyar            23.0    2910.0    2661.0  192518.0    727.0   988.0   
Kab. Klungkung          31.0    9243.0    2558.0   38070.0   5378.0  1242.0   
Kab. Bangli              0.0   11336.0   21457.0   28718.0   1975.0   101.0   
Kab. Karangasem        331.0   88168.0   14062.0   71078.0   8784.0    44.0   
Kab. Buleleng          423.0   14157.0      84.0  128209.0  18045.0    25.0   
Kota Denpasar            0.0       0.0       0.0   29926.0      0.0   274.0   
Provinsi Bali          941.0  131887.0   54395.0  853899.0  40603.0  7259.0   

Tahun                                                         
Komoditas       Kacang Tanah Kacang Hijau Ubi Kayu Ubi Jalar  
Kab/Kota                                                      
Kab. Jembrana          127.0         42.0    452.0       0.0  
Kab. Tabanan            22.0          0.0    757.0     129.0  
Kab. Badung            610.0          0.0   6009.0    8238.0  
Kab. Gianyar           229.0         51.0   3816.0     746.0  
Kab. Klungkung         734.0         35.0   5989.0    1522.0  
Kab. Bangli           1024.0          0.0   5614.0   11729.0  
Kab. Karangasem       3054.0        229.0  55748.0   14206.0  
Kab. Buleleng         1265.0        159.0   7685.0      85.0  
Kota Denpasar            0.0          0.0      0.0       0.0  
Provinsi Bali         7065.0        516.0  86070.0   36655.0  

[10 rows x 70 columns]

In [ ]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {komo}" for yr, komo in wide_flat.columns]
wide_flat.reset_index().to_excel("data_231.xlsx", index=False, engine="openpyxl")

382 (Luas Panen Padi per Bulan Menurut Kabupaten/Kota di Provinsi Bali)

In [200]:
data_382 = getDataByVarId(domain='5100', varId='382')
data_382.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,382,0,Tidak ada,124,2024,1,Januari,138201241,138201241,131.08
1,1,Kab. Jembrana,382,0,Tidak ada,124,2024,2,Februari,138201242,138201242,78.97
2,1,Kab. Jembrana,382,0,Tidak ada,124,2024,3,Maret,138201243,138201243,663.06
3,1,Kab. Jembrana,382,0,Tidak ada,124,2024,4,April,138201244,138201244,1159.99
4,1,Kab. Jembrana,382,0,Tidak ada,124,2024,5,Mei,138201245,138201245,922.99


In [ ]:
# ========== 1) Ambil & bersihkan ==========
# pakai kolom apa adanya dari API
d = data_382[['vervar_label','tahun_label','turtahun_label','turtahun_val','content_label']].copy()

# nama kab/kota & urutan kemunculan
d['Kab/Kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Kab/Kota'].drop_duplicates().tolist()

# bulan: langsung dari API (tanpa mapping), hanya bersihkan spasi/tag
d['Bulan'] = (d['turtahun_label'].astype(str)
              .str.replace(r'<.*?>','', regex=True)
              .str.replace(r'\s+',' ', regex=True)
              .str.strip())

# index bulan untuk pengurutan (1–12) langsung dari API
d['BulanIdx'] = pd.to_numeric(d['turtahun_val'], errors='coerce')

# tahun numerik
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# parse angka BPS
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s:  # koma sebagai desimal
        s = s.replace('.', '').replace(',', '.')
    else:         # titik sebagai ribuan
        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ========== 2) Urutan kolom: Tahun naik, lalu bulan sesuai turtahun_val ==========
years = sorted(d['Tahun'].unique())
# ambil urutan bulan dari API (1–12); fallback ke urutan kemunculan jika BulanIdx kosong
if d['BulanIdx'].notna().any():
    bulan_order = (d[['Bulan','BulanIdx']]
                   .dropna(subset=['BulanIdx'])
                   .sort_values('BulanIdx')
                   .drop_duplicates('Bulan'))['Bulan'].tolist()
else:
    bulan_order = d['Bulan'].drop_duplicates().tolist()

# ========== 3) Pivot ==========
wide = d.pivot_table(index='Kab/Kota',
                     columns=['Tahun','Bulan'],
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# susun kolom lengkap sesuai urutan
full_cols = pd.MultiIndex.from_product([years, bulan_order], names=['Tahun','Bulan'])
wide = wide.reindex(columns=full_cols)

# urutkan baris sesuai kemunculan kab/kota
wide.index = pd.CategoricalIndex(wide.index,
                                 categories=[k for k in order_kab if k in wide.index],
                                 ordered=True)
wide = wide.sort_index()


In [226]:
wide

Tahun                2018                                                      \
Bulan             Januari  Februari      Maret     April        Mei      Juni   
Kab/Kota                                                                        
Kab. Jembrana     25759.0  217703.0   141409.0   70153.0    97158.0   91026.0   
Kab. Tabanan     260635.0  169224.0   335183.0  570172.0   187282.0  332707.0   
Kab. Badung      102613.0    4245.0    14484.0  245378.0    19825.0  176385.0   
Kab. Gianyar      24370.0   52739.0   156048.0  157162.0   276187.0  152507.0   
Kab. Klungkung    73185.0   14831.0    28902.0   34495.0     3289.0       0.0   
Kab. Bangli       28103.0    3769.0     4906.0   16159.0    42475.0   24101.0   
Kab. Karangasem   56038.0   38559.0    27879.0   73854.0   213123.0   61242.0   
Kab. Buleleng     15119.0    3239.0   109782.0   26907.0   215747.0   28537.0   
Kota Denpasar     10531.0    8259.0    63482.0   43677.0    53104.0    7898.0   
Provinsi Bali    815683.0  550773.0  1056585.0  148012.0  1286615.0  874403.0   

Tahun                                                    ...      2024  \
Bulan                Juli   Agustus September   Oktober  ...     April   
Kab/Kota                                                 ...             
Kab. Jembrana    217341.0   68655.0   32405.0   50993.0  ...  115999.0   
Kab. Tabanan     195757.0   15255.0  153426.0  349271.0  ...  203877.0   
Kab. Badung      100293.0   47995.0  167136.0  177338.0  ...  114425.0   
Kab. Gianyar     115939.0   81691.0  122236.0   95221.0  ...  108176.0   
Kab. Klungkung    25954.0   14412.0   35403.0   13157.0  ...    1875.0   
Kab. Bangli       12285.0   30881.0    3946.0   36633.0  ...   25235.0   
Kab. Karangasem   62478.0    3935.0   57412.0   79044.0  ...   42319.0   
Kab. Buleleng    131976.0  217255.0  134669.0  109414.0  ...  197641.0   
Kota Denpasar     11072.0    2633.0    2633.0   50453.0  ...    3812.0   
Provinsi Bali    873095.0  655422.0   74478.0  961524.0  ...  864542.0   

Tahun                                                                \
Bulan                  Mei       Juni      Juli   Agustus September   
Kab/Kota                                                              
Kab. Jembrana      92299.0   161483.0   96209.0   51857.0   83553.0   
Kab. Tabanan      331043.0   449142.0  223386.0  132951.0  100196.0   
Kab. Badung       233293.0    78991.0   40586.0   81741.0  173682.0   
Kab. Gianyar      294016.0   277487.0   14933.0  133535.0   72995.0   
Kab. Klungkung     28825.0    41982.0   44498.0    3932.0   21827.0   
Kab. Bangli        20953.0    48068.0   22338.0   44979.0   36661.0   
Kab. Karangasem   240194.0    73836.0   16685.0   25614.0   51865.0   
Kab. Buleleng     314289.0     9136.0   82049.0  245962.0  168249.0   
Kota Denpasar      95126.0    11252.0       0.0    2232.0   36288.0   
Provinsi Bali    1650038.0  1233601.0  675081.0  758191.0  745317.0   

Tahun                                                       
Bulan              Oktober  November  Desember     Tahunan  
Kab/Kota                                                    
Kab. Jembrana     138007.0    4693.0   13811.0    887459.0  
Kab. Tabanan      278488.0  197193.0  246279.0   2690639.0  
Kab. Badung       211773.0   12244.0   27641.0   1401266.0  
Kab. Gianyar      106089.0  163533.0   17925.0    188277.0  
Kab. Klungkung     58847.0   23229.0    2603.0    395741.0  
Kab. Bangli        43166.0    1402.0   10766.0    341926.0  
Kab. Karangasem    47032.0  118792.0    6981.0    800098.0  
Kab. Buleleng     145281.0   56068.0   61843.0    163412.0  
Kota Denpasar      29369.0   70865.0   29873.0    346374.0  
Provinsi Bali    1058053.0   81307.0  665304.0  10380393.0  

[10 rows x 91 columns]

In [227]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {bln}" for yr, bln in wide_flat.columns]
wide_flat.reset_index().to_excel("data_382.xlsx", index=False, engine="openpyxl")

383 (Produksi Padi per Bulan Menurut Kabupaten/Kota di Provinsi Bali)

In [228]:
data_383 = getDataByVarId(domain='5100', varId='383')
data_383.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,383,0,Tidak ada,124,2024,1,Januari,138301241,138301241,898.74
1,1,Kab. Jembrana,383,0,Tidak ada,124,2024,2,Februari,138301242,138301242,541.45
2,1,Kab. Jembrana,383,0,Tidak ada,124,2024,3,Maret,138301243,138301243,4546.24
3,1,Kab. Jembrana,383,0,Tidak ada,124,2024,4,April,138301244,138301244,7953.42
4,1,Kab. Jembrana,383,0,Tidak ada,124,2024,5,Mei,138301245,138301245,5643.51


In [229]:
# ========== 1) Ambil & bersihkan ==========
# pakai kolom apa adanya dari API
d = data_383[['vervar_label','tahun_label','turtahun_label','turtahun_val','content_label']].copy()

# nama kab/kota & urutan kemunculan
d['Kab/Kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Kab/Kota'].drop_duplicates().tolist()

# bulan: langsung dari API (tanpa mapping), hanya bersihkan spasi/tag
d['Bulan'] = (d['turtahun_label'].astype(str)
              .str.replace(r'<.*?>','', regex=True)
              .str.replace(r'\s+',' ', regex=True)
              .str.strip())

# index bulan untuk pengurutan (1–12) langsung dari API
d['BulanIdx'] = pd.to_numeric(d['turtahun_val'], errors='coerce')

# tahun numerik
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# parse angka BPS
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s:  # koma sebagai desimal
        s = s.replace('.', '').replace(',', '.')
    else:         # titik sebagai ribuan
        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ========== 2) Urutan kolom: Tahun naik, lalu bulan sesuai turtahun_val ==========
years = sorted(d['Tahun'].unique())
# ambil urutan bulan dari API (1–12); fallback ke urutan kemunculan jika BulanIdx kosong
if d['BulanIdx'].notna().any():
    bulan_order = (d[['Bulan','BulanIdx']]
                   .dropna(subset=['BulanIdx'])
                   .sort_values('BulanIdx')
                   .drop_duplicates('Bulan'))['Bulan'].tolist()
else:
    bulan_order = d['Bulan'].drop_duplicates().tolist()

# ========== 3) Pivot ==========
wide = d.pivot_table(index='Kab/Kota',
                     columns=['Tahun','Bulan'],
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# susun kolom lengkap sesuai urutan
full_cols = pd.MultiIndex.from_product([years, bulan_order], names=['Tahun','Bulan'])
wide = wide.reindex(columns=full_cols)

# urutkan baris sesuai kemunculan kab/kota
wide.index = pd.CategoricalIndex(wide.index,
                                 categories=[k for k in order_kab if k in wide.index],
                                 ordered=True)
wide = wide.sort_index()


In [230]:
wide

Tahun                 2018                                              \
Bulan              Januari   Februari      Maret      April        Mei   
Kab/Kota                                                                 
Kab. Jembrana     154816.0  1308431.0   849892.0   421631.0   565664.0   
Kab. Tabanan     1559341.0  1012442.0  2005351.0  3411256.0  1074736.0   
Kab. Badung       661834.0   273794.0    93419.0  1582641.0  1149499.0   
Kab. Gianyar     1659975.0   359234.0  1062929.0  1070517.0  1622913.0   
Kab. Klungkung    439712.0    89108.0    17365.0   207254.0    18656.0   
Kab. Bangli       151361.0    20365.0   265088.0    87313.0   206977.0   
Kab. Karangasem    33902.0   233275.0   168663.0   446803.0  1383044.0   
Kab. Buleleng      90725.0    19436.0   658773.0   161462.0  1289253.0   
Kota Denpasar      69271.0    54326.0   417572.0   287298.0   307583.0   
Provinsi Bali    5126055.0  3370411.0  6536108.0  9129333.0  7618325.0   

Tahun                                                                   ...  \
Bulan                 Juni       Juli    Agustus  September    Oktober  ...   
Kab/Kota                                                                ...   
Kab. Jembrana     529963.0  1265382.0   399717.0   173962.0    27375.0  ...   
Kab. Tabanan     1909272.0  1123371.0   875423.0   863193.0   196504.0  ...   
Kab. Badung      1022721.0   581522.0   278286.0   105051.0  1114634.0  ...   
Kab. Gianyar      896152.0   681274.0   480028.0   742744.0   578592.0  ...   
Kab. Klungkung         0.0   147219.0    81749.0   253511.0    94214.0  ...   
Kab. Bangli       117467.0    59876.0   150512.0   102664.0    95308.0  ...   
Kab. Karangasem   397425.0   405446.0   255359.0   327302.0   450625.0  ...   
Kab. Buleleng      17053.0   788657.0  1298264.0   878164.0   713478.0  ...   
Kota Denpasar      45746.0     6413.0    15251.0    20489.0   392603.0  ...   
Provinsi Bali    5089276.0  5116877.0  3834589.0  4412539.0  5678244.0  ...   

Tahun                 2024                                              \
Bulan                April        Mei       Juni       Juli    Agustus   
Kab/Kota                                                                 
Kab. Jembrana     795342.0   564351.0   987367.0   588258.0   317073.0   
Kab. Tabanan     1253188.0  1942004.0   263481.0  1310453.0   779933.0   
Kab. Badung       754748.0  1421067.0   481161.0   247223.0   497912.0   
Kab. Gianyar      705198.0  1655258.0  1562203.0   840702.0   751779.0   
Kab. Klungkung    121432.0   170022.0   247628.0   262468.0   231926.0   
Kab. Bangli       138604.0   100986.0   231853.0   107746.0   216953.0   
Kab. Karangasem   254611.0   135070.0   415207.0    93826.0   144037.0   
Kab. Buleleng     107065.0  1724742.0   501362.0   450265.0   134978.0   
Kota Denpasar     276449.0   549826.0    65036.0        0.0    12901.0   
Provinsi Bali    5370222.0  9478956.0  7126627.0  3900941.0  4302294.0   

Tahun                                                                    
Bulan            September    Oktober   November   Desember     Tahunan  
Kab/Kota                                                                 
Kab. Jembrana     519565.0   858181.0   291826.0    85882.0   5606488.0  
Kab. Tabanan      628406.0  1746529.0  1236744.0   154460.0  16322678.0  
Kab. Badung      1368707.0  1668885.0   964893.0   217826.0   9711336.0  
Kab. Gianyar      473409.0    68804.0  1060593.0  1162526.0  11496606.0  
Kab. Klungkung    140346.0   378382.0   149361.0   167374.0   2467567.0  
Kab. Bangli       200005.0   235493.0    76486.0    58559.0   1780951.0  
Kab. Karangasem   336389.0   305045.0   770462.0   452775.0   4808636.0  
Kab. Buleleng     946604.0   817384.0   315453.0   347943.0   8994272.0  
Kota Denpasar     264639.0   214181.0   516797.0   217855.0   2358801.0  
Provinsi Bali     487807.0   691212.0  5382615.0   425534.0  63547335.0  

[10 rows x 91 columns]

In [231]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {bln}" for yr, bln in wide_flat.columns]
wide_flat.reset_index().to_excel("data_383.xlsx", index=False, engine="openpyxl")

384 (Produksi Beras per Bulan Menurut Kabupaten/Kota di Provinsi Bali)

In [232]:
data_384 = getDataByVarId(domain='5100', varId='384')
data_384.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,384,0,Tidak ada,124,2024,1,Januari,138401241,138401241,507.0
1,1,Kab. Jembrana,384,0,Tidak ada,124,2024,2,Februari,138401242,138401242,305.0
2,1,Kab. Jembrana,384,0,Tidak ada,124,2024,3,Maret,138401243,138401243,2564.0
3,1,Kab. Jembrana,384,0,Tidak ada,124,2024,4,April,138401244,138401244,4485.0
4,1,Kab. Jembrana,384,0,Tidak ada,124,2024,5,Mei,138401245,138401245,3183.0


In [233]:
# ========== 1) Ambil & bersihkan ==========
# pakai kolom apa adanya dari API
d = data_384[['vervar_label','tahun_label','turtahun_label','turtahun_val','content_label']].copy()

# nama kab/kota & urutan kemunculan
d['Kab/Kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Kab/Kota'].drop_duplicates().tolist()

# bulan: langsung dari API (tanpa mapping), hanya bersihkan spasi/tag
d['Bulan'] = (d['turtahun_label'].astype(str)
              .str.replace(r'<.*?>','', regex=True)
              .str.replace(r'\s+',' ', regex=True)
              .str.strip())

# index bulan untuk pengurutan (1–12) langsung dari API
d['BulanIdx'] = pd.to_numeric(d['turtahun_val'], errors='coerce')

# tahun numerik
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# parse angka BPS
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s:  # koma sebagai desimal
        s = s.replace('.', '').replace(',', '.')
    else:         # titik sebagai ribuan
        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ========== 2) Urutan kolom: Tahun naik, lalu bulan sesuai turtahun_val ==========
years = sorted(d['Tahun'].unique())
# ambil urutan bulan dari API (1–12); fallback ke urutan kemunculan jika BulanIdx kosong
if d['BulanIdx'].notna().any():
    bulan_order = (d[['Bulan','BulanIdx']]
                   .dropna(subset=['BulanIdx'])
                   .sort_values('BulanIdx')
                   .drop_duplicates('Bulan'))['Bulan'].tolist()
else:
    bulan_order = d['Bulan'].drop_duplicates().tolist()

# ========== 3) Pivot ==========
wide = d.pivot_table(index='Kab/Kota',
                     columns=['Tahun','Bulan'],
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# susun kolom lengkap sesuai urutan
full_cols = pd.MultiIndex.from_product([years, bulan_order], names=['Tahun','Bulan'])
wide = wide.reindex(columns=full_cols)

# urutkan baris sesuai kemunculan kab/kota
wide.index = pd.CategoricalIndex(wide.index,
                                 categories=[k for k in order_kab if k in wide.index],
                                 ordered=True)
wide = wide.sort_index()


In [234]:
wide

Tahun                 2018                                              \
Bulan              Januari   Februari      Maret      April        Mei   
Kab/Kota                                                                 
Kab. Jembrana       8686.0   734096.0   476832.0   236556.0   317366.0   
Kab. Tabanan      874869.0   568031.0  1125103.0  1913887.0   602981.0   
Kab. Badung       371322.0   153612.0   524128.0   887942.0   644927.0   
Kab. Gianyar       93133.0   201548.0   596357.0   600614.0   910536.0   
Kab. Klungkung    246701.0    49994.0    97426.0    11628.0    10467.0   
Kab. Bangli        84921.0    11426.0   148728.0    48987.0   116125.0   
Kab. Karangasem   190207.0   130879.0    94628.0   250679.0   775958.0   
Kab. Buleleng      50901.0    10905.0   369605.0   905883.0   723336.0   
Kota Denpasar      38865.0     3048.0   234279.0   161189.0    17257.0   
Provinsi Bali    2875976.0  1890971.0  3667086.0  5122017.0  4274266.0   

Tahun                                                                   ...  \
Bulan                 Juni       Juli    Agustus  September    Oktober  ...   
Kab/Kota                                                                ...   
Kab. Jembrana     297336.0   709943.0   224261.0    97601.0   153588.0  ...   
Kab. Tabanan     1071198.0   630268.0   491157.0   484295.0  1102487.0  ...   
Kab. Badung       573798.0   326263.0   156133.0   589389.0   625366.0  ...   
Kab. Gianyar      502787.0   382229.0    26932.0   416717.0   324619.0  ...   
Kab. Klungkung         0.0    82597.0    45865.0   142232.0    52859.0  ...   
Kab. Bangli        65905.0    33593.0    84445.0     5760.0    53473.0  ...   
Kab. Karangasem   222976.0   227476.0   143269.0   183633.0   252823.0  ...   
Kab. Buleleng      95676.0   442476.0   728392.0   492694.0   400297.0  ...   
Kota Denpasar      25666.0     3598.0     8557.0    11495.0    22027.0  ...   
Provinsi Bali    2855342.0  2870825.0  2151399.0  2475656.0  3185782.0  ...   

Tahun                2024                                                    \
Bulan               April       Mei      Juni      Juli   Agustus September   
Kab/Kota                                                                      
Kab. Jembrana     44850.0   31830.0   55680.0   33180.0   17880.0   29300.0   
Kab. Tabanan      70670.0  109520.0  148590.0   73900.0   43980.0   35440.0   
Kab. Badung       42560.0   80140.0   27140.0   13940.0   28080.0   77190.0   
Kab. Gianyar      39770.0   93350.0   88100.0   47410.0   42400.0   26700.0   
Kab. Klungkung     6850.0    9590.0   13970.0   14800.0   13080.0    7910.0   
Kab. Bangli        7820.0    5700.0   13080.0    6080.0   12240.0   11280.0   
Kab. Karangasem   14360.0   76170.0   23420.0    5290.0    8120.0   18970.0   
Kab. Buleleng     60380.0   97270.0   28270.0   25390.0   76120.0   53380.0   
Kota Denpasar     15590.0   31010.0    3670.0       0.0     730.0   14920.0   
Provinsi Bali    302860.0  534570.0  401910.0  220000.0  242630.0  275100.0   

Tahun                                                     
Bulan             Oktober  November  Desember    Tahunan  
Kab/Kota                                                  
Kab. Jembrana     48400.0   16460.0    4840.0   316180.0  
Kab. Tabanan      98500.0   69750.0   87110.0   920520.0  
Kab. Badung       94120.0   54420.0   12280.0   547680.0  
Kab. Gianyar      38800.0   59810.0   65560.0   648360.0  
Kab. Klungkung    21340.0    8420.0    9440.0   139160.0  
Kab. Bangli       13280.0    4310.0    3300.0   100450.0  
Kab. Karangasem   17200.0   43450.0   25530.0   271170.0  
Kab. Buleleng     46100.0   17790.0   19620.0   507230.0  
Kota Denpasar     12080.0   29150.0   12290.0   133040.0  
Provinsi Bali    389810.0  303560.0  239980.0  3583790.0  

[10 rows x 91 columns]

In [235]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {bln}" for yr, bln in wide_flat.columns]
wide_flat.reset_index().to_excel("data_384.xlsx", index=False, engine="openpyxl")

431 (Jumlah Petani Menurut Kabupaten/Kota dan Jenis Kelamin di Provinsi Bali)

In [236]:
data_431 = getDataByVarId(domain='5100', varId='431')
data_431.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
2,1,Kab. Jembrana,431,43,Laki-Laki + Perempuan,123,2023,0,Tahun,1431431230,1431431230,39587.0
5,2,Kab. Tabanan,431,43,Laki-Laki + Perempuan,123,2023,0,Tahun,2431431230,2431431230,54890.0
8,3,Kab. Badung,431,43,Laki-Laki + Perempuan,123,2023,0,Tahun,3431431230,3431431230,23621.0
11,4,Kab. Gianyar,431,43,Laki-Laki + Perempuan,123,2023,0,Tahun,4431431230,4431431230,32700.0
14,5,Kab. Klungkung,431,43,Laki-Laki + Perempuan,123,2023,0,Tahun,5431431230,5431431230,16296.0


In [239]:
# ============= 1) Ambil & bersihkan dari API =============
d = data_431[['vervar_label','tahun_label','turvar_label','content_label']].copy()

# Nama kab/kota & urutan kemunculan
d['Kab/Kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Kab/Kota'].drop_duplicates().tolist()

# Label jenis kelamin dari API (tanpa mapping, cuma dirapikan)
d['JK'] = (d['turvar_label'].astype(str)
           .str.replace(r'<.*?>','', regex=True)
           .str.replace(r'\s+',' ', regex=True)
           .str.strip())

# Tahun numerik
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# Nilai numerik (support format BPS)
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s: s = s.replace('.', '').replace(',', '.')
    else:        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ============= 2) Pivot =============
wide = d.pivot_table(index='Kab/Kota',
                     columns=['Tahun','JK'],
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# Susun kolom per tahun mengikuti preferensi JK, tapi hanya yang ADA di data
years = sorted(d['Tahun'].unique())
jk_pref = ['Laki-laki', 'Perempuan', 'Laki-laki + Perempuan']  # pakai label API kalau ada
present = [j for j in jk_pref if j in wide.columns.get_level_values(1).unique()]
# kalau ada label lain dari API, ikutkan di belakang tanpa dihitung apa-apa
others  = [j for j in wide.columns.get_level_values(1).unique() if j not in present]
jk_order = present + others

full_cols = pd.MultiIndex.from_product([years, jk_order], names=['Tahun','JK'])
wide = wide.reindex(columns=full_cols)

# Urut baris sesuai kemunculan kab/kota
wide.index = pd.CategoricalIndex(wide.index,
                                 categories=[k for k in order_kab if k in wide.index],
                                 ordered=True)
wide = wide.sort_index()


In [240]:
wide

Tahun                 2013                                        2018  \
JK               Laki-laki  Perempuan Laki-Laki + Perempuan  Laki-laki   
Kab/Kota                                                                 
Kab. Jembrana     366890.0   118680.0              485570.0   375730.0   
Kab. Tabanan      602710.0   171260.0              773970.0   574340.0   
Kab. Badung       319350.0    91480.0              410830.0   285720.0   
Kab. Gianyar      391780.0    56440.0              448220.0   392100.0   
Kab. Klungkung    211600.0    47680.0              259280.0   186180.0   
Kab. Bangli       415100.0   124410.0              539510.0   402280.0   
Kab. Karangasem   683120.0   245710.0              928830.0   631480.0   
Kab. Buleleng     842880.0   405040.0             1247920.0   746300.0   
Kota Denpasar      55660.0     8850.0               64510.0    49090.0   
Provinsi Bali    3889090.0  1269550.0             5158640.0  3643220.0   

Tahun                                                 2023            \
JK               Perempuan Laki-Laki + Perempuan Laki-laki Perempuan   
Kab/Kota                                                               
Kab. Jembrana     127340.0              503070.0       NaN       NaN   
Kab. Tabanan      127620.0              701960.0       NaN       NaN   
Kab. Badung        80150.0              365870.0       NaN       NaN   
Kab. Gianyar       94940.0              487040.0       NaN       NaN   
Kab. Klungkung     38410.0              224590.0       NaN       NaN   
Kab. Bangli        99230.0              501510.0       NaN       NaN   
Kab. Karangasem   262430.0              893910.0       NaN       NaN   
Kab. Buleleng     287090.0             1033390.0       NaN       NaN   
Kota Denpasar      13960.0               63050.0       NaN       NaN   
Provinsi Bali    1131170.0             4774390.0       NaN       NaN   

Tahun                                  
JK              Laki-Laki + Perempuan  
Kab/Kota                               
Kab. Jembrana                395870.0  
Kab. Tabanan                 548900.0  
Kab. Badung                  236210.0  
Kab. Gianyar                 327000.0  
Kab. Klungkung               162960.0  
Kab. Bangli                  423980.0  
Kab. Karangasem              670350.0  
Kab. Buleleng                819630.0  
Kota Denpasar                 31870.0  
Provinsi Bali               3616770.0

In [241]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {bln}" for yr, bln in wide_flat.columns]
wide_flat.reset_index().to_excel("data_431.xlsx", index=False, engine="openpyxl")

348 (Produksi Kelapa Menurut Kabupaten/Kota di Provinsi Bali)

In [124]:
data_348 = getDataByVarId(domain='5100', varId='348')
data_348.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,348,0,Tidak ada,124,2024,0,Tahun,134801240,134801240,17931
1,2,Kab. Tabanan,348,0,Tidak ada,124,2024,0,Tahun,234801240,234801240,16181
2,3,Kab. Badung,348,0,Tidak ada,124,2024,0,Tahun,334801240,334801240,2242
3,4,Kab. Gianyar,348,0,Tidak ada,124,2024,0,Tahun,434801240,434801240,3365
4,5,Kab. Klungkung,348,0,Tidak ada,124,2024,0,Tahun,534801240,534801240,2690


In [125]:
# --- ambil & bersihkan untuk urutan asli ---
data_348 = data_348[['vervar_label','tahun_label','content_label']].copy()
data_348['kabupaten_kota'] = (data_348['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_348['kabupaten_kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_348.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['kabupaten_kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='kabupaten_kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [126]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kabupaten_kota,,,,,,,,,,
Kab. Jembrana,18578,18748,17520,16464,16685,16696,16950,18042,17894,17931
Kab. Tabanan,18982,18364,11738,11882,15060,15098,15148,15239,15127,16181
Kab. Badung,2588,2330,1772,1853,1913,1949,1765,1995,2009,2242
Kab. Gianyar,3925,4143,3988,3646,3581,3662,3583,3530,3181,3365
Kab. Klungkung,2644,2502,2612,2425,2959,2826,2766,3092,2597,2690
Kab. Bangli,3041,2942,2924,2936,2822,2941,3032,3057,2935,3204
Kab. Karangasem,15337,14622,14562,16631,14233,14467,14589,14282,13805,14392
Kab. Buleleng,8722,9116,8037,9402,9557,9511,9483,9420,8744,9062
Kota Denpasar,61,62,71,52,54,53,54,76,54,76


In [127]:
wide.reset_index().to_excel("data_348.xlsx", index=False, engine="openpyxl")

349 (Produksi Kopi Arabika Menurut Kabupaten/Kota di Provinsi Bali)

In [49]:
data_349 = getDataByVarId(domain='5100', varId='349')
data_349.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,349,0,Tidak ada,124,2024,0,Tahun,134901240,134901240,0
1,2,Kab. Tabanan,349,0,Tidak ada,124,2024,0,Tahun,234901240,234901240,16
2,3,Kab. Badung,349,0,Tidak ada,124,2024,0,Tahun,334901240,334901240,520
3,4,Kab. Gianyar,349,0,Tidak ada,124,2024,0,Tahun,434901240,434901240,26
4,5,Kab. Klungkung,349,0,Tidak ada,124,2024,0,Tahun,534901240,534901240,0


In [50]:
# --- ambil & bersihkan untuk urutan asli ---
data_349 = data_349[['vervar_label','tahun_label','content_label']].copy()
data_349['kabupaten/kota'] = (data_349['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_349['kabupaten/kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_349.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['kabupaten/kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='kabupaten/kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [51]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kabupaten/kota,,,,,,,,,,
Kab. Jembrana,0,0,0,0,0,0,0,0,0,0
Kab. Tabanan,14,18,11,21,21,24,32,23,23,16
Kab. Badung,667,633,582,598,534,568,562,561,460,520
Kab. Gianyar,53,51,19,22,19,24,26,21,28,26
Kab. Klungkung,0,0,0,0,0,0,0,0,0,0
Kab. Bangli,2456,2346,2201,2252,2247,2249,2173,2082,1960,2164
Kab. Karangasem,104,118,122,87,84,73,77,70,65,68
Kab. Buleleng,860,886,539,1237,1278,1252,1114,1135,1107,992
Kota Denpasar,0,0,0,0,0,0,0,0,0,0


In [11]:
wide.reset_index().to_excel("data_349.xlsx", index=False, engine="openpyxl")

350 (Produksi Kopi Robusta Menurut Kabupaten/Kota di Provinsi Bali)

In [142]:
data_350 = getDataByVarId(domain='5100', varId='350')
data_350.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,350,0,Tidak ada,124,2024,0,Tahun,135001240,135001240,271.0
1,2,Kab. Tabanan,350,0,Tidak ada,124,2024,0,Tahun,235001240,235001240,4479.0
2,3,Kab. Badung,350,0,Tidak ada,124,2024,0,Tahun,335001240,335001240,116.0
3,4,Kab. Gianyar,350,0,Tidak ada,124,2024,0,Tahun,435001240,435001240,42.0
4,5,Kab. Klungkung,350,0,Tidak ada,124,2024,0,Tahun,535001240,535001240,47.0


In [143]:
# ================= 1) Ambil & bereskan =================
d = data_350[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [144]:
wide # ilngain satu 0 dibelakang

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,289.0,286.0,213.0,228.0,244.0,274.0,279.0,282.0,284.0,271.0
Kab. Tabanan,6109.0,6101.0,5323.0,5212.0,5521.0,5527.0,5588.0,5708.0,5062.0,4479.0
Kab. Badung,218.0,200.0,195.0,191.0,164.0,163.0,137.0,136.0,113.0,116.0
Kab. Gianyar,93.0,92.0,52.0,61.0,61.0,61.0,65.0,44.0,39.0,42.0
Kab. Klungkung,25.0,24.0,20.0,22.0,17.0,18.0,10.0,7.0,4.0,47.0
Kab. Bangli,102.0,136.0,100.0,116.0,81.0,84.0,75.0,58.0,58.0,6.0
Kab. Karangasem,245.0,217.0,208.0,223.0,201.0,154.0,132.0,138.0,119.0,107.0
Kab. Buleleng,7550.0,6027.0,3988.0,5387.0,4830.0,4830.0,5379.0,5285.0,5255.0,4855.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [145]:
wide.reset_index().to_excel("data_350.xlsx", index=False, engine="openpyxl")

351 (Produksi Cengkeh Menurut Kabupaten/Kota di Provinsi Bali)

In [133]:
data_351 = getDataByVarId(domain='5100', varId='351')
data_351.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,351,0,Tidak ada,124,2024,0,Tahun,135101240,135101240,193.0
1,2,Kab. Tabanan,351,0,Tidak ada,124,2024,0,Tahun,235101240,235101240,349.0
2,3,Kab. Badung,351,0,Tidak ada,124,2024,0,Tahun,335101240,335101240,38.0
3,4,Kab. Gianyar,351,0,Tidak ada,124,2024,0,Tahun,435101240,435101240,1.0
4,5,Kab. Klungkung,351,0,Tidak ada,124,2024,0,Tahun,535101240,535101240,8.0


In [141]:
# ================= 1) Ambil & bereskan =================
d = data_351[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [135]:
wide # ilangin satu 0 dibelakang

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,692.0,683.0,144.0,605.0,660.0,658.0,616.0,618.0,392.0,193.0
Kab. Tabanan,711.0,668.0,1.0,439.0,423.0,453.0,445.0,158.0,171.0,349.0
Kab. Badung,32.0,32.0,30.0,38.0,31.0,29.0,35.0,34.0,37.0,38.0
Kab. Gianyar,27.0,27.0,22.0,20.0,16.0,18.0,19.0,4.0,3.0,1.0
Kab. Klungkung,117.0,89.0,77.0,76.0,70.0,66.0,33.0,18.0,6.0,8.0
Kab. Bangli,49.0,20.0,0.0,29.0,21.0,21.0,20.0,14.0,13.0,16.0
Kab. Karangasem,213.0,224.0,188.0,108.0,137.0,155.0,5.0,48.0,35.0,64.0
Kab. Buleleng,4033.0,2355.0,251.0,1978.0,1750.0,2174.0,2288.0,2298.0,2073.0,1673.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [136]:
wide.reset_index().to_excel("data_351.xlsx", index=False, engine="openpyxl")

352 (Produksi Kakao Menurut Kabupaten/Kota di Provinsi Bali)

In [68]:
data_352 = getDataByVarId(domain='5100', varId='352')
data_352.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,352,0,Tidak ada,124,2024,0,Tahun,135201240,135201240,3259
1,2,Kab. Tabanan,352,0,Tidak ada,124,2024,0,Tahun,235201240,235201240,937
2,3,Kab. Badung,352,0,Tidak ada,124,2024,0,Tahun,335201240,335201240,37
3,4,Kab. Gianyar,352,0,Tidak ada,124,2024,0,Tahun,435201240,435201240,38
4,5,Kab. Klungkung,352,0,Tidak ada,124,2024,0,Tahun,535201240,535201240,1


In [69]:
# --- ambil & bereskan ---
data_352 = data_352[['vervar_label','tahun_label','content_label']].copy()
data_352['kabupaten/kota'] = (data_352['vervar_label'].astype(str)
    .str.replace(r'<.*?>','', regex=True)
    .str.replace(r'\s+',' ',  regex=True)
    .str.strip())

# urutan asli + paksa "Provinsi Bali" di paling akhir
order = data_352['kabupaten/kota'].drop_duplicates().tolist()
prov_alias = ['Provinsi Bali','Prov. Bali','Bali']
tail = [n for n in prov_alias if n in order]
order = [x for x in order if x not in tail] + tail

# --- siapkan untuk pivot ---
df = (data_352.rename(columns={'tahun_label':'tahun','content_label':'nilai'})
      [['kabupaten/kota','tahun','nilai']])

# tahun -> int
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# (opsional) nilai -> float kalau masih format BPS
def parse_bps(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if ',' in s: s = s.replace('.', '').replace(',', '.')
    else:        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan
df['nilai'] = df['nilai'].map(parse_bps)

# --- PIVOT: kolom = tahun (melebar) ---
wide = df.pivot_table(index='kabupaten/kota', columns='tahun',
                      values='nilai', aggfunc='first', sort=False)

# urut kolom & baris
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index,
            categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()


In [70]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kabupaten/kota,,,,,,,,,,
Kab. Jembrana,2745.0,2853.0,2689.0,2605.0,2942.0,3009.0,6341.0,2943.0,3112.0,3259.0
Kab. Tabanan,2111.0,2067.0,866.0,981.0,895.0,921.0,4530.0,1039.0,1049.0,937.0
Kab. Badung,182.0,188.0,99.0,65.0,88.0,78.0,455.0,60.0,82.0,37.0
Kab. Gianyar,194.0,141.0,146.0,115.0,107.0,107.0,292.0,57.0,46.0,38.0
Kab. Klungkung,36.0,25.0,24.0,22.0,22.0,22.0,42.0,4.0,2.0,1.0
Kab. Bangli,136.0,109.0,111.0,90.0,76.0,62.0,228.0,24.0,24.0,14.0
Kab. Karangasem,188.0,220.0,176.0,173.0,172.0,169.0,727.0,67.0,77.0,80.0
Kab. Buleleng,770.0,600.0,852.0,762.0,649.0,628.0,1261.0,542.0,504.0,446.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
wide.reset_index().to_excel("data_352.xlsx", index=False, engine="openpyxl")

353 (Produksi Tembakau Menurut Kabupaten/Kota di Provinsi Bali)

In [72]:
data_353 = getDataByVarId(domain='5100', varId='353')
data_353.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,353,0,Tidak ada,124,2024,0,Tahun,135301240,135301240,0
1,2,Kab. Tabanan,353,0,Tidak ada,124,2024,0,Tahun,235301240,235301240,0
2,3,Kab. Badung,353,0,Tidak ada,124,2024,0,Tahun,335301240,335301240,0
3,4,Kab. Gianyar,353,0,Tidak ada,124,2024,0,Tahun,435301240,435301240,135
4,5,Kab. Klungkung,353,0,Tidak ada,124,2024,0,Tahun,535301240,535301240,0


In [73]:
# --- ambil & bereskan ---
data_353 = data_353[['vervar_label','tahun_label','content_label']].copy()
data_353['kabupaten/kota'] = (data_353['vervar_label'].astype(str)
    .str.replace(r'<.*?>','', regex=True)
    .str.replace(r'\s+',' ',  regex=True)
    .str.strip())

# urutan asli + paksa "Provinsi Bali" di paling akhir
order = data_353['kabupaten/kota'].drop_duplicates().tolist()
prov_alias = ['Provinsi Bali','Prov. Bali','Bali']
tail = [n for n in prov_alias if n in order]
order = [x for x in order if x not in tail] + tail

# --- siapkan untuk pivot ---
df = (data_353.rename(columns={'tahun_label':'tahun','content_label':'nilai'})
      [['kabupaten/kota','tahun','nilai']])

# tahun -> int
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# (opsional) nilai -> float kalau masih format BPS
def parse_bps(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if ',' in s: s = s.replace('.', '').replace(',', '.')
    else:        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan
df['nilai'] = df['nilai'].map(parse_bps)

# --- PIVOT: kolom = tahun (melebar) ---
wide = df.pivot_table(index='kabupaten/kota', columns='tahun',
                      values='nilai', aggfunc='first', sort=False)

# urut kolom & baris
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index,
            categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()


In [74]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kabupaten/kota,,,,,,,,,,
Kab. Jembrana,30.0,14.0,19.0,0.0,11.0,20.0,3.0,1.0,0.0,0.0
Kab. Tabanan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kab. Badung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kab. Gianyar,210.0,225.0,250.0,303.0,256.0,181.0,198.0,152.0,121.0,135.0
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0
Kab. Bangli,12.0,11.0,12.0,10.0,4.0,0.0,22.0,14.0,13.0,9.0
Kab. Karangasem,14.0,44.0,65.0,38.0,47.0,68.0,31.0,26.0,20.0,20.0
Kab. Buleleng,491.0,806.0,867.0,787.0,842.0,59.0,50.0,80.0,90.0,95.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
wide.reset_index().to_excel("data_353.xlsx", index=False, engine="openpyxl")

354 (Produksi Jambu Mete Menurut Kabupaten/Kota di Provinsi Bali)

In [76]:
data_354 = getDataByVarId(domain='5100', varId='354')
data_354.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,354,0,Tidak ada,124,2024,0,Tahun,135401240,135401240,0
1,2,Kab. Tabanan,354,0,Tidak ada,124,2024,0,Tahun,235401240,235401240,0
2,3,Kab. Badung,354,0,Tidak ada,124,2024,0,Tahun,335401240,335401240,4
3,4,Kab. Gianyar,354,0,Tidak ada,124,2024,0,Tahun,435401240,435401240,0
4,5,Kab. Klungkung,354,0,Tidak ada,124,2024,0,Tahun,535401240,535401240,73


In [ ]:
# --- ambil & bereskan ---
data_354 = data_354[['vervar_label','tahun_label','content_label']].copy()
data_354['kabupaten/kota'] = (data_354['vervar_label'].astype(str)
    .str.replace(r'<.*?>','', regex=True)
    .str.replace(r'\s+',' ',  regex=True)
    .str.strip())

# urutan asli + paksa "Provinsi Bali" di paling akhir
order = data_354['kabupaten/kota'].drop_duplicates().tolist()
prov_alias = ['Provinsi Bali','Prov. Bali','Bali']
tail = [n for n in prov_alias if n in order]
order = [x for x in order if x not in tail] + tail

# --- siapkan untuk pivot ---
df = (data_354.rename(columns={'tahun_label':'tahun','content_label':'nilai'})
      [['kabupaten/kota','tahun','nilai']])

# tahun -> int
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# (opsional) nilai -> float kalau masih format BPS
def parse_bps(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if ',' in s: s = s.replace('.', '').replace(',', '.')
    else:        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan
df['nilai'] = df['nilai'].map(parse_bps)

# --- PIVOT: kolom = tahun (melebar) ---
wide = df.pivot_table(index='kabupaten/kota', columns='tahun',
                      values='nilai', aggfunc='first', sort=False)

# urut kolom & baris
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index,
            categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()


In [78]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kabupaten/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kab. Tabanan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kab. Badung,16.0,16.0,11.0,7.0,6.0,6.0,5.0,4.0,6.0,4.0
Kab. Gianyar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kab. Klungkung,60.0,60.0,59.0,59.0,68.0,66.0,68.0,69.0,70.0,73.0
Kab. Bangli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kab. Karangasem,3590.0,3621.0,3152.0,3145.0,3113.0,3064.0,2394.0,1808.0,1740.0,1794.0
Kab. Buleleng,632.0,570.0,285.0,474.0,481.0,488.0,495.0,414.0,350.0,331.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
wide.reset_index().to_excel("data_354.xlsx", index=False, engine="openpyxl")

355 (Produksi Enau/Aren Menurut Kabupaten/Kota di Provinsi Bali)

In [146]:
data_355 = getDataByVarId(domain='5100', varId='355')
data_355.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
1,2,Kab. Tabanan,355,0,Tidak ada,124,2024,0,Tahun,235501240,235501240,10.0
2,3,Kab. Badung,355,0,Tidak ada,124,2024,0,Tahun,335501240,335501240,3.0
3,4,Kab. Gianyar,355,0,Tidak ada,124,2024,0,Tahun,435501240,435501240,7.0
6,7,Kab. Karangasem,355,0,Tidak ada,124,2024,0,Tahun,735501240,735501240,7.0
7,8,Kab. Buleleng,355,0,Tidak ada,124,2024,0,Tahun,835501240,835501240,30.0


In [147]:
# ================= 1) Ambil & bereskan =================
d = data_355[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [148]:
wide # kelebihan satu 0 dibelakang harusnya 10; 3;...

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Tabanan,1403.0,3.0,1.0,2.0,3.0,10.0,10.0,9.0,99.0,10.0
Kab. Badung,4.0,4.0,4.0,4.0,4.0,9.0,1.0,3.0,4.0,3.0
Kab. Gianyar,0.0,0.0,0.0,0.0,3.0,5.0,6.0,5.0,7.0,7.0
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Bangli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Karangasem,5.0,4.0,34.0,5.0,8.0,7.0,6.0,2.0,5.0,7.0
Kab. Buleleng,16.0,6.0,31.0,40.0,42.0,30.0,30.0,29.0,29.0,30.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [149]:
wide.reset_index().to_excel("data_355.xlsx", index=False, engine="openpyxl")


356 (Produksi Panili Menurut Kabupaten/Kota di Provinsi Bali)

In [150]:
data_356 = getDataByVarId(domain='5100', varId='356')
data_356.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,356,0,Tidak ada,118,2018,0,Tahun,135601180,135601180,0.16
1,2,Kab. Tabanan,356,0,Tidak ada,118,2018,0,Tahun,235601180,235601180,0.00
2,3,Kab. Badung,356,0,Tidak ada,118,2018,0,Tahun,335601180,335601180,1.00
3,4,Kab. Gianyar,356,0,Tidak ada,118,2018,0,Tahun,435601180,435601180,0.81
4,5,Kab. Klungkung,356,0,Tidak ada,118,2018,0,Tahun,535601180,535601180,0.00


In [151]:
# ================= 1) Ambil & bereskan =================
d = data_356[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [152]:
wide # harusnya 0,...

tahun,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
kab/kota,,,,,,,,,,
Kab. Jembrana,10.48,10.53,40.20,43.76,2.26,0.96,0.03,0.02,0.02,0.16
Kab. Tabanan,7.52,0.69,0.00,2.40,1.36,0.64,0.73,0.21,0.00,0.00
Kab. Badung,2.00,1.60,1.84,1.93,1.93,0.00,0.00,0.06,0.01,1.00
Kab. Gianyar,2.30,2.24,2.07,2.21,2.25,2.25,0.83,1.23,0.81,0.81
Kab. Klungkung,0.36,0.16,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Kab. Bangli,0.35,0.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Kab. Karangasem,0.98,1.21,1.11,1.11,1.15,1.71,0.93,1.09,0.08,0.12
Kab. Buleleng,4.72,4.42,0.52,1.43,0.62,0.00,0.00,0.00,0.00,0.00
Kota Denpasar,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [153]:
wide.reset_index().to_excel("data_356.xlsx", index=False, engine="openpyxl")

187 (Produksi Buah Mangga Provinsi Bali Menurut Kabupaten/Kota)

In [154]:
data_187 = getDataByVarId(domain='5100', varId='187')
data_187.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,187,0,Tidak ada,124,2024,0,Tahun,118701240,118701240,4403
1,2,Kab. Tabanan,187,0,Tidak ada,124,2024,0,Tahun,218701240,218701240,333
2,3,Kab. Badung,187,0,Tidak ada,124,2024,0,Tahun,318701240,318701240,946
3,4,Kab. Gianyar,187,0,Tidak ada,124,2024,0,Tahun,418701240,418701240,2162
4,5,Kab. Klungkung,187,0,Tidak ada,124,2024,0,Tahun,518701240,518701240,402


In [155]:
# ================= 1) Ambil & bereskan =================
d = data_187[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [156]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,4352.0,3760.0,1495.0,84.0,1885.0,871.0,851.0,2796.0,1680.0,4403.0
Kab. Tabanan,174.0,75.0,93.0,466.0,1004.0,137.0,146.0,196.0,602.0,333.0
Kab. Badung,1064.0,329.0,1073.0,794.0,1032.0,964.0,1762.0,605.0,1525.0,946.0
Kab. Gianyar,1147.0,865.0,1998.0,4348.0,4698.0,9724.0,4083.0,2103.0,2239.0,2162.0
Kab. Klungkung,719.0,556.0,1174.0,1770.0,976.0,736.0,809.0,990.0,778.0,402.0
Kab. Bangli,3344.0,629.0,3463.0,1593.0,3885.0,5202.0,3937.0,1357.0,890.0,0.0
Kab. Karangasem,6580.0,14203.0,17373.0,15751.0,13409.0,11737.0,11281.0,10557.0,10691.0,11121.0
Kab. Buleleng,36792.0,21823.0,29554.0,36949.0,38714.0,27072.0,31442.0,29529.0,31856.0,41363.0
Kota Denpasar,154.0,152.0,302.0,372.0,92.0,1245.0,855.0,1005.0,1813.0,1345.0


In [157]:
wide.reset_index().to_excel("data_187.xlsx", index=False, engine="openpyxl")

200 (Produksi Buah Jeruk Provinsi Bali Menurut Kabupaten/Kota)

In [158]:
data_200 = getDataByVarId(domain='5100', varId='200')
data_200.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,200,0,Tidak ada,124,2024,0,Tahun,120001240,120001240,242
1,2,Kab. Tabanan,200,0,Tidak ada,124,2024,0,Tahun,220001240,220001240,6638
2,3,Kab. Badung,200,0,Tidak ada,124,2024,0,Tahun,320001240,320001240,1362
3,4,Kab. Gianyar,200,0,Tidak ada,124,2024,0,Tahun,420001240,420001240,25974
4,5,Kab. Klungkung,200,0,Tidak ada,124,2024,0,Tahun,520001240,520001240,19


In [162]:
# ================= 1) Ambil & bereskan =================
d = data_200[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [163]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,2129.0,905.0,860.0,452.0,91.0,138.0,97.0,116.0,154.0,242.0
Kab. Tabanan,801.0,71.0,564.0,1605.0,207.0,123.0,1340.0,429.0,2700.0,6638.0
Kab. Badung,1452.0,1321.0,495.0,1814.0,2660.0,3217.0,3307.0,2659.0,1861.0,1362.0
Kab. Gianyar,17194.0,14415.0,60095.0,114509.0,174509.0,351295.0,126101.0,39654.0,45237.0,25974.0
Kab. Klungkung,59.0,80.0,69.0,80.0,77.0,42.0,313.0,91.0,3.0,19.0
Kab. Bangli,96987.0,63426.0,101338.0,102051.0,168476.0,131587.0,104528.0,87011.0,73063.0,83799.0
Kab. Karangasem,430.0,265.0,574.0,291.0,368.0,420.0,386.0,418.0,433.0,442.0
Kab. Buleleng,10376.0,3766.0,5001.0,4772.0,3382.0,3560.0,5521.0,4657.0,7686.0,4867.0
Kota Denpasar,6.0,11.0,10.0,11.0,5.0,11.0,23.0,36.0,28.0,43.0


In [161]:
wide.reset_index().to_excel("data_200.xlsx", index=False, engine="openpyxl")

201 (Produksi Buah Pisang Provinsi Bali Menurut Kabupaten/Kota di Bali)

In [167]:
data_201 = getDataByVarId(domain='5100', varId='201')
data_201.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,201,0,Tidak ada,124,2024,0,Tahun,120101240,120101240,24209
1,2,Kab. Tabanan,201,0,Tidak ada,124,2024,0,Tahun,220101240,220101240,13278
2,3,Kab. Badung,201,0,Tidak ada,124,2024,0,Tahun,320101240,320101240,11545
3,4,Kab. Gianyar,201,0,Tidak ada,124,2024,0,Tahun,420101240,420101240,20980
4,5,Kab. Klungkung,201,0,Tidak ada,124,2024,0,Tahun,520101240,520101240,2651


In [168]:
# ================= 1) Ambil & bereskan =================
d = data_201[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [169]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,23053.0,23265.0,38849.0,11495.0,10606.0,19394.0,18672.0,18668.0,12877.0,24209.0
Kab. Tabanan,13781.0,8329.0,14211.0,19166.0,13722.0,34031.0,9421.0,16919.0,9617.0,13278.0
Kab. Badung,11860.0,4822.0,2074.0,6612.0,18063.0,20219.0,18917.0,18407.0,17423.0,11545.0
Kab. Gianyar,9692.0,8674.0,12728.0,12658.0,24281.0,59563.0,237082.0,14821.0,25083.0,20980.0
Kab. Klungkung,4227.0,2722.0,4584.0,7715.0,4036.0,4326.0,11322.0,13036.0,5241.0,2651.0
Kab. Bangli,88240.0,100040.0,151450.0,119445.0,115033.0,58287.0,43543.0,82485.0,29807.0,39333.0
Kab. Karangasem,17377.0,10717.0,17720.0,19299.0,19547.0,20521.0,21019.0,22172.0,23018.0,23570.0
Kab. Buleleng,21727.0,24473.0,31464.0,42179.0,26394.0,25632.0,22205.0,28657.0,24387.0,33984.0
Kota Denpasar,278.0,168.0,271.0,237.0,111.0,268.0,354.0,483.0,472.0,541.0


In [170]:
wide.reset_index().to_excel("data_201.xlsx", index=False, engine="openpyxl")

202 (Produksi Buah Salak Provinsi Bali Menurut Kabupaten/Kota)

In [171]:
data_202 = getDataByVarId(domain='5100', varId='202')
data_202.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,202,0,Tidak ada,124,2024,0,Tahun,120201240,120201240,49.0
1,2,Kab. Tabanan,202,0,Tidak ada,124,2024,0,Tahun,220201240,220201240,500.0
2,3,Kab. Badung,202,0,Tidak ada,124,2024,0,Tahun,320201240,320201240,846.0
3,4,Kab. Gianyar,202,0,Tidak ada,124,2024,0,Tahun,420201240,420201240,734.0
4,5,Kab. Klungkung,202,0,Tidak ada,124,2024,0,Tahun,520201240,520201240,0.0


In [172]:
# ================= 1) Ambil & bereskan =================
d = data_202[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [173]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,6.0,149.0,22.0,45.0,130.0,139.0,45.0,190.0,72.0,49.0
Kab. Tabanan,98.0,46.0,352.0,189.0,180.0,586.0,264.0,284.0,485.0,500.0
Kab. Badung,41.0,40.0,10.0,146.0,331.0,1740.0,1405.0,1405.0,1337.0,846.0
Kab. Gianyar,126.0,59.0,14.0,127.0,474.0,4569.0,2073.0,31.0,245.0,734.0
Kab. Klungkung,26.0,27.0,26.0,28.0,16.0,5.0,14.0,0.0,1.0,0.0
Kab. Bangli,2290.0,2067.0,2097.0,4205.0,2107.0,46440.0,800.0,272.0,742.0,0.0
Kab. Karangasem,24391.0,19531.0,19955.0,18622.0,22194.0,21774.0,22267.0,23970.0,24061.0,24972.0
Kab. Buleleng,225.0,303.0,125.0,241.0,208.0,135.0,213.0,225.0,113.0,175.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [174]:
wide.reset_index().to_excel("data_202.xlsx", index=False, engine="openpyxl")

203 (Produksi Buah Semangka Provinsi Bali Menurut Kabupaten/Kota)

In [175]:
data_203 = getDataByVarId(domain='5100', varId='203')
data_203.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,203,0,Tidak ada,123,2023,0,Tahun,120301230,120301230,7622.0
1,2,Kab. Tabanan,203,0,Tidak ada,123,2023,0,Tahun,220301230,220301230,4.0
2,3,Kab. Badung,203,0,Tidak ada,123,2023,0,Tahun,320301230,320301230,3563.0
3,4,Kab. Gianyar,203,0,Tidak ada,123,2023,0,Tahun,420301230,420301230,744.0
4,5,Kab. Klungkung,203,0,Tidak ada,123,2023,0,Tahun,520301230,520301230,567.0


In [176]:
# ================= 1) Ambil & bereskan =================
d = data_203[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [177]:
wide

tahun,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
kab/kota,,,,,,,,,,
Kab. Jembrana,9402.0,6999.0,10792.0,8777.0,2546.0,11123.0,10123.0,8813.0,14872.0,7622.0
Kab. Tabanan,2161.0,1938.0,195.0,320.0,472.0,614.0,565.0,104.0,146.0,4.0
Kab. Badung,7387.0,4171.0,2963.0,3866.0,132.0,1932.0,1904.0,684.0,5282.0,3563.0
Kab. Gianyar,1639.0,1577.0,796.0,762.0,1444.0,597.0,106.0,82.0,1699.0,744.0
Kab. Klungkung,336.0,283.0,313.0,455.0,96.0,38.0,570.0,0.0,128.0,567.0
Kab. Bangli,0.0,0.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,NaN
Kab. Karangasem,35.0,70.0,0.0,20.0,643.0,0.0,90.0,0.0,0.0,NaN
Kab. Buleleng,619.0,288.0,189.0,165.0,9662.0,73.0,56.0,106.0,311.0,548.0
Kota Denpasar,1006.0,14310.0,1104.0,10182.0,7.0,5342.0,2631.0,3029.0,2122.0,6278.0


In [178]:
wide.reset_index().to_excel("data_203.xlsx", index=False, engine="openpyxl")

204 (Produksi Buah Pepaya Provinsi Bali Menurut Kabupaten/Kota)

In [179]:
data_204 = getDataByVarId(domain='5100', varId='204')
data_204.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,204,0,Tidak ada,124,2024,0,Tahun,120401240,120401240,1432
1,2,Kab. Tabanan,204,0,Tidak ada,124,2024,0,Tahun,220401240,220401240,2690
2,3,Kab. Badung,204,0,Tidak ada,124,2024,0,Tahun,320401240,320401240,444
3,4,Kab. Gianyar,204,0,Tidak ada,124,2024,0,Tahun,420401240,420401240,4016
4,5,Kab. Klungkung,204,0,Tidak ada,124,2024,0,Tahun,520401240,520401240,65


In [180]:
# ================= 1) Ambil & bereskan =================
d = data_204[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [181]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,67.0,146.0,58.0,54.0,34.0,50.0,271.0,637.0,1495.0,1432.0
Kab. Tabanan,480.0,338.0,229.0,300.0,304.0,537.0,338.0,2800.0,2339.0,2690.0
Kab. Badung,461.0,349.0,260.0,893.0,1332.0,1410.0,768.0,1562.0,1190.0,444.0
Kab. Gianyar,2745.0,1232.0,1452.0,1592.0,1311.0,4005.0,6206.0,803.0,980.0,4016.0
Kab. Klungkung,210.0,305.0,436.0,878.0,338.0,279.0,175.0,150.0,105.0,65.0
Kab. Bangli,852.0,801.0,527.0,707.0,900.0,6671.0,1457.0,1378.0,3991.0,1201.0
Kab. Karangasem,786.0,689.0,647.0,604.0,1033.0,1118.0,1108.0,1167.0,1205.0,1190.0
Kab. Buleleng,3134.0,1202.0,2690.0,5224.0,1613.0,2681.0,2188.0,2773.0,4762.0,3968.0
Kota Denpasar,16.0,23.0,38.0,29.0,23.0,37.0,43.0,56.0,54.0,88.0


In [182]:
wide.reset_index().to_excel("data_204.xlsx", index=False, engine="openpyxl")

205 (Produksi Buah Rambutan Provinsi Bali Menurut Kabupaten/Kota di Bali)

In [183]:
data_205 = getDataByVarId(domain='5100', varId='205')
data_205.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,205,0,Tidak ada,123,2023,0,Tahun,120501230,120501230,564
1,2,Kab. Tabanan,205,0,Tidak ada,123,2023,0,Tahun,220501230,220501230,116
2,3,Kab. Badung,205,0,Tidak ada,123,2023,0,Tahun,320501230,320501230,483
3,4,Kab. Gianyar,205,0,Tidak ada,123,2023,0,Tahun,420501230,420501230,377
4,5,Kab. Klungkung,205,0,Tidak ada,123,2023,0,Tahun,520501230,520501230,63


In [184]:
# ================= 1) Ambil & bereskan =================
d = data_205[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [185]:
wide

tahun,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
kab/kota,,,,,,,,,,
Kab. Jembrana,96.0,5578.0,3022.0,41.0,2546.0,2434.0,472.0,1372.0,375.0,564.0
Kab. Tabanan,386.0,235.0,238.0,16.0,472.0,108.0,313.0,194.0,136.0,116.0
Kab. Badung,965.0,616.0,289.0,140.0,132.0,362.0,411.0,195.0,642.0,483.0
Kab. Gianyar,632.0,438.0,512.0,1426.0,1444.0,1852.0,2075.0,927.0,212.0,377.0
Kab. Klungkung,281.0,166.0,287.0,80.0,96.0,57.0,51.0,531.0,55.0,63.0
Kab. Bangli,262.0,784.0,805.0,222.0,28.0,45.0,25.0,31.0,36.0,437.0
Kab. Karangasem,796.0,437.0,631.0,428.0,643.0,677.0,697.0,685.0,731.0,760.0
Kab. Buleleng,12375.0,12442.0,8870.0,10568.0,9662.0,15908.0,11895.0,19425.0,16912.0,12171.0
Kota Denpasar,9.0,5.0,5.0,7.0,7.0,2.0,3.0,8.0,7.0,7.0


In [186]:
wide.reset_index().to_excel("data_205.xlsx", index=False, engine="openpyxl")

331 (Produksi Bawang Daun Provinsi Bali Menurut Kabupaten/Kota)

In [187]:
data_331 = getDataByVarId(domain='5100', varId='331')
data_331.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
1,2,Kab. Tabanan,331,0,Tidak ada,123,2023,0,Tahun,233101230,233101230,810.0
7,8,Kab. Buleleng,331,0,Tidak ada,123,2023,0,Tahun,833101230,833101230,8.0
9,10,<b>Provinsi Bali</b>,331,0,Tidak ada,123,2023,0,Tahun,1033101230,1033101230,818.0
0,1,Kab. Jembrana,331,0,Tidak ada,122,2022,0,Tahun,133101220,133101220,0.0
1,2,Kab. Tabanan,331,0,Tidak ada,122,2022,0,Tahun,233101220,233101220,774.0


In [188]:
# ================= 1) Ambil & bereskan =================
d = data_331[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [189]:
wide

tahun,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Tabanan,1369.0,1201.0,1785.0,606.0,1478.0,788.0,1156.0,1055.0,774.0,810.0
Kab. Badung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Gianyar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Bangli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Karangasem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Buleleng,135.0,57.0,27.0,56.0,37.0,36.0,0.0,0.0,0.0,8.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [190]:
wide.reset_index().to_excel("data_331.xlsx", index=False, engine="openpyxl")

332 (Produksi Bawang Merah Provinsi Bali Menurut Kabupaten/Kota)

In [191]:
data_332 = getDataByVarId(domain='5100', varId='332')
data_332.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,332,0,Tidak ada,124,2024,0,Tahun,133201240,133201240,0.0
1,2,Kab. Tabanan,332,0,Tidak ada,124,2024,0,Tahun,233201240,233201240,30.0
2,3,Kab. Badung,332,0,Tidak ada,124,2024,0,Tahun,333201240,333201240,172.0
3,4,Kab. Gianyar,332,0,Tidak ada,124,2024,0,Tahun,433201240,433201240,20.0
4,5,Kab. Klungkung,332,0,Tidak ada,124,2024,0,Tahun,533201240,533201240,0.0


In [192]:
# ================= 1) Ambil & bereskan =================
d = data_332[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [193]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,12.0,31.0,18.0,9.0,0.0,0.0,NaN,0.0
Kab. Tabanan,3.0,197.0,94.0,108.0,18.0,108.0,80.0,104.0,380.0,30.0
Kab. Badung,22.0,0.0,8.0,75.0,0.0,16.0,0.0,0.0,24.0,172.0
Kab. Gianyar,0.0,1.0,58.0,22.0,52.0,28.0,9.0,22.0,NaN,20.0
Kab. Klungkung,1.0,27.0,15.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0
Kab. Bangli,9556.0,17141.0,18736.0,22470.0,18995.0,13263.0,21434.0,30236.0,33431.0,32622.0
Kab. Karangasem,408.0,498.0,979.0,1053.0,96.0,446.0,1010.0,876.0,903.0,1825.0
Kab. Buleleng,158.0,160.0,396.0,437.0,419.0,275.0,526.0,156.0,359.0,329.0
Kota Denpasar,0.0,0.0,8.0,68.0,88.0,63.0,156.0,99.0,277.0,100.0


In [194]:
wide.reset_index().to_excel("data_332.xlsx", index=False, engine="openpyxl")

333 (Produksi Bawang Putih Provinsi Bali Menurut Kabupaten/Kota)

In [195]:
data_333 = getDataByVarId(domain='5100', varId='333')
data_333.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
1,2,Kab. Tabanan,333,0,Tidak ada,123,2023,0,Tahun,233301230,233301230,8.0
6,7,Kab. Karangasem,333,0,Tidak ada,123,2023,0,Tahun,733301230,733301230,79.0
7,8,Kab. Buleleng,333,0,Tidak ada,123,2023,0,Tahun,833301230,833301230,81.0
9,10,<b>Provinsi Bali</b>,333,0,Tidak ada,123,2023,0,Tahun,1033301230,1033301230,167.0
0,1,Kab. Jembrana,333,0,Tidak ada,122,2022,0,Tahun,133301220,133301220,0.0


In [196]:
# ================= 1) Ambil & bereskan =================
d = data_333[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [197]:
wide

tahun,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Tabanan,12.0,33.0,30.0,0.0,0.0,713.0,323.0,288.0,151.0,8.0
Kab. Badung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Gianyar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Bangli,0.0,0.0,0.0,0.0,0.0,411.0,917.0,183.0,0.0,NaN
Kab. Karangasem,19.0,48.0,11.0,0.0,108.0,0.0,0.0,0.0,0.0,79.0
Kab. Buleleng,1.0,13.0,0.0,0.0,0.0,438.0,89.0,569.0,538.0,81.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [198]:
wide.reset_index().to_excel("data_333.xlsx", index=False, engine="openpyxl")

334 (Produksi Buncis Provinsi Bali Menurut Kabupaten/Kota)

In [199]:
data_334 = getDataByVarId(domain='5100', varId='334')
data_334.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
1,2,Kab. Tabanan,334,0,Tidak ada,123,2023,0,Tahun,233401230,233401230,605.0
2,3,Kab. Badung,334,0,Tidak ada,123,2023,0,Tahun,333401230,333401230,627.0
3,4,Kab. Gianyar,334,0,Tidak ada,123,2023,0,Tahun,433401230,433401230,41.0
5,6,Kab. Bangli,334,0,Tidak ada,123,2023,0,Tahun,633401230,633401230,1340.0
6,7,Kab. Karangasem,334,0,Tidak ada,123,2023,0,Tahun,733401230,733401230,2319.0


In [200]:
# ================= 1) Ambil & bereskan =================
d = data_334[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [201]:
wide

tahun,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Tabanan,2164.0,1749.0,2339.0,2048.0,2169.0,1066.0,1681.0,1406.0,975.0,605.0
Kab. Badung,754.0,784.0,770.0,920.0,904.0,895.0,823.0,643.0,781.0,627.0
Kab. Gianyar,13.0,51.0,76.0,215.0,21.0,25.0,60.0,63.0,16.0,41.0
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Kab. Bangli,5579.0,2562.0,2194.0,1703.0,1802.0,2496.0,2163.0,2869.0,3506.0,1340.0
Kab. Karangasem,5376.0,3143.0,1945.0,1272.0,1972.0,3226.0,2067.0,2056.0,2446.0,2319.0
Kab. Buleleng,78.0,2.0,396.0,390.0,0.0,25.0,47.0,0.0,0.0,166.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [202]:
wide.reset_index().to_excel("data_334.xlsx", index=False, engine="openpyxl")

335 (Produksi Cabe Provinsi Bali Menurut Kabupaten/Kota)

In [203]:
data_335 = getDataByVarId(domain='5100', varId='335')
data_335.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,335,0,Tidak ada,124,2024,0,Tahun,133501240,133501240,151
1,2,Kab. Tabanan,335,0,Tidak ada,124,2024,0,Tahun,233501240,233501240,530
2,3,Kab. Badung,335,0,Tidak ada,124,2024,0,Tahun,333501240,333501240,1353
3,4,Kab. Gianyar,335,0,Tidak ada,124,2024,0,Tahun,433501240,433501240,769
4,5,Kab. Klungkung,335,0,Tidak ada,124,2024,0,Tahun,533501240,533501240,636


In [204]:
# ================= 1) Ambil & bereskan =================
d = data_335[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [205]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,40.0,85.0,165.0,58.0,86.0,124.0,389.0,230.0,135.0,151.0
Kab. Tabanan,3011.0,2551.0,5310.0,2405.0,901.0,1175.0,1371.0,841.0,1784.0,530.0
Kab. Badung,1613.0,1442.0,1701.0,2089.0,1956.0,1504.0,1119.0,1453.0,1030.0,1353.0
Kab. Gianyar,666.0,1066.0,1441.0,6169.0,1864.0,4919.0,404.0,192.0,202.0,769.0
Kab. Klungkung,6184.0,5243.0,1008.0,1161.0,2324.0,1725.0,1943.0,1828.0,603.0,636.0
Kab. Bangli,12025.0,11986.0,11177.0,16103.0,10545.0,8552.0,12967.0,7386.0,3841.0,5441.0
Kab. Karangasem,15770.0,14491.0,14522.0,12118.0,6074.0,8792.0,11015.0,11104.0,12393.0,17743.0
Kab. Buleleng,6075.0,14461.0,8822.0,5036.0,15088.0,16581.0,11705.0,11905.0,7550.0,3040.0
Kota Denpasar,1.0,0.0,18.0,16.0,6.0,9.0,8.0,8.0,67.0,26.0


In [206]:
wide.reset_index().to_excel("data_335.xlsx", index=False, engine="openpyxl")

336 (Produksi Kacang Panjang Provinsi Bali Menurut Kabupaten/Kota)

In [207]:
data_336 = getDataByVarId(domain='5100', varId='336')
data_336.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,336,0,Tidak ada,123,2023,0,Tahun,133601230,133601230,58.0
1,2,Kab. Tabanan,336,0,Tidak ada,123,2023,0,Tahun,233601230,233601230,38.0
2,3,Kab. Badung,336,0,Tidak ada,123,2023,0,Tahun,333601230,333601230,60.0
3,4,Kab. Gianyar,336,0,Tidak ada,123,2023,0,Tahun,433601230,433601230,16.0
4,5,Kab. Klungkung,336,0,Tidak ada,123,2023,0,Tahun,533601230,533601230,998.0


In [208]:
# ================= 1) Ambil & bereskan =================
d = data_336[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [209]:
wide

tahun,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
kab/kota,,,,,,,,,,
Kab. Jembrana,7.0,32.0,6.0,13.0,19.0,26.0,27.0,85.0,61.0,58.0
Kab. Tabanan,824.0,524.0,634.0,699.0,663.0,115.0,417.0,153.0,71.0,38.0
Kab. Badung,300.0,149.0,306.0,309.0,153.0,67.0,94.0,180.0,188.0,60.0
Kab. Gianyar,31.0,93.0,68.0,6.0,19.0,7.0,136.0,65.0,21.0,16.0
Kab. Klungkung,297.0,323.0,2127.0,246.0,236.0,236.0,677.0,1023.0,527.0,998.0
Kab. Bangli,116.0,104.0,139.0,142.0,225.0,68.0,139.0,20.0,0.0,55.0
Kab. Karangasem,2308.0,2104.0,3074.0,1508.0,1789.0,867.0,3145.0,4427.0,3830.0,3446.0
Kab. Buleleng,56.0,19.0,0.0,0.0,0.0,0.0,20.0,3.0,26.0,7.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [210]:
wide.reset_index().to_excel("data_336.xlsx", index=False, engine="openpyxl")

337 (Produksi Kangkung Provinsi Bali Menurut Kabupaten/Kota)

In [211]:
data_337 = getDataByVarId(domain='5100', varId='337')
data_337.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,337,0,Tidak ada,122,2022,0,Tahun,133701220,133701220,2
1,2,Kab. Tabanan,337,0,Tidak ada,122,2022,0,Tahun,233701220,233701220,38
2,3,Kab. Badung,337,0,Tidak ada,122,2022,0,Tahun,333701220,333701220,110
3,4,Kab. Gianyar,337,0,Tidak ada,122,2022,0,Tahun,433701220,433701220,29
4,5,Kab. Klungkung,337,0,Tidak ada,122,2022,0,Tahun,533701220,533701220,0


In [212]:
# ================= 1) Ambil & bereskan =================
d = data_337[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [213]:
wide

tahun,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0
Kab. Tabanan,180.0,174.0,270.0,66.0,101.0,55.0,22.0,52.0,22.0,38.0
Kab. Badung,982.0,2361.0,1729.0,1291.0,651.0,536.0,439.0,187.0,133.0,110.0
Kab. Gianyar,119.0,178.0,198.0,121.0,108.0,148.0,208.0,87.0,55.0,29.0
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kab. Bangli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kab. Karangasem,437.0,523.0,560.0,776.0,364.0,183.0,92.0,170.0,201.0,120.0
Kab. Buleleng,82.0,66.0,47.0,52.0,15.0,0.0,0.0,2.0,73.0,105.0
Kota Denpasar,3869.0,4577.0,3279.0,3515.0,6295.0,8168.0,5922.0,5865.0,5431.0,4854.0


In [214]:
wide.reset_index().to_excel("data_337.xlsx", index=False, engine="openpyxl")

338 (Produksi Kentang Provinsi Bali Menurut Kabupaten/Kota)

In [9]:
data_338 = getDataByVarId(domain='5100', varId='338')
data_338.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
1,2,Kab. Tabanan,338,0,Tidak ada,124,2024,0,Tahun,233801240,233801240,1.0
7,8,Kab. Buleleng,338,0,Tidak ada,124,2024,0,Tahun,833801240,833801240,137.0
9,10,<b>Provinsi Bali</b>,338,0,Tidak ada,124,2024,0,Tahun,1033801240,1033801240,138.0
1,2,Kab. Tabanan,338,0,Tidak ada,123,2023,0,Tahun,233801230,233801230,12.0
7,8,Kab. Buleleng,338,0,Tidak ada,123,2023,0,Tahun,833801230,833801230,228.0


In [219]:
# ================= 1) Ambil & bereskan =================
d = data_338[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [220]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Tabanan,1104.0,336.0,0.0,25.0,119.0,108.0,74.0,0.0,12.0,1.0
Kab. Badung,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Gianyar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Bangli,181.0,84.0,51.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Karangasem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Buleleng,648.0,251.0,373.0,111.0,89.0,268.0,40.0,66.0,228.0,137.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [221]:
wide.reset_index().to_excel("data_338.xlsx", index=False, engine="openpyxl")

339 (Produksi Kubis Provinsi Bali Menurut Kabupaten/Kota)

In [222]:
data_339 = getDataByVarId(domain='5100', varId='339')
data_339.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
1,2,Kab. Tabanan,339,0,Tidak ada,124,2024,0,Tahun,233901240,233901240,451.0
2,3,Kab. Badung,339,0,Tidak ada,124,2024,0,Tahun,333901240,333901240,72.0
3,4,Kab. Gianyar,339,0,Tidak ada,124,2024,0,Tahun,433901240,433901240,393.0
5,6,Kab. Bangli,339,0,Tidak ada,124,2024,0,Tahun,633901240,633901240,10173.0
6,7,Kab. Karangasem,339,0,Tidak ada,124,2024,0,Tahun,733901240,733901240,554.0


In [223]:
# ================= 1) Ambil & bereskan =================
d = data_339[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [224]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Tabanan,10288.0,8299.0,9582.0,12205.0,5640.0,4200.0,2854.0,2103.0,1686.0,451.0
Kab. Badung,493.0,632.0,386.0,396.0,306.0,216.0,360.0,396.0,279.0,72.0
Kab. Gianyar,460.0,469.0,188.0,60.0,123.0,118.0,57.0,24.0,134.0,393.0
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Bangli,29837.0,34716.0,24519.0,19736.0,10287.0,20223.0,28809.0,23588.0,20458.0,10173.0
Kab. Karangasem,2386.0,612.0,358.0,346.0,527.0,447.0,493.0,493.0,655.0,554.0
Kab. Buleleng,1743.0,1028.0,473.0,562.0,693.0,1314.0,6419.0,7126.0,1518.0,802.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [225]:
wide.reset_index().to_excel("data_339.xlsx", index=False, engine="openpyxl")

340 (Produksi Mentimun Provinsi Bali Menurut Kabupaten/Kota)

In [226]:
data_340 = getDataByVarId(domain='5100', varId='340')
data_340.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,340,0,Tidak ada,124,2024,0,Tahun,134001240,134001240,165.0
1,2,Kab. Tabanan,340,0,Tidak ada,124,2024,0,Tahun,234001240,234001240,277.0
2,3,Kab. Badung,340,0,Tidak ada,124,2024,0,Tahun,334001240,334001240,887.0
3,4,Kab. Gianyar,340,0,Tidak ada,124,2024,0,Tahun,434001240,434001240,105.0
4,5,Kab. Klungkung,340,0,Tidak ada,124,2024,0,Tahun,534001240,534001240,2119.0


In [227]:
# ================= 1) Ambil & bereskan =================
d = data_340[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [228]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,18.0,8.0,29.0,7.0,122.0,99.0,172.0,309.0,165.0
Kab. Tabanan,3948.0,5310.0,4307.0,4116.0,2760.0,1969.0,2275.0,529.0,431.0,277.0
Kab. Badung,0.0,34.0,33.0,536.0,621.0,592.0,745.0,851.0,809.0,887.0
Kab. Gianyar,45.0,158.0,576.0,65.0,130.0,6.0,15.0,2.0,26.0,105.0
Kab. Klungkung,924.0,1151.0,2497.0,3188.0,2471.0,1552.0,2506.0,1513.0,1925.0,2119.0
Kab. Bangli,0.0,211.0,51.0,52.0,37.0,242.0,501.0,315.0,733.0,957.0
Kab. Karangasem,2666.0,891.0,968.0,946.0,557.0,719.0,1679.0,1284.0,1166.0,1173.0
Kab. Buleleng,0.0,9.0,0.0,0.0,0.0,4.0,17.0,18.0,169.0,523.0
Kota Denpasar,0.0,0.0,16.0,14.0,0.0,0.0,0.0,0.0,NaN,NaN


In [229]:
wide.reset_index().to_excel("data_340.xlsx", index=False, engine="openpyxl")

341 (Produksi Petsai/Sawi Provinsi Bali Menurut Kabupaten/Kota)

In [230]:
data_341 = getDataByVarId(domain='5100', varId='341')
data_341.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
1,2,Kab. Tabanan,341,0,Tidak ada,124,2024,0,Tahun,234101240,234101240,1348.0
2,3,Kab. Badung,341,0,Tidak ada,124,2024,0,Tahun,334101240,334101240,15.0
3,4,Kab. Gianyar,341,0,Tidak ada,124,2024,0,Tahun,434101240,434101240,18.0
4,5,Kab. Klungkung,341,0,Tidak ada,124,2024,0,Tahun,534101240,534101240,3740.0
5,6,Kab. Bangli,341,0,Tidak ada,124,2024,0,Tahun,634101240,634101240,1907.0


In [231]:
# ================= 1) Ambil & bereskan =================
d = data_341[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [232]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,12.0,20.0,3.0,NaN
Kab. Tabanan,7549.0,7833.0,9662.0,12738.0,5865.0,6189.0,4659.0,4735.0,4626.0,1348.0
Kab. Badung,128.0,0.0,0.0,23.0,0.0,0.0,13.0,89.0,9.0,15.0
Kab. Gianyar,25.0,1.0,8.0,8.0,13.0,113.0,39.0,1.0,1.0,18.0
Kab. Klungkung,3670.0,4578.0,6080.0,8965.0,10174.0,11585.0,3994.0,2783.0,3245.0,3740.0
Kab. Bangli,1818.0,3070.0,2698.0,2183.0,2195.0,2381.0,2663.0,2895.0,2398.0,1907.0
Kab. Karangasem,7657.0,8165.0,6785.0,4652.0,5427.0,5469.0,8002.0,7038.0,7456.0,7101.0
Kab. Buleleng,362.0,483.0,72.0,134.0,216.0,202.0,2021.0,1848.0,161.0,12.0
Kota Denpasar,5393.0,5437.0,5569.0,5489.0,4431.0,3113.0,3117.0,2964.0,2711.0,2667.0


In [233]:
wide.reset_index().to_excel("data_341.xlsx", index=False, engine="openpyxl")

342 (Produksi Terung Provinsi Bali Menurut Kabupaten/Kota)

In [234]:
data_342 = getDataByVarId(domain='5100', varId='342')
data_342.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,342,0,Tidak ada,123,2023,0,Tahun,134201230,134201230,95.0
1,2,Kab. Tabanan,342,0,Tidak ada,123,2023,0,Tahun,234201230,234201230,53.0
2,3,Kab. Badung,342,0,Tidak ada,123,2023,0,Tahun,334201230,334201230,170.0
3,4,Kab. Gianyar,342,0,Tidak ada,123,2023,0,Tahun,434201230,434201230,43.0
4,5,Kab. Klungkung,342,0,Tidak ada,123,2023,0,Tahun,534201230,534201230,103.0


In [235]:
# ================= 1) Ambil & bereskan =================
d = data_342[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [236]:
wide

tahun,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,15.0,15.0,20.0,1.0,12.0,56.0,57.0,60.0,95.0
Kab. Tabanan,138.0,44.0,22.0,26.0,14.0,10.0,68.0,0.0,7.0,53.0
Kab. Badung,0.0,0.0,30.0,0.0,0.0,0.0,144.0,182.0,373.0,170.0
Kab. Gianyar,29.0,18.0,13.0,117.0,88.0,110.0,42.0,168.0,23.0,43.0
Kab. Klungkung,688.0,279.0,13.0,69.0,24.0,42.0,11.0,0.0,18.0,103.0
Kab. Bangli,62.0,93.0,367.0,603.0,797.0,946.0,1880.0,762.0,492.0,665.0
Kab. Karangasem,268.0,125.0,1122.0,464.0,415.0,1208.0,727.0,390.0,219.0,452.0
Kab. Buleleng,10.0,0.0,5.0,0.0,0.0,322.0,0.0,0.0,0.0,78.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [237]:
wide.reset_index().to_excel("data_342.xlsx", index=False, engine="openpyxl")

343 (Produksi Tomat Provinsi Bali Menurut Kabupaten/Kota)

In [238]:
data_343 = getDataByVarId(domain='5100', varId='343')
data_343.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,343,0,Tidak ada,124,2024,0,Tahun,134301240,134301240,168.0
1,2,Kab. Tabanan,343,0,Tidak ada,124,2024,0,Tahun,234301240,234301240,220.0
2,3,Kab. Badung,343,0,Tidak ada,124,2024,0,Tahun,334301240,334301240,262.0
3,4,Kab. Gianyar,343,0,Tidak ada,124,2024,0,Tahun,434301240,434301240,66.0
4,5,Kab. Klungkung,343,0,Tidak ada,124,2024,0,Tahun,534301240,534301240,62.0


In [239]:
# ================= 1) Ambil & bereskan =================
d = data_343[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [240]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,12.0,70.0,16.0,41.0,98.0,32.0,168.0
Kab. Tabanan,6367.0,9398.0,15435.0,17298.0,6278.0,3162.0,2565.0,396.0,528.0,220.0
Kab. Badung,1366.0,956.0,1003.0,1022.0,1023.0,672.0,217.0,333.0,273.0,262.0
Kab. Gianyar,4.0,0.0,8.0,2.0,3.0,6.0,24.0,4.0,13.0,66.0
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,62.0
Kab. Bangli,3588.0,8428.0,4191.0,4646.0,5798.0,7845.0,7199.0,2021.0,2121.0,1794.0
Kab. Karangasem,4637.0,3195.0,1905.0,1838.0,1808.0,1392.0,1215.0,2167.0,1430.0,1525.0
Kab. Buleleng,754.0,2829.0,1978.0,785.0,191.0,717.0,910.0,985.0,821.0,284.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [241]:
wide.reset_index().to_excel("data_343.xlsx", index=False, engine="openpyxl")

344 (Produksi Wortel Provinsi Bali Menurut Kabupaten/Kota)

In [242]:
data_344 = getDataByVarId(domain='5100', varId='344')
data_344.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
1,2,Kab. Tabanan,344,0,Tidak ada,124,2024,0,Tahun,234401240,234401240,914.0
7,8,Kab. Buleleng,344,0,Tidak ada,124,2024,0,Tahun,834401240,834401240,571.0
9,10,<b>Provinsi Bali</b>,344,0,Tidak ada,124,2024,0,Tahun,1034401240,1034401240,1485.0
1,2,Kab. Tabanan,344,0,Tidak ada,123,2023,0,Tahun,234401230,234401230,865.0
7,8,Kab. Buleleng,344,0,Tidak ada,123,2023,0,Tahun,834401230,834401230,811.0


In [243]:
# ================= 1) Ambil & bereskan =================
d = data_344[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [244]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
kab/kota,,,,,,,,,,
Kab. Jembrana,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Tabanan,1325.0,2289.0,1400.0,2542.0,1174.0,1435.0,1524.0,1071.0,865.0,914.0
Kab. Badung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Gianyar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Klungkung,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Bangli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Karangasem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
Kab. Buleleng,1480.0,2043.0,850.0,1083.0,724.0,416.0,257.0,760.0,811.0,571.0
Kota Denpasar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [245]:
wide.reset_index().to_excel("data_344.xlsx", index=False, engine="openpyxl")

234 (Produksi Perikanan Menurut Kabupaten/Kota di Provinsi Bali)

In [246]:
data_234 = getDataByVarId(domain='5100', varId='234')
data_234.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,234,0,Tidak ada,122,2022,0,Tahun,123401220,123401220,24014.0
1,2,Kab. Tabanan,234,0,Tidak ada,122,2022,0,Tahun,223401220,223401220,3880.0
2,3,Kab. Badung,234,0,Tidak ada,122,2022,0,Tahun,323401220,323401220,8095.0
3,4,Kab. Gianyar,234,0,Tidak ada,122,2022,0,Tahun,423401220,423401220,1730.0
4,5,Kab. Klungkung,234,0,Tidak ada,122,2022,0,Tahun,523401220,523401220,17702.0


In [247]:
# ================= 1) Ambil & bereskan =================
d = data_234[['vervar_label','verval_val','tahun_label','content_label']].copy()

# Nama wilayah
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# Urutan baris berdasar kode wilayah (naik)
order_by_code = (
    d.groupby('kab/kota')['verval_val'].min().sort_values().index.tolist()
)

# Tahun numerik
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# ================= 2) Parser angka (smart) =================
def parse_bps_smart(s):
    """Bedakan titik sebagai ribuan vs desimal.
       Contoh: '1.234,56'->1234.56 ; '1.234'->1234 ; '10.0'->10.0 ; '3,5'->3.5
    """
    if pd.isna(s): return np.nan
    s = str(s).strip().replace('\xa0',' ')
    t = re.sub(r'[^0-9,.\-]', '', s)
    if t in {"", "-", ".", ",", "-.", ".-", "-,", ",-"}: 
        return np.nan

    if "," in t and "." in t:
        # Gaya ID/EU: 1.234,56
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "," in t:
        # Koma sebagai desimal
        t = t.replace(".", "").replace(",", ".")
        return float(t)

    if "." in t:
        parts = t.split(".")
        # Pola ribuan: 1.234 atau 1.234.567 (kelompok 3 digit)
        if len(parts) > 1 and all(len(p) == 3 for p in parts[1:]) and len(parts[0]) <= 3:
            return float("".join(parts))
        # Selain itu: titik sebagai desimal (10.0, 3.5, dst)
        return float(t)

    return float(t)

d['nilai'] = d['content_label'].map(parse_bps_smart)

# ================= 3) Pivot =================
wide = d.pivot_table(index='kab/kota',
                     columns='tahun',
                     values='nilai',
                     aggfunc='first',
                     sort=False)

# Urut kolom (tahun naik) & baris (kode wilayah)
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

# >>> wide sekarang sudah selaras dengan tabel biru (contoh 2024: 10, 3, 7, -, -, 7, 30, -, 58)


In [248]:
wide

tahun,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
kab/kota,,,,,,,,,,
Kab. Jembrana,15293.0,25365.6,22051.9,12089.7,7606.27,14840.3,28208.07,26797.0,22434.0,24014.0
Kab. Tabanan,4183.7,4784.1,3760.0,3665.7,3877.50,3967.4,3732.99,3851.0,3864.0,3880.0
Kab. Badung,49094.7,6620.5,7059.5,11628.1,6077.40,16970.1,5233.29,8546.0,9202.0,8095.0
Kab. Gianyar,2605.9,2638.8,1383.3,1388.0,1359.10,1394.6,1513.54,1492.0,1697.0,1730.0
Kab. Klungkung,103002.9,85791.5,108216.8,108027.9,1859.23,2575.4,2253.26,19243.0,18369.0,17702.0
Kab. Bangli,6148.5,7326.9,7266.9,7204.5,6021.80,5181.5,4844.13,4917.0,4910.0,4984.0
Kab. Karangasem,21394.4,22122.1,25371.3,24414.4,42207.40,27085.0,27056.67,26225.0,26465.0,23818.0
Kab. Buleleng,16304.9,19863.4,19148.5,19350.2,20125.51,20837.6,20660.45,14919.0,20237.0,18605.0
Kota Denpasar,45943.0,46193.6,34615.2,45759.3,40941.21,34680.2,18750.08,21705.0,34402.0,31029.0


In [249]:
wide.reset_index().to_excel("data_234.xlsx", index=False, engine="openpyxl")

330 (Nilai Produksi Perikanan Menurut Kabupaten/Kota di Provinsi Bali)

In [250]:
data_330 = getDataByVarId(domain='5100', varId='330')
data_330.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,330,0,Tidak ada,122,2022,0,Tahun,133001220,133001220,597849971
1,2,Kab. Tabanan,330,0,Tidak ada,122,2022,0,Tahun,233001220,233001220,107197840
2,3,Kab. Badung,330,0,Tidak ada,122,2022,0,Tahun,333001220,333001220,158589646
3,4,Kab. Gianyar,330,0,Tidak ada,122,2022,0,Tahun,433001220,433001220,62272543
4,5,Kab. Klungkung,330,0,Tidak ada,122,2022,0,Tahun,533001220,533001220,256287350


In [258]:
import pandas as pd, numpy as np, re

# --- ambil & bereskan ---
d = data_330[['vervar_label','verval_val','tahun_label','content_label']].copy()
d['kab/kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())

# urutan baris: kode wilayah (verval_val) naik
order_by_code = (d.groupby('kab/kota')['verval_val']
                   .min().sort_values().index.tolist())

# tahun numerik utk urutan kolom
d['tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['tahun']).copy()
d['tahun'] = d['tahun'].astype(int)

# NILAI: pakai persis content_label (rapikan tag/spasi saja)
d['nilai_str'] = (d['content_label'].astype(str)
                  .str.replace(r'<.*?>','', regex=True)
                  .str.replace(r'\s+',' ', regex=True)
                  .str.strip())

# pivot -> isi string sesuai content_label
wide = d.pivot_table(index='kab/kota', columns='tahun',
                     values='nilai_str', aggfunc='first', sort=False)

# urut kolom & baris
wide = wide.reindex(sorted([c for c in wide.columns if pd.notna(c)]), axis=1)
wide = wide.reindex([k for k in order_by_code if k in wide.index])

In [259]:
wide

tahun,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
kab/kota,,,,,,,,,,
Kab. Jembrana,151960769,294952617,186035957,158739330,286146521,276410136,330626487,324063457,287703410,597849971
Kab. Tabanan,83606750,100746600,76160987,77301689,24892240,95173873,96662590,99039372,100163115,107197840
Kab. Badung,128953515,120962928,110468396,212839328,136621232,521164055,160950870,196459415,171186830,158589646
Kab. Gianyar,79058000,82070840,41261663,38518843,14454500,33663742,38860840,41571751,48529355,62272543
Kab. Klungkung,136705825,159257915,151019824,148932102,26790984,37057772,47308545,102682558,263999228,256287350
Kab. Bangli,117133800,155833600,163722180,163379336,103095100,131063480,135360698,148744516,137101940,138625735
Kab. Karangasem,274133220,318965720,312713683,302999592,577319207,610664126,580821685,663594835,648495700,583641270
Kab. Buleleng,366806190,487345440,435133654,420385536,525493816,538972986,579295683,354961970,612644108,529675243
Kota Denpasar,986773650,1182635730,792448578,926271871,808468510,1049989073,788087935,977543947,943607030,875002422


In [260]:
wide.reset_index().to_excel("data_330.xlsx", index=False, engine="openpyxl")

78 (Banyaknya Pegawai Negeri Sipil di Provinsi Bali Menurut Pemerintah Daerah pada Akhir Tahun)

In [261]:
data_78 = getDataByVarId(domain='5100', varId='78')
data_78.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pemkab. Jembrana,78,41,Laki-laki,123,2023,0,Tahun,178411230,178411230,1354
1,1,Pemkab. Jembrana,78,42,Perempuan,123,2023,0,Tahun,178421230,178421230,1382
2,1,Pemkab. Jembrana,78,43,Laki-Laki + Perempuan,123,2023,0,Tahun,178431230,178431230,2736
3,2,Pemkab. Tabanan,78,41,Laki-laki,123,2023,0,Tahun,278411230,278411230,2413
4,2,Pemkab. Tabanan,78,42,Perempuan,123,2023,0,Tahun,278421230,278421230,3008


In [262]:
# ============= 1) Ambil & bersihkan dari API =============
d = data_78[['vervar_label','tahun_label','turvar_label','content_label']].copy()

# Nama kab/kota & urutan kemunculan
d['Kab/Kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Kab/Kota'].drop_duplicates().tolist()

# Label jenis kelamin dari API (tanpa mapping, cuma dirapikan)
d['JK'] = (d['turvar_label'].astype(str)
           .str.replace(r'<.*?>','', regex=True)
           .str.replace(r'\s+',' ', regex=True)
           .str.strip())

# Tahun numerik
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# Nilai numerik (support format BPS)
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s: s = s.replace('.', '').replace(',', '.')
    else:        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ============= 2) Pivot =============
wide = d.pivot_table(index='Kab/Kota',
                     columns=['Tahun','JK'],
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# Susun kolom per tahun mengikuti preferensi JK, tapi hanya yang ADA di data
years = sorted(d['Tahun'].unique())
jk_pref = ['Laki-laki', 'Perempuan', 'Laki-laki + Perempuan']  # pakai label API kalau ada
present = [j for j in jk_pref if j in wide.columns.get_level_values(1).unique()]
# kalau ada label lain dari API, ikutkan di belakang tanpa dihitung apa-apa
others  = [j for j in wide.columns.get_level_values(1).unique() if j not in present]
jk_order = present + others

full_cols = pd.MultiIndex.from_product([years, jk_order], names=['Tahun','JK'])
wide = wide.reindex(columns=full_cols)

# Urut baris sesuai kemunculan kab/kota
wide.index = pd.CategoricalIndex(wide.index,
                                 categories=[k for k in order_kab if k in wide.index],
                                 ordered=True)
wide = wide.sort_index()


In [263]:
wide

Tahun                   2017                                      2018  \
JK                 Laki-laki Perempuan Laki-Laki + Perempuan Laki-laki   
Kab/Kota                                                                 
Pemkab. Jembrana      2003.0    1629.0                3632.0    1846.0   
Pemkab. Tabanan       3908.0    3798.0                7706.0    3633.0   
Pemkab. Badung        4401.0    4097.0                8498.0    4209.0   
Pemkab. Gianyar       3513.0    3201.0                6714.0    3316.0   
Pemkab. Klungkung     2550.0    1874.0                4424.0    2423.0   
Pemkab. Bangli        3056.0    2222.0                5278.0    2930.0   
Pemkab. Karangasem    3999.0    2774.0                6773.0    3806.0   
Pemkab. Buleleng      4680.0    3683.0                8363.0    4381.0   
Pemkot. Denpasar      9745.0    8354.0               18099.0    9090.0   
Jumlah               37855.0   31632.0               69487.0   35634.0   

Tahun                                                   2019            \
JK                 Perempuan Laki-Laki + Perempuan Laki-laki Perempuan   
Kab/Kota                                                                 
Pemkab. Jembrana      1562.0                3408.0    1809.0    1608.0   
Pemkab. Tabanan       3647.0                7280.0    3572.0    3766.0   
Pemkab. Badung        4041.0                8250.0    4053.0    4159.0   
Pemkab. Gianyar       3085.0                6401.0    3199.0    3216.0   
Pemkab. Klungkung     1827.0                4250.0    2363.0    1871.0   
Pemkab. Bangli        2159.0                5089.0    2792.0    2092.0   
Pemkab. Karangasem    2706.0                6512.0    3723.0    2729.0   
Pemkab. Buleleng      3562.0                7943.0    4219.0    3618.0   
Pemkot. Denpasar      7924.0               17014.0    8749.0    8185.0   
Jumlah               30513.0               66147.0   34479.0   31244.0   

Tahun                                         2020  ...                        \
JK                 Laki-Laki + Perempuan Laki-laki  ... Laki-Laki + Perempuan   
Kab/Kota                                            ...                         
Pemkab. Jembrana                  3417.0    1671.0  ...                3208.0   
Pemkab. Tabanan                   7338.0    3307.0  ...                6915.0   
Pemkab. Badung                    8212.0    3719.0  ...                7628.0   
Pemkab. Gianyar                   6415.0    2980.0  ...                6077.0   
Pemkab. Klungkung                 4234.0    2279.0  ...                4143.0   
Pemkab. Bangli                    4884.0    2674.0  ...                4703.0   
Pemkab. Karangasem                6452.0    3497.0  ...                6137.0   
Pemkab. Buleleng                  7837.0    3897.0  ...                7348.0   
Pemkot. Denpasar                 16934.0    8220.0  ...               16226.0   
Jumlah                           65723.0   32244.0  ...               62385.0   

Tahun                   2021                                      2022  \
JK                 Laki-laki Perempuan Laki-Laki + Perempuan Laki-laki   
Kab/Kota                                                                 
Pemkab. Jembrana      1635.0    1541.0                3176.0    1494.0   
Pemkab. Tabanan       3026.0    3455.0                6481.0    2714.0   
Pemkab. Badung        3466.0    3738.0                7204.0    3309.0   
Pemkab. Gianyar       2729.0    2935.0                5664.0    2474.0   
Pemkab. Klungkung     2140.0    1780.0                3920.0    2073.0   
Pemkab. Bangli        2469.0    1966.0                4435.0    2347.0   
Pemkab. Karangasem    3281.0    2561.0                5842.0    3017.0   
Pemkab. Buleleng      3712.0    3505.0                7217.0    3479.0   
Pemkot. Denpasar      7878.0    7915.0               15793.0    7325.0   
Jumlah               30336.0   29396.0               59732.0   28232.0   

Tahun                                                   2

In [265]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {bln}" for yr, bln in wide_flat.columns]
wide_flat.reset_index().to_excel("data_78.xlsx", index=False, engine="openpyxl")

357 (Banyaknya Pegawai Negeri Sipil Menurut Tingkat Kepangkatan dan Jenis Kelamin di Provinsi Bali pada Akhir Tahun)

In [266]:
data_357 = getDataByVarId(domain='5100', varId='357')
data_357.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Golongan I,357,41,Laki-laki,124,2024,0,Tahun,1357411240,1357411240,175
1,1,Golongan I,357,42,Perempuan,124,2024,0,Tahun,1357421240,1357421240,64
2,1,Golongan I,357,43,Laki-Laki + Perempuan,124,2024,0,Tahun,1357431240,1357431240,239
3,2,Golongan II,357,41,Laki-laki,124,2024,0,Tahun,2357411240,2357411240,5560
4,2,Golongan II,357,42,Perempuan,124,2024,0,Tahun,2357421240,2357421240,3559


In [268]:
# ============= 1) Ambil & bersihkan dari API =============
d = data_357[['vervar_label','tahun_label','turvar_label','content_label']].copy()

# Nama kab/kota & urutan kemunculan
d['Golongan Kepangkatan'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Golongan Kepangkatan'].drop_duplicates().tolist()

# Label jenis kelamin dari API (tanpa mapping, cuma dirapikan)
d['JK'] = (d['turvar_label'].astype(str)
           .str.replace(r'<.*?>','', regex=True)
           .str.replace(r'\s+',' ', regex=True)
           .str.strip())

# Tahun numerik
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# Nilai numerik (support format BPS)
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s: s = s.replace('.', '').replace(',', '.')
    else:        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ============= 2) Pivot =============
wide = d.pivot_table(index='Golongan Kepangkatan',
                     columns=['Tahun','JK'],
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# Susun kolom per tahun mengikuti preferensi JK, tapi hanya yang ADA di data
years = sorted(d['Tahun'].unique())
jk_pref = ['Laki-laki', 'Perempuan', 'Laki-laki + Perempuan']  # pakai label API kalau ada
present = [j for j in jk_pref if j in wide.columns.get_level_values(1).unique()]
# kalau ada label lain dari API, ikutkan di belakang tanpa dihitung apa-apa
others  = [j for j in wide.columns.get_level_values(1).unique() if j not in present]
jk_order = present + others

full_cols = pd.MultiIndex.from_product([years, jk_order], names=['Tahun','JK'])
wide = wide.reindex(columns=full_cols)

# Urut baris sesuai kemunculan kab/kota
wide.index = pd.CategoricalIndex(wide.index,
                                 categories=[k for k in order_kab if k in wide.index],
                                 ordered=True)
wide = wide.sort_index()


In [269]:
wide

Tahun                     2017                                      2018  \
JK                   Laki-laki Perempuan Laki-Laki + Perempuan Laki-laki   
Golongan Kepangkatan                                                       
Golongan I              1002.0     301.0                1303.0     910.0   
Golongan II             8558.0    4781.0               13339.0    7993.0   
Golongan III           15424.0   16311.0               31735.0   14610.0   
Golongan IV            12871.0   10239.0               23110.0   12121.0   
Jumlah                 37855.0   31632.0               69487.0   35634.0   

Tahun                                                     2019            \
JK                   Perempuan Laki-Laki + Perempuan Laki-laki Perempuan   
Golongan Kepangkatan                                                       
Golongan I               281.0                1191.0     738.0     214.0   
Golongan II             4140.0               12133.0    7218.0    3747.0   
Golongan III           16185.0               30795.0   15177.0   17686.0   
Golongan IV             9907.0               22028.0   11346.0    9597.0   
Jumlah                 30513.0               66147.0   34479.0   31244.0   

Tahun                                           2020  ...  \
JK                   Laki-Laki + Perempuan Laki-laki  ...   
Golongan Kepangkatan                                  ...   
Golongan I                           952.0     638.0  ...   
Golongan II                        10965.0    6714.0  ...   
Golongan III                       32863.0   14552.0  ...   
Golongan IV                        20943.0   10340.0  ...   
Jumlah                             65723.0   32244.0  ...   

Tahun                                 2021      2022            \
JK                   Laki-Laki + Perempuan Laki-laki Perempuan   
Golongan Kepangkatan                                             
Golongan I                           607.0     337.0     129.0   
Golongan II                         9671.0    5469.0    3513.0   
Golongan III                       31382.0   13811.0   16697.0   
Golongan IV                        18072.0    8615.0    8045.0   
Jumlah                             59732.0   28232.0   28384.0   

Tahun                                           2023            \
JK                   Laki-Laki + Perempuan Laki-laki Perempuan   
Golongan Kepangkatan                                             
Golongan I                           466.0     235.0     105.0   
Golongan II                         8982.0    4581.0    2983.0   
Golongan III                       30508.0   13302.0   16174.0   
Golongan IV                        16660.0    7622.0    7336.0   
Jumlah                             56616.0   25740.0   26598.0   

Tahun                                           2024            \
JK                   Laki-Laki + Perempuan Laki-laki Perempuan   
Golongan Kepangkatan                                             
Golongan I                           340.0     175.0      64.0   
Golongan II                         7564.0    5560.0    3559.0   
Golongan III                       29476.0   19893.0   21488.0   
Golongan IV                        14958.0    8660.0    7931.0   
Jumlah                             52338.0   34288.0   33042.0   

Tahun                                       
JK                   Laki-Laki + Perempuan  
Golongan Kepangkatan                        
Golongan I                           239.0  
Golongan II                         9119.0  
Golongan III                       41381.0  
Golongan IV                        16591.0  
Jumlah                             67330.0  

[5 rows x 24 columns]

In [271]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {bln}" for yr, bln in wide_flat.columns]
wide_flat.reset_index().to_excel("data_357.xlsx", index=False, engine="openpyxl")

240 (Rata-rata Harga Eceran Emas Perhiasan di Kota Denpasar)

In [272]:
data_240 = getDataByVarId(domain='5100', varId='240')
data_240.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,18 Karat,240,0,Tidak ada,116,2016,0,Tahun,124001160,124001160,385638
1,2,22 Karat,240,0,Tidak ada,116,2016,0,Tahun,224001160,224001160,498715
2,3,24 Karat,240,0,Tidak ada,116,2016,0,Tahun,324001160,324001160,490059
0,1,18 Karat,240,0,Tidak ada,115,2015,0,Tahun,124001150,124001150,347521
1,2,22 Karat,240,0,Tidak ada,115,2015,0,Tahun,224001150,224001150,485477


In [273]:
# --- ambil & bersihkan untuk urutan asli ---
data_240 = data_240[['vervar_label','tahun_label','content_label']].copy()
data_240['Jenis Emas Perhiasan'] = (data_240['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_240['Jenis Emas Perhiasan'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_240.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Jenis Emas Perhiasan','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Jenis Emas Perhiasan', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [274]:
wide

tahun,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Jenis Emas Perhiasan,,,,,,,,,,
18 Karat,218102,247940,264882,264882,316615,348410,337737,340764,347521,385638
22 Karat,256604,308607,332544,332544,414925,480787,468989,476764,485477,498715
24 Karat,284393,343349,368935,368935,459346,540845,522325,519967,531554,490059


In [275]:
wide.reset_index().to_excel("data_240.xlsx", index=False, engine="openpyxl")

146 (PDRB Tahunan Provinsi Bali Atas Dasar Harga Berlaku Menurut Pengeluaran)

In [276]:
data_146 = getDataByVarId(domain='5100', varId='146')
data_146.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,146,0,Tidak ada,124,2024,0,Tahun,114601240,114601240,157246.97
1,2,Pengeluaran Konsumsi LNPRT,146,0,Tidak ada,124,2024,0,Tahun,214601240,214601240,5717.25
2,3,Pengeluaran Konsumsi Pemerintah,146,0,Tidak ada,124,2024,0,Tahun,314601240,314601240,29939.52
3,4,Pembentukan Modal Tetap Bruto,146,0,Tidak ada,124,2024,0,Tahun,414601240,414601240,82995.66
4,5,Perubahan Inventori,146,0,Tidak ada,124,2024,0,Tahun,514601240,514601240,191.91


In [277]:
# --- ambil & bersihkan untuk urutan asli ---
data_146 = data_146[['vervar_label','tahun_label','content_label']].copy()
data_146['Komponen Pengeluaran'] = (data_146['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_146['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_146.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Komponen Pengeluaran','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Komponen Pengeluaran', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [278]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Komponen Pengeluaran,,,,,,,,,,
Pengeluaran Konsumsi Rumah Tangga,85910.95,95497.69,102152.93,111762.44,121140.03,119957.69,122107.15,133351.88,145781.43,157246.97
Pengeluaran Konsumsi LNPRT,2197.12,2465.26,2580.41,2764.03,3163.69,3055.82,3185.25,3716.57,4386.59,5717.25
Pengeluaran Konsumsi Pemerintah,17750.68,19977.81,22603.58,24531.44,26712.14,27338.43,26143.25,26361.02,27578.69,29939.52
Pembentukan Modal Tetap Bruto,55333.04,60018.36,63293.06,71936.11,74886.65,67493.63,67166.54,72801.51,77493.22,82995.66
Perubahan Inventori,358.06,465.07,651.03,691.27,589.96,455.78,511.17,567.05,195.01,191.91
Net Ekspor Barang dan Jasa,14862.81,15665.39,21754.84,21951.48,25441.62,5924.36,1353.06,8569.86,18923.24,22350.21
a. Ekspor Barang dan Jasa Luar Negeri,63633.15,78332.45,86638.51,96781.56,97203.75,24877.04,7425.43,47274.75,97712.97,117272.46
b. Impor Barang dan Jasa Luar Negeri,13669.75,17637.50,20777.90,26079.89,22692.69,4777.88,863.88,6658.62,13622.71,16100.35
c. Net Ekspor Barang dan Jasa Antar Daerah,-35100.58,-45029.56,-44105.77,-48750.19,-49069.43,-14174.80,-5208.49,-32046.27,-65167.02,-78821.91


In [279]:
wide.reset_index().to_excel("data_146.xlsx", index=False, engine="openpyxl")

147 (PDRB Tahunan Provinsi Bali Atas Dasar Harga Konstan 2010 Menurut Pengeluaran)

In [280]:
data_147 = getDataByVarId(domain='5100', varId='147')
data_147.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,147,0,Tidak ada,124,2024,0,Tahun,114701240,114701240,93941.57
1,2,Pengeluaran Konsumsi LNPRT,147,0,Tidak ada,124,2024,0,Tahun,214701240,214701240,3307.93
2,3,Pengeluaran Konsumsi Pemerintah,147,0,Tidak ada,124,2024,0,Tahun,314701240,314701240,16884.04
3,4,Pembentukan Modal Tetap Bruto,147,0,Tidak ada,124,2024,0,Tahun,414701240,414701240,49708.02
4,5,Perubahan Inventori,147,0,Tidak ada,124,2024,0,Tahun,514701240,514701240,93.11


In [281]:
# --- ambil & bersihkan untuk urutan asli ---
data_147 = data_147[['vervar_label','tahun_label','content_label']].copy()
data_147['Komponen Pengeluaran'] = (data_147['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_147['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_147.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Komponen Pengeluaran','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Komponen Pengeluaran', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [282]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Komponen Pengeluaran,,,,,,,,,,
Pengeluaran Konsumsi Rumah Tangga,68660.37,73229.89,76190.69,80025.34,84688.99,81601.58,81726.57,85027.85,89763.49,93941.57
Pengeluaran Konsumsi LNPRT,1547.87,1680.65,1746.82,1864.02,2082.68,2000.72,2060.65,2288.63,2603.83,3307.93
Pengeluaran Konsumsi Pemerintah,12934.62,13756.97,14335.26,15131.40,16438.66,16453.63,15956.78,15701.45,15872.78,16884.04
Pembentukan Modal Tetap Bruto,41397.44,45030.74,46623.27,51076.08,52755.13,46598.91,44547.84,45712.96,47091.10,49708.02
Perubahan Inventori,176.19,224.56,239.28,246.51,229.51,210.49,268.99,289.15,97.53,93.11
Net Ekspor Barang dan Jasa,4410.08,3373.64,5798.00,5729.31,6498.39,633.63,-689.14,1810.74,4018.93,4251.37
a. Ekspor Barang dan Jasa Luar Negeri,45736.99,52314.23,56024.96,59447.66,59187.15,14121.60,4340.07,28410.21,57716.73,66947.78
b. Impor Barang dan Jasa Luar Negeri,9263.66,11369.06,13317.18,16568.49,14315.44,3123.42,549.12,4036.99,8027.54,9065.76
c. Net Ekspor Barang dan Jasa Antar Daerah,-32063.26,-37571.53,-36909.78,-37149.86,-38373.32,-10364.55,-4480.09,-22562.48,-45670.26,-53630.65


In [283]:
wide.reset_index().to_excel("data_147.xlsx", index=False, engine="openpyxl")

148 (Distribusi PDRB Tahunan Provinsi Bali Menurut Pengeluaran)

In [300]:
data_148 = getDataByVarId(domain='5100', varId='148')
data_148.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,148,0,Tidak ada,124,2024,0,Tahun,114801240,114801240,52.69
1,2,Pengeluaran Konsumsi LNPRT,148,0,Tidak ada,124,2024,0,Tahun,214801240,214801240,1.92
2,3,Pengeluaran Konsumsi Pemerintah,148,0,Tidak ada,124,2024,0,Tahun,314801240,314801240,10.03
3,4,Pembentukan Modal Tetap Bruto,148,0,Tidak ada,124,2024,0,Tahun,414801240,414801240,27.81
4,5,Perubahan Inventori,148,0,Tidak ada,124,2024,0,Tahun,514801240,514801240,0.06


In [301]:
# --- ambil & bersihkan untuk urutan asli ---
data_148 = data_148[['vervar_label','tahun_label','content_label']].copy()
data_148['Komponen Pengeluaran'] = (data_148['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_148['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_148.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Komponen Pengeluaran','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Komponen Pengeluaran', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [302]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Komponen Pengeluaran,,,,,,,,,,
Pengeluaran Konsumsi Rumah Tangga,48.70,49.20,47.95,47.84,48.08,53.50,55.39,54.35,53.14,52.69
Pengeluaran Konsumsi LNPRT,1.25,1.27,1.21,1.18,1.26,1.36,1.44,1.51,1.60,1.92
Pengeluaran Konsumsi Pemerintah,10.06,10.29,10.61,10.50,10.60,12.19,11.86,10.74,10.05,10.03
Pembentukan Modal Tetap Bruto,31.37,30.92,29.71,30.79,29.72,30.10,30.47,29.67,28.25,27.81
Perubahan Inventori,0.20,0.24,0.31,0.30,0.23,0.20,0.23,0.23,0.07,0.06
Net Ekspor Barang dan Jasa,8.43,8.07,10.21,9.40,10.10,2.64,0.61,3.49,6.90,7.49
a. Ekspor Barang dan Jasa Luar Negeri,36.07,40.36,40.67,41.42,38.58,11.09,3.37,19.27,35.62,39.29
b. Impor Barang dan Jasa Luar Negeri,7.75,9.09,9.75,11.16,9.01,2.13,0.39,2.71,4.97,5.39
c. Net Ekspor Barang dan Jasa Antar Daerah,-19.90,-23.20,-20.70,-20.87,-19.48,-6.32,-2.36,-13.06,-23.75,-26.41


In [ ]:
wide.reset_index().to_excel("data_148.xlsx", index=False, engine="openpyxl")

149 (Pertumbuhan PDRB/Ekonomi Tahunan Provinsi Bali Menurut Pengeluaran)

In [12]:
data_149 = getDataByVarId(domain='5100', varId='149')
data_149.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,149,0,Tidak ada,124,2024,0,Tahun,114901240,114901240,4.65
1,2,Pengeluaran Konsumsi LNPRT,149,0,Tidak ada,124,2024,0,Tahun,214901240,214901240,27.04
2,3,Pengeluaran Konsumsi Pemerintah,149,0,Tidak ada,124,2024,0,Tahun,314901240,314901240,6.37
3,4,Pembentukan Modal Tetap Bruto,149,0,Tidak ada,124,2024,0,Tahun,414901240,414901240,5.56
6,7,a. Ekspor Barang dan Jasa Luar Negeri,149,0,Tidak ada,124,2024,0,Tahun,714901240,714901240,15.99


In [13]:
# --- ambil & bersihkan untuk urutan asli ---
data_149 = data_149[['vervar_label','tahun_label','content_label']].copy()
data_149['Komponen Pengeluaran'] = (data_149['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_149['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_149.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Komponen Pengeluaran','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Komponen Pengeluaran', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [14]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Komponen Pengeluaran,,,,,,,,,,
Pengeluaran Konsumsi Rumah Tangga,7.46,6.66,4.04,5.03,5.83,-3.65,0.15,4.04,5.57,4.65
Pengeluaran Konsumsi LNPRT,3.17,8.58,3.94,6.71,11.73,-3.94,3.00,11.06,13.77,27.04
Pengeluaran Konsumsi Pemerintah,6.56,6.36,4.20,8.64,8.64,0.09,-3.02,-1.60,1.09,6.37
Pembentukan Modal Tetap Bruto,6.69,8.78,3.54,9.55,3.29,-11.67,-4.40,2.62,3.01,5.56
a. Ekspor Barang dan Jasa Luar Negeri,6.02,14.38,7.09,6.11,-0.44,-76.14,-69.27,554.60,103.15,15.99
b. Impor Barang dan Jasa Luar Negeri,-9.86,22.73,17.14,24.41,-13.60,-78.18,-82.42,635.18,98.85,12.93
PDRB,6.03,6.33,5.56,6.31,5.60,-9.34,-2.46,4.84,5.71,5.48


In [15]:
wide.reset_index().to_excel("data_149.xlsx", index=False, engine="openpyxl")

150 (PDRB Triwulanan Provinsi Bali Atas Dasar Harga Berlaku Menurut Pengeluaran)

In [16]:
data_150 = getDataByVarId(domain='5100', varId='150')
data_150.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,150,0,Tidak ada,125,2025,31,Triwulan I,1150012531,1150012531,41322.67
1,1,Pengeluaran Konsumsi Rumah Tangga,150,0,Tidak ada,125,2025,32,Triwulan II,1150012532,1150012532,42135.98
5,2,Pengeluaran Konsumsi LNPRT,150,0,Tidak ada,125,2025,31,Triwulan I,2150012531,2150012531,1472.76
6,2,Pengeluaran Konsumsi LNPRT,150,0,Tidak ada,125,2025,32,Triwulan II,2150012532,2150012532,1299.77
10,3,Pengeluaran Konsumsi Pemerintah,150,0,Tidak ada,125,2025,31,Triwulan I,3150012531,3150012531,5676.56


In [20]:
# --- Ambil & bersihkan untuk urutan asli ---
data_150 = data_150[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_150['Komponen Pengeluaran'] = (
    data_150['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_150['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_150['Triwulan'] = data_150['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_150.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Komponen Pengeluaran','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Komponen Pengeluaran',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [21]:
wide

Tahun                                            2016              \
Triwulan                                   Triwulan I Triwulan II   
Komponen Pengeluaran                                                
Pengeluaran Konsumsi Rumah Tangga            23043.63    23789.01   
Pengeluaran Konsumsi LNPRT                     564.88      610.14   
Pengeluaran Konsumsi Pemerintah               2616.29     5119.46   
Pembentukan Modal Tetap Bruto                14644.46    14826.97   
Perubahan Inventori                             99.13      100.20   
Net Ekspor Barang dan Jasa                    5040.23     3293.20   
a. Ekspor Barang dan Jasa Luar Negeri        18666.06    19463.75   
b. Impor Barang dan Jasa Luar Negeri          3878.76     3863.58   
c. Net Ekspor Barang dan Jasa Antar Daerah   -9747.07   -12306.97   
PDRB                                         46008.62    47738.99   

Tahun                                                                \
Triwulan                                   Triwulan III Triwulan IV   
Komponen Pengeluaran                                                  
Pengeluaran Konsumsi Rumah Tangga              24138.00    24527.05   
Pengeluaran Konsumsi LNPRT                       644.87      645.36   
Pengeluaran Konsumsi Pemerintah                 5143.88     7098.17   
Pembentukan Modal Tetap Bruto                  15014.86    15532.08   
Perubahan Inventori                              129.51      136.22   
Net Ekspor Barang dan Jasa                      4743.88     2588.08   
a. Ekspor Barang dan Jasa Luar Negeri          19927.61    20275.03   
b. Impor Barang dan Jasa Luar Negeri            4597.38     5297.79   
c. Net Ekspor Barang dan Jasa Antar Daerah    -10586.36   -12389.16   
PDRB                                           49815.01    50526.96   

Tahun                                                       2017              \
Triwulan                                      Tahunan Triwulan I Triwulan II   
Komponen Pengeluaran                                                           
Pengeluaran Konsumsi Rumah Tangga            95497.69   25043.13    25454.98   
Pengeluaran Konsumsi LNPRT                    2465.26     601.26      615.64   
Pengeluaran Konsumsi Pemerintah              19977.81    3448.95     5186.47   
Pembentukan Modal Tetap Bruto                60018.36   15204.93    15632.57   
Perubahan Inventori                            465.07     156.87      156.78   
Net Ekspor Barang dan Jasa                   15665.39    5910.50     5577.40   
a. Ekspor Barang dan Jasa Luar Negeri        78332.45   21183.48    22430.76   
b. Impor Barang dan Jasa Luar Negeri         17637.50    4784.83     5317.99   
c. Net Ekspor Barang dan Jasa Antar Daerah  -45029.56  -10488.15   -11535.37   
PDRB                                        194089.58   50365.63    52623.85   

Tahun                                                                \
Triwulan                                   Triwulan III Triwulan IV   
Komponen Pengeluaran                                                  
Pengeluaran Konsumsi Rumah Tangga              25728.46    25926.36   
Pengeluaran Konsumsi LNPRT                       647.64      715.88   
Pengeluaran Konsumsi Pemerintah                 5904.89     8063.28   
Pembentukan Modal Tetap Bruto                  15957.63    16497.93   
Perubahan Inventori                              168.72      168.66   
Net Ekspor Barang dan Jasa                      6730.94     3536.00   
a. Ekspor Barang dan Jasa Luar Negeri          22990.51    20033.76   
b. Impor Barang dan Jasa Luar Negeri            5331.46     5343.62   
c. Net Ekspor Barang dan Jasa Antar Daerah    -10928.11   -11154.13   
PDRB                                           55138.28    54908.10   

Tahun                                                  ...       2024  \
Triwulan                                      Tahunan  ... Triwulan I   
Komponen Pengeluaran                                   ...         

In [23]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_150.xlsx", index=False, engine="openpyxl")

151 (PDRB Triwulanan Provinsi Bali Atas Dasar Harga Konstan 2010 Menurut Pengeluaran)

In [24]:
data_151 = getDataByVarId(domain='5100', varId='151')
data_151.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,151,0,Tidak ada,125,2025,31,Triwulan I,1151012531,1151012531,24299.91
1,1,Pengeluaran Konsumsi Rumah Tangga,151,0,Tidak ada,125,2025,32,Triwulan II,1151012532,1151012532,24512.53
5,2,Pengeluaran Konsumsi LNPRT,151,0,Tidak ada,125,2025,31,Triwulan I,2151012531,2151012531,842.52
6,2,Pengeluaran Konsumsi LNPRT,151,0,Tidak ada,125,2025,32,Triwulan II,2151012532,2151012532,736.65
10,3,Pengeluaran Konsumsi Pemerintah,151,0,Tidak ada,125,2025,31,Triwulan I,3151012531,3151012531,3145.50


In [25]:
# --- Ambil & bersihkan untuk urutan asli ---
data_151 = data_151[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_151['Komponen Pengeluaran'] = (
    data_151['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_151['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_151['Triwulan'] = data_151['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_151.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Komponen Pengeluaran','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Komponen Pengeluaran',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [26]:
wide

Tahun                                            2016              \
Triwulan                                   Triwulan I Triwulan II   
Komponen Pengeluaran                                                
Pengeluaran Konsumsi Rumah Tangga            17870.67    18344.74   
Pengeluaran Konsumsi LNPRT                     386.80      417.24   
Pengeluaran Konsumsi Pemerintah               1839.35     3542.46   
Pembentukan Modal Tetap Bruto                10915.53    11146.04   
Perubahan Inventori                             54.90       48.75   
Net Ekspor Barang dan Jasa                    1817.05      447.56   
a. Ekspor Barang dan Jasa Luar Negeri        12552.87    13039.55   
b. Impor Barang dan Jasa Luar Negeri          2728.86     2564.94   
c. Net Ekspor Barang dan Jasa Antar Daerah   -8006.96   -10027.04   
PDRB                                         32884.32    33946.79   

Tahun                                                                \
Triwulan                                   Triwulan III Triwulan IV   
Komponen Pengeluaran                                                  
Pengeluaran Konsumsi Rumah Tangga              18408.37    18606.11   
Pengeluaran Konsumsi LNPRT                       436.82      439.78   
Pengeluaran Konsumsi Pemerintah                 3541.41     4833.75   
Pembentukan Modal Tetap Bruto                  11341.23    11627.93   
Perubahan Inventori                               61.61       59.30   
Net Ekspor Barang dan Jasa                      1211.52     -102.50   
a. Ekspor Barang dan Jasa Luar Negeri          13164.60    13557.20   
b. Impor Barang dan Jasa Luar Negeri            2801.14     3274.11   
c. Net Ekspor Barang dan Jasa Antar Daerah     -9151.94   -10385.59   
PDRB                                           35000.97    35464.37   

Tahun                                                       2017              \
Triwulan                                      Tahunan Triwulan I Triwulan II   
Komponen Pengeluaran                                                           
Pengeluaran Konsumsi Rumah Tangga            73229.89   18725.25    19028.40   
Pengeluaran Konsumsi LNPRT                    1680.65     409.45      417.76   
Pengeluaran Konsumsi Pemerintah              13756.97    2282.51     3246.21   
Pembentukan Modal Tetap Bruto                45030.74   11247.77    11571.80   
Perubahan Inventori                            224.56      60.11       58.37   
Net Ekspor Barang dan Jasa                    3373.64    2215.55     1633.38   
a. Ekspor Barang dan Jasa Luar Negeri        52314.23   13965.29    14512.10   
b. Impor Barang dan Jasa Luar Negeri         11369.06    3120.78     3410.35   
c. Net Ekspor Barang dan Jasa Antar Daerah  -37571.53   -8628.97    -9468.37   
PDRB                                        137296.45   34940.65    35955.93   

Tahun                                                                \
Triwulan                                   Triwulan III Triwulan IV   
Komponen Pengeluaran                                                  
Pengeluaran Konsumsi Rumah Tangga              19231.69    19205.34   
Pengeluaran Konsumsi LNPRT                       437.43      482.18   
Pengeluaran Konsumsi Pemerintah                 3684.86     5121.68   
Pembentukan Modal Tetap Bruto                  11776.59    12027.10   
Perubahan Inventori                               60.48       60.31   
Net Ekspor Barang dan Jasa                      1967.58      -18.51   
a. Ekspor Barang dan Jasa Luar Negeri          14672.47    12875.09   
b. Impor Barang dan Jasa Luar Negeri            3416.82     3369.23   
c. Net Ekspor Barang dan Jasa Antar Daerah     -9288.06    -9524.38   
PDRB                                           37158.63    36878.10   

Tahun                                                  ...       2024  \
Triwulan                                      Tahunan  ... Triwulan I   
Komponen Pengeluaran                                   ...         

In [27]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_151.xlsx", index=False, engine="openpyxl")

152 (Distribusi PDRB Triwulanan Provinsi Bali Menurut Pengeluaran)

In [28]:
data_152 = getDataByVarId(domain='5100', varId='152')
data_152.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,152,0,Tidak ada,125,2025,31,Triwulan I,1152012531,1152012531,54.72
1,1,Pengeluaran Konsumsi Rumah Tangga,152,0,Tidak ada,125,2025,32,Triwulan II,1152012532,1152012532,51.77
5,2,Pengeluaran Konsumsi LNPRT,152,0,Tidak ada,125,2025,31,Triwulan I,2152012531,2152012531,1.95
6,2,Pengeluaran Konsumsi LNPRT,152,0,Tidak ada,125,2025,32,Triwulan II,2152012532,2152012532,1.60
10,3,Pengeluaran Konsumsi Pemerintah,152,0,Tidak ada,125,2025,31,Triwulan I,3152012531,3152012531,7.52


In [29]:
# --- Ambil & bersihkan untuk urutan asli ---
data_152 = data_152[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_152['Komponen Pengeluaran'] = (
    data_151['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_152['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_152['Triwulan'] = data_152['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_152.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Komponen Pengeluaran','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Komponen Pengeluaran',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [30]:
wide

Tahun                                            2016              \
Triwulan                                   Triwulan I Triwulan II   
Komponen Pengeluaran                                                
Pengeluaran Konsumsi Rumah Tangga               50.09       49.83   
Pengeluaran Konsumsi LNPRT                       1.23        1.28   
Pengeluaran Konsumsi Pemerintah                  5.69       10.72   
Pembentukan Modal Tetap Bruto                   31.83       31.06   
Perubahan Inventori                              0.22        0.21   
Net Ekspor Barang dan Jasa                      10.95        6.90   
a. Ekspor Barang dan Jasa Luar Negeri           40.57       40.77   
b. Impor Barang dan Jasa Luar Negeri             8.43        8.09   
c. Net Ekspor Barang dan Jasa Antar Daerah     -21.19      -25.78   
PDRB                                           100.00      100.00   

Tahun                                                                        \
Triwulan                                   Triwulan III Triwulan IV Tahunan   
Komponen Pengeluaran                                                          
Pengeluaran Konsumsi Rumah Tangga                 48.46       48.54   49.20   
Pengeluaran Konsumsi LNPRT                         1.29        1.28    1.27   
Pengeluaran Konsumsi Pemerintah                   10.33       14.05   10.29   
Pembentukan Modal Tetap Bruto                     30.14       30.74   30.92   
Perubahan Inventori                                0.26        0.27    0.24   
Net Ekspor Barang dan Jasa                         9.52        5.12    8.07   
a. Ekspor Barang dan Jasa Luar Negeri             40.00       40.13   40.36   
b. Impor Barang dan Jasa Luar Negeri               9.23       10.49    9.09   
c. Net Ekspor Barang dan Jasa Antar Daerah       -21.25      -24.52  -23.20   
PDRB                                             100.00      100.00  100.00   

Tahun                                            2017              \
Triwulan                                   Triwulan I Triwulan II   
Komponen Pengeluaran                                                
Pengeluaran Konsumsi Rumah Tangga               49.72       48.37   
Pengeluaran Konsumsi LNPRT                       1.19        1.17   
Pengeluaran Konsumsi Pemerintah                  6.85        9.86   
Pembentukan Modal Tetap Bruto                   30.19       29.71   
Perubahan Inventori                              0.31        0.30   
Net Ekspor Barang dan Jasa                      11.74       10.60   
a. Ekspor Barang dan Jasa Luar Negeri           42.06       42.62   
b. Impor Barang dan Jasa Luar Negeri             9.50       10.11   
c. Net Ekspor Barang dan Jasa Antar Daerah     -20.82      -21.92   
PDRB                                           100.00      100.00   

Tahun                                                                        \
Triwulan                                   Triwulan III Triwulan IV Tahunan   
Komponen Pengeluaran                                                          
Pengeluaran Konsumsi Rumah Tangga                 46.66       47.22   47.95   
Pengeluaran Konsumsi LNPRT                         1.17        1.30    1.21   
Pengeluaran Konsumsi Pemerintah                   10.71       14.69   10.61   
Pembentukan Modal Tetap Bruto                     28.94       30.05   29.71   
Perubahan Inventori                                0.31        0.31    0.31   
Net Ekspor Barang dan Jasa                        12.21        6.44   10.21   
a. Ekspor Barang dan Jasa Luar Negeri             41.70       36.49   40.67   
b. Impor Barang dan Jasa Luar Negeri               9.67        9.73    9.75   
c. Net Ekspor Barang dan Jasa Antar Daerah       -19.82      -20.31  -20.70   
PDRB                                             100.00      100.00  100.00   

Tahun                                       ...       2024              \
Triwulan                                    ... Triwulan I Triwulan II   


In [31]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_152.xlsx", index=False, engine="openpyxl")

153 (Pertumbuhan PDRB Triwulanan Provinsi Bali (q to q) Menurut Pengeluaran)

In [38]:
data_153 = getDataByVarId(domain='5100', varId='153')
data_153.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,153,0,Tidak ada,125,2025,31,Triwulan I,1153012531,1153012531,1.23
1,1,Pengeluaran Konsumsi Rumah Tangga,153,0,Tidak ada,125,2025,32,Triwulan II,1153012532,1153012532,0.87
5,2,Pengeluaran Konsumsi LNPRT,153,0,Tidak ada,125,2025,31,Triwulan I,2153012531,2153012531,-4.89
6,2,Pengeluaran Konsumsi LNPRT,153,0,Tidak ada,125,2025,32,Triwulan II,2153012532,2153012532,-12.57
10,3,Pengeluaran Konsumsi Pemerintah,153,0,Tidak ada,125,2025,31,Triwulan I,3153012531,3153012531,-40.87


In [40]:
# --- Ambil & bersihkan untuk urutan asli ---
data_153 = data_153[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_153['Komponen Pengeluaran'] = (
    data_153['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_153['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_153['Triwulan'] = data_153['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_153.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Komponen Pengeluaran','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Komponen Pengeluaran',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [41]:
wide

Tahun                                       2016                           \
Triwulan                              Triwulan I Triwulan II Triwulan III   
Komponen Pengeluaran                                                        
Pengeluaran Konsumsi Rumah Tangga           0.46        2.65         0.35   
Pengeluaran Konsumsi LNPRT                 -8.47        7.87         4.69   
Pengeluaran Konsumsi Pemerintah           -58.54       92.59        -0.03   
Pembentukan Modal Tetap Bruto               0.64        2.11         1.75   
a. Ekspor Barang dan Jasa Luar Negeri       6.66        3.88         0.96   
b. Impor Barang dan Jasa Luar Negeri       11.43       -6.01         9.21   
PDRB                                       -1.70        3.23         3.11   

Tahun                                                           2017  \
Triwulan                              Triwulan IV Tahunan Triwulan I   
Komponen Pengeluaran                                                   
Pengeluaran Konsumsi Rumah Tangga            1.07     NaN       0.64   
Pengeluaran Konsumsi LNPRT                   0.68     NaN      -6.90   
Pengeluaran Konsumsi Pemerintah             36.49     NaN     -52.78   
Pembentukan Modal Tetap Bruto                2.53     NaN      -3.27   
a. Ekspor Barang dan Jasa Luar Negeri        2.98     NaN       3.01   
b. Impor Barang dan Jasa Luar Negeri        16.88     NaN      -4.68   
PDRB                                         1.32     NaN      -1.48   

Tahun                                                                       \
Triwulan                              Triwulan II Triwulan III Triwulan IV   
Komponen Pengeluaran                                                         
Pengeluaran Konsumsi Rumah Tangga            1.62         1.07       -0.14   
Pengeluaran Konsumsi LNPRT                   2.03         4.71       10.23   
Pengeluaran Konsumsi Pemerintah             42.22        13.51       38.99   
Pembentukan Modal Tetap Bruto                2.88         1.77        2.13   
a. Ekspor Barang dan Jasa Luar Negeri        3.92         1.11      -12.25   
b. Impor Barang dan Jasa Luar Negeri         9.28         0.19       -1.39   
PDRB                                         2.91         3.34       -0.75   

Tahun                                          ...       2024              \
Triwulan                              Tahunan  ... Triwulan I Triwulan II   
Komponen Pengeluaran                           ...                          
Pengeluaran Konsumsi Rumah Tangga         NaN  ...       0.67        0.65   
Pengeluaran Konsumsi LNPRT                NaN  ...      29.75      -35.50   
Pengeluaran Konsumsi Pemerintah           NaN  ...     -45.65       65.60   
Pembentukan Modal Tetap Bruto             NaN  ...      -9.50        5.15   
a. Ekspor Barang dan Jasa Luar Negeri     NaN  ...      -2.49       10.73   
b. Impor Barang dan Jasa Luar Negeri      NaN  ...       9.56        1.47   
PDRB                                      NaN  ...      -4.67        6.34   

Tahun                                                                   \
Triwulan                              Triwulan III Triwulan IV Tahunan   
Komponen Pengeluaran                                                     
Pengeluaran Konsumsi Rumah Tangga             1.76        1.57     NaN   
Pengeluaran Konsumsi LNPRT                    5.81       24.73     NaN   
Pengeluaran Konsumsi Pemerintah              -8.32       26.47     NaN   
Pembentukan Modal Tetap Bruto                 3.05        6.24     NaN   
a. Ekspor Barang dan Jasa Luar Negeri        12.20       -4.52     NaN   
b. Impor Barang dan Jasa Luar Negeri         -8.28       16.32     NaN   
PDRB                                          0.40        3.37     NaN   

Tahun                                       2025                           \
Triwulan                              Triwulan I Triwulan II Triwulan III   
Komponen Pengeluaran                                         

In [42]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_153.xlsx", index=False, engine="openpyxl")

154 (Pertumbuhan PDRB Triwulanan Provinsi Bali (y-o-y) Menurut Pengeluaran)

In [43]:
data_154 = getDataByVarId(domain='5100', varId='154')
data_154.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,154,0,Tidak ada,125,2025,31,Triwulan I,1154012531,1154012531,5.31
1,1,Pengeluaran Konsumsi Rumah Tangga,154,0,Tidak ada,125,2025,32,Triwulan II,1154012532,1154012532,5.54
5,2,Pengeluaran Konsumsi LNPRT,154,0,Tidak ada,125,2025,31,Triwulan I,2154012531,2154012531,-19.04
6,2,Pengeluaran Konsumsi LNPRT,154,0,Tidak ada,125,2025,32,Triwulan II,2154012532,2154012532,9.75
10,3,Pengeluaran Konsumsi Pemerintah,154,0,Tidak ada,125,2025,31,Triwulan I,3154012531,3154012531,13.54


In [44]:
# --- Ambil & bersihkan untuk urutan asli ---
data_154 = data_154[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_154['Komponen Pengeluaran'] = (
    data_154['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_154['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_154['Triwulan'] = data_154['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_154.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Komponen Pengeluaran','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Komponen Pengeluaran',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [45]:
wide

Tahun                                       2016                           \
Triwulan                              Triwulan I Triwulan II Triwulan III   
Komponen Pengeluaran                                                        
Pengeluaran Konsumsi Rumah Tangga           7.87        8.23         6.06   
Pengeluaran Konsumsi LNPRT                  6.72       12.17        11.76   
Pengeluaran Konsumsi Pemerintah             3.93       10.77         0.30   
Pembentukan Modal Tetap Bruto               9.52        9.49         9.00   
a. Ekspor Barang dan Jasa Luar Negeri      16.17       12.58        13.68   
b. Impor Barang dan Jasa Luar Negeri       22.63       12.25        21.56   
PDRB                                        6.22        6.53         6.55   

Tahun                                                           2017  \
Triwulan                              Triwulan IV Tahunan Triwulan I   
Komponen Pengeluaran                                                   
Pengeluaran Konsumsi Rumah Tangga            4.60    6.66       4.78   
Pengeluaran Konsumsi LNPRT                   4.07    8.58       5.85   
Pengeluaran Konsumsi Pemerintah              8.97    6.36      24.09   
Pembentukan Modal Tetap Bruto                7.21    8.78       3.04   
a. Ekspor Barang dan Jasa Luar Negeri       15.19   14.38      11.25   
b. Impor Barang dan Jasa Luar Negeri        33.69   22.73      14.36   
PDRB                                         6.02    6.33       6.25   

Tahun                                                                       \
Triwulan                              Triwulan II Triwulan III Triwulan IV   
Komponen Pengeluaran                                                         
Pengeluaran Konsumsi Rumah Tangga            3.73         4.47        3.22   
Pengeluaran Konsumsi LNPRT                   0.12         0.14        9.64   
Pengeluaran Konsumsi Pemerintah             -8.36         4.05        5.96   
Pembentukan Modal Tetap Bruto                3.82         3.84        3.43   
a. Ekspor Barang dan Jasa Luar Negeri       11.29        11.45       -5.03   
b. Impor Barang dan Jasa Luar Negeri        32.96        21.98        2.91   
PDRB                                         5.92         6.16        3.99   

Tahun                                          ...       2024              \
Triwulan                              Tahunan  ... Triwulan I Triwulan II   
Komponen Pengeluaran                           ...                          
Pengeluaran Konsumsi Rumah Tangga        4.04  ...       4.33        4.09   
Pengeluaran Konsumsi LNPRT               3.94  ...      80.93       10.67   
Pengeluaran Konsumsi Pemerintah          4.20  ...      19.69        2.41   
Pembentukan Modal Tetap Bruto            3.54  ...       8.23        8.55   
a. Ekspor Barang dan Jasa Luar Negeri    7.09  ...      24.33       17.50   
b. Impor Barang dan Jasa Luar Negeri    17.14  ...      16.94        6.50   
PDRB                                     5.56  ...       5.98        5.36   

Tahun                                                                   \
Triwulan                              Triwulan III Triwulan IV Tahunan   
Komponen Pengeluaran                                                     
Pengeluaran Konsumsi Rumah Tangga             5.46        4.73    4.65   
Pengeluaran Konsumsi LNPRT                   14.53       10.44   27.04   
Pengeluaran Konsumsi Pemerintah               5.66        4.36    6.37   
Pembentukan Modal Tetap Bruto                 1.92        4.19    5.56   
a. Ekspor Barang dan Jasa Luar Negeri         9.15       15.67   15.99   
b. Impor Barang dan Jasa Luar Negeri         10.01       18.61   12.93   
PDRB                                          5.43        5.19    5.48   

Tahun                                       2025                           \
Triwulan                              Triwulan I Triwulan II Triwulan III   
Komponen Pengeluaran                                         

In [46]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_154.xlsx", index=False, engine="openpyxl")

159 (Indeks Implisit PDRB Tahunan Provinsi Bali Menurut Pengeluaran)

In [51]:
data_159 = getDataByVarId(domain='5100', varId='159')
data_159.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,159,0,Tidak ada,124,2024,0,Tahun,115901240,115901240,167.39
1,2,Pengeluaran Konsumsi LNPRT,159,0,Tidak ada,124,2024,0,Tahun,215901240,215901240,172.83
2,3,Pengeluaran Konsumsi Pemerintah,159,0,Tidak ada,124,2024,0,Tahun,315901240,315901240,177.32
3,4,Pembentukan Modal Tetap Bruto,159,0,Tidak ada,124,2024,0,Tahun,415901240,415901240,166.97
4,5,Perubahan Inventori,159,0,Tidak ada,124,2024,0,Tahun,515901240,515901240,206.10


In [48]:
# --- ambil & bersihkan untuk urutan asli ---
data_159 = data_159[['vervar_label','tahun_label','content_label']].copy()
data_159['Komponen Pengeluaran'] = (data_159['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_159['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_159.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Komponen Pengeluaran','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Komponen Pengeluaran', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [49]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Komponen Pengeluaran,,,,,,,,,,
Pengeluaran Konsumsi Rumah Tangga,125.12,130.41,134.08,139.66,143.04,147.00,149.41,156.83,162.41,167.39
Pengeluaran Konsumsi LNPRT,141.94,146.68,147.72,148.28,151.90,152.74,154.58,162.39,168.47,172.83
Pengeluaran Konsumsi Pemerintah,137.23,145.22,157.68,162.12,162.50,166.15,163.84,167.89,173.75,177.32
Pembentukan Modal Tetap Bruto,133.66,133.28,135.75,140.84,141.95,144.84,150.77,159.26,164.56,166.97
Perubahan Inventori,203.23,207.10,272.08,280.42,257.05,216.53,190.03,196.11,199.95,206.10
a. Ekspor Barang dan Jasa Luar Negeri,139.13,149.73,154.64,162.80,164.23,176.16,171.09,166.40,169.30,175.17
b. Impor Barang dan Jasa Luar Negeri,147.56,155.14,156.02,157.41,158.52,152.97,157.32,164.94,169.70,177.60
c. Net Ekspor Barang dan Jasa Antar Daerah,109.47,119.85,119.50,131.23,127.87,136.76,116.26,142.03,142.69,146.97
PDRB,136.62,141.37,146.99,151.64,154.85,152.02,153.24,162.68,172.07,177.45


In [52]:
wide.reset_index().to_excel("data_159.xlsx", index=False, engine="openpyxl")

160 (Laju Implisit PDRB Tahunan Provinsi Bali Menurut Pengeluaran)

In [53]:
data_160 = getDataByVarId(domain='5100', varId='160')
data_160.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,160,0,Tidak ada,124,2024,0,Tahun,116001240,116001240,3.07
1,2,Pengeluaran Konsumsi LNPRT,160,0,Tidak ada,124,2024,0,Tahun,216001240,216001240,2.59
2,3,Pengeluaran Konsumsi Pemerintah,160,0,Tidak ada,124,2024,0,Tahun,316001240,316001240,2.06
3,4,Pembentukan Modal Tetap Bruto,160,0,Tidak ada,124,2024,0,Tahun,416001240,416001240,1.46
6,7,a. Ekspor Barang dan Jasa Luar Negeri,160,0,Tidak ada,124,2024,0,Tahun,716001240,716001240,3.47


In [56]:
# --- ambil & bersihkan untuk urutan asli ---
data_160 = data_160[['vervar_label','tahun_label','content_label']].copy()
data_160['Komponen Pengeluaran'] = (data_160['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_160['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_160.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Komponen Pengeluaran','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Komponen Pengeluaran', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [57]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Komponen Pengeluaran,,,,,,,,,,
Pengeluaran Konsumsi Rumah Tangga,4.55,4.22,2.81,4.16,2.42,2.77,1.64,4.97,3.55,3.07
Pengeluaran Konsumsi LNPRT,8.19,3.34,0.71,0.38,2.44,0.55,1.20,5.06,3.74,2.59
Pengeluaran Konsumsi Pemerintah,4.21,5.82,8.58,2.82,0.23,2.25,-1.39,2.47,3.49,2.06
Pembentukan Modal Tetap Bruto,6.61,-0.28,1.85,3.75,0.79,2.03,4.10,5.63,3.33,1.46
a. Ekspor Barang dan Jasa Luar Negeri,8.93,7.62,3.28,5.28,0.88,7.27,-2.88,-2.74,1.74,3.47
b. Impor Barang dan Jasa Luar Negeri,9.17,5.13,0.57,0.89,0.71,-3.50,2.84,4.84,2.89,4.65
PDRB,6.39,3.47,3.98,3.16,2.12,-1.83,0.80,6.16,5.77,3.13


In [58]:
wide.reset_index().to_excel("data_160.xlsx", index=False, engine="openpyxl")

163 (Indeks Implisit PDRB Triwulanan Provinsi Bali Menurut Pengeluaran)

In [59]:
data_163 = getDataByVarId(domain='5100', varId='163')
data_163.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,163,0,Tidak ada,125,2025,31,Triwulan I,1163012531,1163012531,170.05
1,1,Pengeluaran Konsumsi Rumah Tangga,163,0,Tidak ada,125,2025,32,Triwulan II,1163012532,1163012532,171.90
5,2,Pengeluaran Konsumsi LNPRT,163,0,Tidak ada,125,2025,31,Triwulan I,2163012531,2163012531,174.81
6,2,Pengeluaran Konsumsi LNPRT,163,0,Tidak ada,125,2025,32,Triwulan II,2163012532,2163012532,176.44
10,3,Pengeluaran Konsumsi Pemerintah,163,0,Tidak ada,125,2025,31,Triwulan I,3163012531,3163012531,180.47


In [60]:
# --- Ambil & bersihkan untuk urutan asli ---
data_163 = data_163[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_163['Komponen Pengeluaran'] = (
    data_163['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_163['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_163['Triwulan'] = data_163['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_163.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Komponen Pengeluaran','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Komponen Pengeluaran',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [61]:
wide

Tahun                                            2016              \
Triwulan                                   Triwulan I Triwulan II   
Komponen Pengeluaran                                                
Pengeluaran Konsumsi Rumah Tangga              128.95      129.68   
Pengeluaran Konsumsi LNPRT                     146.04      146.23   
Pengeluaran Konsumsi Pemerintah                142.24      144.52   
Pembentukan Modal Tetap Bruto                  134.16      133.02   
Perubahan Inventori                            180.57      205.54   
a. Ekspor Barang dan Jasa Luar Negeri          148.70      149.27   
b. Impor Barang dan Jasa Luar Negeri           142.14      150.63   
c. Net Ekspor Barang dan Jasa Antar Daerah     121.73      122.74   
PDRB                                           139.91      140.63   

Tahun                                                                        \
Triwulan                                   Triwulan III Triwulan IV Tahunan   
Komponen Pengeluaran                                                          
Pengeluaran Konsumsi Rumah Tangga                131.13      131.82  130.41   
Pengeluaran Konsumsi LNPRT                       147.63      146.74  146.68   
Pengeluaran Konsumsi Pemerintah                  145.25      146.85  145.22   
Pembentukan Modal Tetap Bruto                    132.39      133.58  133.28   
Perubahan Inventori                              210.21      229.74  207.10   
a. Ekspor Barang dan Jasa Luar Negeri            151.37      149.55  149.73   
b. Impor Barang dan Jasa Luar Negeri             164.13      161.81  155.14   
c. Net Ekspor Barang dan Jasa Antar Daerah       115.67      119.29  119.85   
PDRB                                             142.32      142.47  141.37   

Tahun                                            2017              \
Triwulan                                   Triwulan I Triwulan II   
Komponen Pengeluaran                                                
Pengeluaran Konsumsi Rumah Tangga              133.74      133.77   
Pengeluaran Konsumsi LNPRT                     146.85      147.37   
Pengeluaran Konsumsi Pemerintah                151.10      159.77   
Pembentukan Modal Tetap Bruto                  135.18      135.09   
Perubahan Inventori                            260.95      268.59   
a. Ekspor Barang dan Jasa Luar Negeri          151.69      154.57   
b. Impor Barang dan Jasa Luar Negeri           153.32      155.94   
c. Net Ekspor Barang dan Jasa Antar Daerah     121.55      121.83   
PDRB                                           144.15      146.36   

Tahun                                                                        \
Triwulan                                   Triwulan III Triwulan IV Tahunan   
Komponen Pengeluaran                                                          
Pengeluaran Konsumsi Rumah Tangga                133.78      135.00  134.08   
Pengeluaran Konsumsi LNPRT                       148.06      148.47  147.72   
Pengeluaran Konsumsi Pemerintah                  160.25      157.43  157.68   
Pembentukan Modal Tetap Bruto                    135.50      137.17  135.75   
Perubahan Inventori                              278.97      279.65  272.08   
a. Ekspor Barang dan Jasa Luar Negeri            156.69      155.60  154.64   
b. Impor Barang dan Jasa Luar Negeri             156.04      158.60  156.02   
c. Net Ekspor Barang dan Jasa Antar Daerah       117.66      117.11  119.50   
PDRB                                             148.39      148.89  146.99   

Tahun                                       ...       2024              \
Triwulan                                    ... Triwulan I Triwulan II   
Komponen Pengeluaran                        ...                          
Pengeluaran Konsumsi Rumah Tangga           ...     166.28      167.85   
Pengeluaran Konsumsi LNPRT                  ...     171.66      173.18   
Pengeluaran Konsumsi Pemerintah             ...     178.07      175.06   


In [62]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_163.xlsx", index=False, engine="openpyxl")

164 (Laju Implisit PDRB Triwulanan Provinsi Bali Menurut Pengeluaran)

In [63]:
data_164 = getDataByVarId(domain='5100', varId='164')
data_164.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pengeluaran Konsumsi Rumah Tangga,164,0,Tidak ada,125,2025,31,Triwulan I,1164012531,1164012531,1.08
1,1,Pengeluaran Konsumsi Rumah Tangga,164,0,Tidak ada,125,2025,32,Triwulan II,1164012532,1164012532,1.08
5,2,Pengeluaran Konsumsi LNPRT,164,0,Tidak ada,125,2025,31,Triwulan I,2164012531,2164012531,0.50
6,2,Pengeluaran Konsumsi LNPRT,164,0,Tidak ada,125,2025,32,Triwulan II,2164012532,2164012532,0.94
10,3,Pengeluaran Konsumsi Pemerintah,164,0,Tidak ada,125,2025,31,Triwulan I,3164012531,3164012531,0.52


In [64]:
# --- Ambil & bersihkan untuk urutan asli ---
data_164 = data_164[['vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_164['Komponen Pengeluaran'] = (
    data_164['vervar_label'].astype(str)
      .str.replace(r'<.*?>','', regex=True)
      .str.replace(r'\s+',' ', regex=True)
      .str.strip()
)

order = data_164['Komponen Pengeluaran'].drop_duplicates().tolist()

# --- Normalisasi triwulan ---
tw_map = {
    '1':'Triwulan I','I':'Triwulan I','Triwulan 1':'Triwulan I','Triwulan I':'Triwulan I',
    '2':'Triwulan II','II':'Triwulan II','Triwulan 2':'Triwulan II','Triwulan II':'Triwulan II',
    '3':'Triwulan III','III':'Triwulan III','Triwulan 3':'Triwulan III','Triwulan III':'Triwulan III',
    '4':'Triwulan IV','IV':'Triwulan IV','Triwulan 4':'Triwulan IV','Triwulan IV':'Triwulan IV',
    '0':'Tahunan','TAHUN':'Tahunan','Tahun':'Tahunan','Tahunan':'Tahunan'
}
data_164['Triwulan'] = data_164['turtahun_label'].astype(str).str.strip().map(lambda x: tw_map.get(x, x))

# --- siapkan untuk pivot ---
df = data_164.rename(columns={
    'tahun_label':'Tahun',
    'content_label':'Nilai'
})[['Komponen Pengeluaran','Tahun','Triwulan','Nilai']]

# Tahun numerik
df['Tahun'] = pd.to_numeric(df['Tahun'], errors='coerce')
df = df.dropna(subset=['Tahun']).copy()
df['Tahun'] = df['Tahun'].astype(int)

# --- PIVOT: kolom = (Tahun, Triwulan) ---
wide = df.pivot_table(
    index='Komponen Pengeluaran',
    columns=['Tahun','Triwulan'],
    values='Nilai',
    aggfunc='first',
    sort=False
)

# Urutkan kolom (tahun naik, lalu Triwulan I–IV, Tahunan)
years = sorted(df['Tahun'].unique())
tw_order = ['Triwulan I','Triwulan II','Triwulan III','Triwulan IV','Tahunan']
full_cols = pd.MultiIndex.from_product([years, tw_order], names=['Tahun','Triwulan'])
wide = wide.reindex(columns=full_cols)

# Urutkan baris sesuai urutan kemunculan asli
wide.index = pd.CategoricalIndex(
    wide.index,
    categories=[k for k in order if k in wide.index],
    ordered=True
)
wide = wide.sort_index()


In [65]:
wide

Tahun                                       2016                           \
Triwulan                              Triwulan I Triwulan II Triwulan III   
Komponen Pengeluaran                                                        
Pengeluaran Konsumsi Rumah Tangga           1.58        0.57         1.12   
Pengeluaran Konsumsi LNPRT                  1.62        0.13         0.95   
Pengeluaran Konsumsi Pemerintah             0.57        1.60         0.51   
Pembentukan Modal Tetap Bruto              -2.13       -0.85        -0.48   
a. Ekspor Barang dan Jasa Luar Negeri      -4.26        0.38         1.41   
b. Impor Barang dan Jasa Luar Negeri      -14.28        5.97         8.96   
PDRB                                        1.23        0.51         1.21   

Tahun                                                           2017  \
Triwulan                              Triwulan IV Tahunan Triwulan I   
Komponen Pengeluaran                                                   
Pengeluaran Konsumsi Rumah Tangga            0.53     NaN       1.45   
Pengeluaran Konsumsi LNPRT                  -0.60     NaN       0.07   
Pengeluaran Konsumsi Pemerintah              1.10     NaN       2.90   
Pembentukan Modal Tetap Bruto                0.89     NaN       1.20   
a. Ekspor Barang dan Jasa Luar Negeri       -1.20     NaN       1.43   
b. Impor Barang dan Jasa Luar Negeri        -1.41     NaN      -5.24   
PDRB                                         0.10     NaN       1.17   

Tahun                                                                       \
Triwulan                              Triwulan II Triwulan III Triwulan IV   
Komponen Pengeluaran                                                         
Pengeluaran Konsumsi Rumah Tangga            0.03         0.01        0.91   
Pengeluaran Konsumsi LNPRT                   0.35         0.47        0.28   
Pengeluaran Konsumsi Pemerintah              5.74         0.30       -1.76   
Pembentukan Modal Tetap Bruto               -0.07         0.30        1.23   
a. Ekspor Barang dan Jasa Luar Negeri        1.90         1.38       -0.70   
b. Impor Barang dan Jasa Luar Negeri         1.71         0.06        1.64   
PDRB                                         1.53         1.39        0.34   

Tahun                                          ...       2024              \
Triwulan                              Tahunan  ... Triwulan I Triwulan II   
Komponen Pengeluaran                           ...                          
Pengeluaran Konsumsi Rumah Tangga         NaN  ...       1.51        0.94   
Pengeluaran Konsumsi LNPRT                NaN  ...       1.09        0.89   
Pengeluaran Konsumsi Pemerintah           NaN  ...       0.57       -1.69   
Pembentukan Modal Tetap Bruto             NaN  ...      -0.08        0.94   
a. Ekspor Barang dan Jasa Luar Negeri     NaN  ...       1.10        1.83   
b. Impor Barang dan Jasa Luar Negeri      NaN  ...       1.11        1.08   
PDRB                                      NaN  ...       1.03        0.99   

Tahun                                                                   \
Triwulan                              Triwulan III Triwulan IV Tahunan   
Komponen Pengeluaran                                                     
Pengeluaran Konsumsi Rumah Tangga            -0.41        0.64     NaN   
Pengeluaran Konsumsi LNPRT                   -0.19        0.63     NaN   
Pengeluaran Konsumsi Pemerintah               0.83        1.71     NaN   
Pembentukan Modal Tetap Bruto                 0.86        0.02     NaN   
a. Ekspor Barang dan Jasa Luar Negeri        -0.15        1.01     NaN   
b. Impor Barang dan Jasa Luar Negeri         -0.91        0.86     NaN   
PDRB                                          0.88        0.12     NaN   

Tahun                                       2025                           \
Triwulan                              Triwulan I Triwulan II Triwulan III   
Komponen Pengeluaran                                         

In [66]:
wide_flat = wide.copy()
wide_flat.columns = [f"{th} - {tw}" for th, tw in wide_flat.columns]
wide_flat.reset_index().to_excel("data_164.xlsx", index=False, engine="openpyxl")

181 (Persentase Sumbangan Pendapatan Perempuan Provinsi Bali Menurut Kabupaten/Kota) Data Aghasi

In [74]:
data_181 = getDataByVarId(domain='5100', varId='181')
data_181.head()

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,181,0,Tidak ada,124,2024,0,Tahun,118101240,118101240,39.61
1,2,Kab. Tabanan,181,0,Tidak ada,124,2024,0,Tahun,218101240,218101240,39.38
2,3,Kab. Badung,181,0,Tidak ada,124,2024,0,Tahun,318101240,318101240,36.47
3,4,Kab. Gianyar,181,0,Tidak ada,124,2024,0,Tahun,418101240,418101240,37.86
4,5,Kab. Klungkung,181,0,Tidak ada,124,2024,0,Tahun,518101240,518101240,47.93


In [78]:
# --- ambil & bersihkan untuk urutan asli ---
data_181 = data_181[['vervar_label','tahun_label','content_label']].copy()
data_181['Kabupaten/Kota'] = (data_181['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_181['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_181.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [79]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,,,,,,
Kab. Jembrana,38.43,NaN,38.54,38.47,38.73,38.66,39.16,38.60,39.16,39.61
Kab. Tabanan,35.69,NaN,36.55,37.22,38.28,38.22,38.94,38.55,38.51,39.38
Kab. Badung,35.63,NaN,35.93,36.35,36.52,36.43,36.84,36.87,35.72,36.47
Kab. Gianyar,36.64,NaN,37.25,37.42,37.94,37.91,38.60,38.17,38.24,37.86
Kab. Klungkung,46.11,NaN,46.20,46.19,46.73,46.95,46.81,46.12,48.70,47.93
Kab. Bangli,37.27,NaN,37.28,37.27,38.19,38.21,38.41,38.10,37.00,36.58
Kab. Karangasem,42.40,NaN,42.45,43.02,44.11,44.22,43.93,43.69,44.79,43.90
Kab. Buleleng,38.88,NaN,38.89,38.93,39.29,39.34,40.04,40.02,39.04,39.55
Kota Denpasar,42.16,NaN,42.18,42.19,42.62,42.63,43.12,42.99,41.51,42.01


In [80]:
wide.reset_index().to_excel("data_181.xlsx", index=False, engine="openpyxl")

```FINISH```